In [ ]:
import numpy as np
import pandas as pd
import time
from itertools import permutations, combinations

import os
import json
from pathlib import Path

import matplotlib.pyplot as plt
from matplotlib import colors
from copy import deepcopy,copy

def copy_list(lista):
    copia = []
    for el in lista:
        copia.append(copy(el))
    return copia

from pathlib import Path

data_path = Path('/kaggle/input/abstraction-and-reasoning-challenge/')
training_path = data_path / 'training'
evaluation_path = data_path / 'evaluation'
test_path = data_path / 'test'
#nosound_path = Path('/kaggle/input/arc-nosound-tasks/')

training_tasks = sorted(os.listdir(training_path))
evaluation_tasks = sorted(os.listdir(evaluation_path))
test_tasks = sorted(os.listdir(test_path))
#nosound_tasks = sorted(os.listdir(nosound_path))

def plot_one(ax, i,train_or_test,input_or_output):
    cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
    norm = colors.Normalize(vmin=0, vmax=9)
    
    input_matrix = task[train_or_test][i][input_or_output]
    ax.imshow(input_matrix, cmap=cmap, norm=norm)
    ax.grid(True,which='both',color='lightgrey', linewidth=0.5)    
    ax.set_yticks([x-0.5 for x in range(1+len(input_matrix))])
    ax.set_xticks([x-0.5 for x in range(1+len(input_matrix[0]))])     
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_title(train_or_test + ' '+input_or_output)
    

def plot_task(task):
    """
    Plots the first train and test pairs of a specified task,
    using same color scheme as the ARC app
    """    
    num_train = len(task['train'])
    fig, axs = plt.subplots(2, num_train, figsize=(3*num_train,3*2))
    for i in range(num_train):     
        plot_one(axs[0,i],i,'train','input')
        plot_one(axs[1,i],i,'train','output')        
    plt.tight_layout()
    plt.show()        
        
    num_test = len(task['test'])
    fig, axs = plt.subplots(2, num_test, figsize=(3*num_test,3*2))
    if num_test==1: 
        plot_one(axs[0],0,'test','input')
        plot_one(axs[1],0,'test','output')     
    else:
        for i in range(num_test):      
            plot_one(axs[0,i],i,'test','input')
            plot_one(axs[1,i],i,'test','output')  
    plt.tight_layout()
    plt.show() 
    
    
def mostra_tab(tab):
    cmap = colors.ListedColormap(['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00','#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
    norm = colors.Normalize(vmin=0, vmax=9)
    fig, ax = plt.subplots(1, 1, figsize=(3,3))
    ax.imshow(tab, cmap=cmap, norm=norm)
    ax.grid(True,which='both',color='lightgrey', linewidth=0.5)    
    ax.set_yticks([x-0.5 for x in range(1+len(tab))])
    ax.set_xticks([x-0.5 for x in range(1+len(tab[0]))])     
    ax.set_xticklabels([])
    ax.set_yticklabels([])     
    plt.tight_layout()
    plt.show()
    
def mostra_tabs(tabs):
    if len(tabs)==1:
        mostra_tab(tabs[0])
        return
    cmap = colors.ListedColormap(['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00','#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
    norm = colors.Normalize(vmin=0, vmax=9)
    fig, ax = plt.subplots(1, len(tabs), figsize=(3*len(tabs),3))
    for i,tab in enumerate(tabs):
        ax[i].imshow(tab, cmap=cmap, norm=norm)
        ax[i].grid(True,which='both',color='lightgrey', linewidth=0.5)    
        ax[i].set_yticks([x-0.5 for x in range(1+len(tab))])
        ax[i].set_xticks([x-0.5 for x in range(1+len(tab[0]))])     
        ax[i].set_xticklabels([])
        ax[i].set_yticklabels([])     
    plt.tight_layout()
    plt.show()
    
def print_molesto(molestia):
    print('\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n')
    print('$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')
    print('$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')
    print('$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')
    print()
    print(molestia)
    print()
    print('$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')
    
    
    
##################################
# Un paio di funzioni utili dopo #
    
#prima usavo lei
from itertools import product
#poi è servita più generalità :)
def cartesian(list_of_lists):
    dim = len(list_of_lists)
    if dim==0:
        yield []
    else:
        for first_part in cartesian(list_of_lists[:dim-1]):
            for second_part in list_of_lists[dim-1]:
                yield first_part + [second_part]
        
def flattener(pred):
    str_pred = str([row for row in pred])
    str_pred = str_pred.replace(', ', '')
    str_pred = str_pred.replace('[[', '|')
    str_pred = str_pred.replace('][', '|')
    str_pred = str_pred.replace(']]', '|')
    return str_pred

def elenca_sottoinsiemi_di(insieme,cardinalita_max=3):
    iinsieme = range(len(insieme)) #insieme degli indici
    sottoinsiemi_di_card = [[] for _ in range(cardinalita_max+1)]
    sottoinsiemi_di_card[0].append([]) #di cardinalita 0 c'è solo l'insieme vuoto
    for c in range(1,cardinalita_max+1):
        for sottoinsieme_ridotto,elemento in product(sottoinsiemi_di_card[c-1], iinsieme):
            if len(sottoinsieme_ridotto)==0 or elemento>max(sottoinsieme_ridotto):
                #genera un nuovo sottoinsieme che contiene elemento + insieme_ridotto
                sottoinsieme = [elemento]
                for el in sottoinsieme_ridotto:
                    sottoinsieme.append(el)
                sottoinsiemi_di_card[c].append(sottoinsieme)
    lista_sottoinsiemi=[]
    for c in range(cardinalita_max+1):
        for sottoinsieme in sottoinsiemi_di_card[c]:
            lista_sottoinsiemi.append([insieme[index] for index in sottoinsieme])
    return lista_sottoinsiemi

def elenca_solo_sottoinsiemi_di(insieme,cardinalita=3):
    iinsieme = range(len(insieme)) #insieme degli indici
    sottoinsiemi_di_card = [[] for _ in range(cardinalita+1)]
    sottoinsiemi_di_card[0].append([]) #di cardinalita 0 c'è solo l'insieme vuoto
    for c in range(1,cardinalita+1):
        for sottoinsieme_ridotto,elemento in product(sottoinsiemi_di_card[c-1], iinsieme):
            if len(sottoinsieme_ridotto)==0 or elemento>max(sottoinsieme_ridotto):
                #genera un nuovo sottoinsieme che contiene elemento + insieme_ridotto
                sottoinsieme = [elemento]
                for el in sottoinsieme_ridotto:
                    sottoinsieme.append(el)
                sottoinsiemi_di_card[c].append(sottoinsieme)
    lista_sottoinsiemi = []
    for sottoinsieme in sottoinsiemi_di_card[cardinalita]:
        lista_sottoinsiemi.append([insieme[index] for index in sottoinsieme])
    return lista_sottoinsiemi

def elenca_solo_sottosequenze_di(insieme,cardinalita=3):
    sottoinsiemi_di_card = [[] for _ in range(cardinalita+1)]
    sottoinsiemi_di_card[0].append([]) #di cardinalita 0 c'è solo l'insieme vuoto
    for c in range(1,cardinalita+1):
        for sottoinsieme_ridotto,elemento in product(sottoinsiemi_di_card[c-1], insieme):
            if elemento not in sottoinsieme_ridotto:
                #genera un nuovo sottoinsieme che contiene elemento + insieme_ridotto
                sottoinsieme = [elemento]
                for el in sottoinsieme_ridotto:
                    sottoinsieme.append(el)
                sottoinsiemi_di_card[c].append(sottoinsieme)
    return sottoinsiemi_di_card[cardinalita]




def ide(y,x,shape,to_shape=False):
    if to_shape:
        return shape
    else:
        return y,x
def rotO(y,x,shape,to_shape=False):
    if to_shape:
        return shape[1],shape[0]
    else:
        return x , shape[0]-1-y
def rot180(y,x,shape,to_shape=False):
    if to_shape:
        return shape
    else:
        return shape[0]-1-y , shape[1]-1-x
def rotA(y,x,shape,to_shape=False):
    if to_shape:
        return shape[1],shape[0]
    else:
        return shape[1]-1-x , y
def refO(y,x,shape,to_shape=False):
    if to_shape:
        return shape
    else:
        return y , shape[1]-1-x
def refV(y,x,shape,to_shape=False):
    if to_shape:
        return shape
    else:
        return shape[0]-1-y , x
def refD1(y,x,shape,to_shape=False):
    if to_shape:
        return shape[1],shape[0]
    else:
        return x , y
def refD2(y,x,shape,to_shape=False):
    if to_shape:
        return shape[1],shape[0]
    else:
        return shape[1]-1-x , shape[0]-1-y

def are_equal(tab_start, tab_end, f):
    shape_start = (len(tab_start),len(tab_start[0]))
    shape_end = (len(tab_end),len(tab_end[0]))
    new_shape_end = f(0,0,shape_start,to_shape=True)
    if shape_end!=new_shape_end:
        return False
    for y,x in product(range(shape_start[0]),range(shape_start[1])):
        new_y,new_x = f(y,x,shape_start)
        if tab_end[new_y][new_x] != tab_start[y][x]:
            return False
    return True

def find_geo_invariants(tab):
    invariants = []
    for i in range(8):
        if are_equal(tab,tab,geometry[i]):
            invariants.append(i)
    return invariants

def apply_transf(f,tab):
    shape = (len(tab),len(tab[0]))
    new_shape = f(0,0,shape,to_shape=True)
    new_tab = [[0 for _ in range(new_shape[1])] for _ in range(new_shape[0])]
    #mostra_tab(new_tab)
    for y,x in product(range(shape[0]),range(shape[1])):
        new_y,new_x = f(y,x,shape)
        new_tab[new_y][new_x] = 0 + tab[y][x]
    return new_tab

'''
geometry     = [ide, rotO, rot180, rotA, refO, refV, refD1, refD2]
geometry_inv = [ide, rotA, rot180, rotO, refO, refV, refD1, refD2]
geometry_name = {0:'Identità', 1:'Rotazione oraria', 2:'Rotazione 180 gradi', 3:'Rotazione antioraria',
                 4:'Riflessione orizzontale', 5:'Riflessione verticale', 6:'Riflessione diagonale 1', 7:'Riflessione diagonale 2'}
'''
geometry     = [ide, rotO, rotA, refO, refV, refD1, refD2, rot180]
geometry_inv = [ide, rotA, rotO, refO, refV, refD1, refD2, rot180]
geometry_name = {0:'Identità', 1:'Rotazione oraria', 2:'Rotazione antioraria', 3:'Riflessione orizzontale',
                 4:'Riflessione verticale', 5:'Riflessione diagonale 1', 6:'Riflessione diagonale 2',7:'Rotazione 180 gradi'}
       

# Esempi

In [ ]:
task_esempi = [1,7,4,9,0,5,16]
#1 fill yellow
#7 attacca rosso a blu
#4 ripeti ogg con forma-colore
#9 colora dal più grande al più piccolo
#0 and logico
#5 3x3
#16 denoiss
    
for n,task_da_fare in enumerate([1,4,7,9,0,5]):
    print('Esempio Task numero: ',n+1)
    task_file = str(training_path / training_tasks[task_da_fare])
    with open(task_file, 'r') as f:
        task = json.load(f)
    plot_task(task)


___


<br><br><br>

# APPROCCIO STANDARD

<h3>Regressione Lineare</h3>

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression


# shape giusta -> 0,4,5,6,9,10,11,15
for n,task_da_fare in enumerate([0,4,5,9,10,15]):
    print('PROVO TASK',n+1)
    task_file = str(training_path / training_tasks[task_da_fare])
    with open(task_file, 'r') as f:
        task = json.load(f)
       
    #formatta i dati
    inp_shape = np.array(task['train'][0]['input']).shape
    out_shape = np.array(task['train'][0]['output']).shape 
    imgs_in, imgs_out = [], []
    for pairIO in task['train']:
        img_in = np.array(pairIO['input'])
        img_out = np.array(pairIO['output'])
        imgs_in.append(img_in)
        imgs_out.append(img_out)
        if img_in.shape!=inp_shape or img_out.shape!=out_shape:
            print('no bueno')
            break        
    datas_in = [img_in.flatten() for img_in in imgs_in]
    datas_out = [img_out.flatten() for img_out in imgs_out]
    datas_in = pd.DataFrame(datas_in)
    datas_out = pd.DataFrame(datas_out)
            
        
    #allena il modello
    model = LinearRegression().fit(datas_in, datas_out)
       
    
    #visualizza risultati
    test_img_in = np.array(task['test'][0]['input'])
    test_img_out = np.array(task['test'][0]['output'])
    test_data_in = test_img_in.flatten()
    imgs_in.append(test_img_in)
    imgs_out.append(test_img_out)
    preds = []
    for i in range(len(datas_in)):
        data_in = datas_in.iloc[i].to_numpy().reshape((1,-1))
        preds.append(model.predict(data_in).reshape(out_shape).round())
    test_data_in = test_data_in.reshape((1,-1))
    preds.append(model.predict(test_data_in).reshape(out_shape).round())
    imgs_in.insert(-1,[[9]])
    imgs_out.insert(-1,[[9]])
    preds.insert(-1,[[9]])
    mostra_tabs(imgs_in)
    mostra_tabs(imgs_out)
    mostra_tabs(preds)



<br><br><br>

<h3>Rete Neurale</h3>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self, inp, oup):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        #self.conv1 = nn.Conv2d(1, 6, 3)
        #self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(inp, 2*inp) 
        self.fc2 = nn.Linear(2*inp, 2*oup)
        self.fc3 = nn.Linear(2*oup, oup)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        #x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        #x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        #x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def train(model, x, y, optimizer, criterion):
    model.zero_grad()
    output = model.forward(x)
    loss = criterion(output,y)
    loss.backward()
    optimizer.step()
    return loss, output

from torch.optim import Adam,SGD
criterion = nn.MSELoss()

EPOCHS = 5000


# shape giusta -> 0,4,5,6,9,10,11,15
for n,task_da_fare in enumerate([4,9,10]):
    print('PROVO TASK',n+1)
    task_file = str(training_path / training_tasks[task_da_fare])
    with open(task_file, 'r') as f:
        task = json.load(f)
       
    #formatta i dati
    inp_shape = np.array(task['train'][0]['input']).shape
    out_shape = np.array(task['train'][0]['output']).shape 
    imgs_in, imgs_out = [], []
    for pairIO in task['train']:
        img_in = np.array(pairIO['input'])
        img_out = np.array(pairIO['output'])
        imgs_in.append(img_in)
        imgs_out.append(img_out)
        if img_in.shape!=inp_shape or img_out.shape!=out_shape:
            print('no bueno')
            break        
    datas_in = [torch.Tensor(img_in.flatten()) for img_in in imgs_in]
    datas_out = [torch.Tensor(img_out.flatten()) for img_out in imgs_out]
            
    
    #allena la rete neurale
    net = Net(len(datas_in[0]), len(datas_out[0]))
    optm = SGD(net.parameters(), lr=0.0001, momentum=0.95)
    
    for e in range(EPOCHS):
        for data_in, data_out in zip(datas_in, datas_out):
            loss, predictions = train(net, data_in, data_out, optm, criterion)
            #print(loss,predictions)
            
    
    
    #visualizza risultati
    test_img_in = np.array(task['test'][0]['input'])
    test_img_out = np.array(task['test'][0]['output'])
    test_data_in = torch.Tensor(test_img_in.flatten())
    datas_in.append(test_data_in)
    imgs_in.append(test_img_in)
    imgs_out.append(test_img_out)
    preds = [net.forward(data_in).detach().numpy().reshape(out_shape).round() for data_in in datas_in]
    imgs_in.insert(-1,[[9]])
    imgs_out.insert(-1,[[9]])
    preds.insert(-1,[[9]])
    mostra_tabs(imgs_in)
    mostra_tabs(imgs_out)
    mostra_tabs(preds)
    '''
    for data_in, img_in, img_out in zip(datas_in, imgs_in, imgs_out):
        pred = net.forward(data_in)
        pred = pred.detach().numpy().reshape(out_shape).round() #.astype(int)
        mostra_tabs([img_in, [[9]], img_out, pred])'''

<h3>Perché il machine learning NON funziona?</h3>
* o non è abbastanza complesso
* o è abbastanza complesso ma trova una soluzione _sbagliata_

<h3>Cos'è una soluzione?</h3>
* concetto di complessità
* _la soluzione è la funzione di minima complessità tra quelle che risolvono il train_



___


<br><br><br><br><br><br>

# APPROCCIO FALLIMENTARE MA COLORATO

# Oggetti

<h2>Classe Oggetto</h2>
Componente connessa di colore (e non solo)
* forma
* colore
* posione
* _altro..._

<h2>Classe Immagine</h2>
Lista di Oggetti

<h2>Classe Task</h2>
Lista di coppie di Immagini (input, output)

In [ ]:
#################################
#  DEFINISCO LA CLASSE OGGETTI  #
#################################
class Oggetto:
    def __init__(self, forma=None, colore=None, posizione=None, colori_adiacenti=None, oggetti_adiacenti=None,
                 tocca_bordo=None, is_specific_ASS_form=False, is_specific_ASS_color=False, is_shape_mult=False):
        #colore:  int tra 0 e 9
        self.colore = colore
        #forma, dimensione e area:  tabella di 0 e 1
        self.forma = forma
        self.dimensione = None if forma is None else (len(self.forma),len(self.forma[0]))  # (y,x)
        self.dimmezzi = None if forma is None else (self.dimensione[0]/2,self.dimensione[1]/2)  # (y,x)
        self.area = None if forma is None else self.calcola_area()
        #posizione e lati up down sx dx:  posizioni di centro e angoli
        self.posizione = None if posizione is None else tuple(posizione)       # (y,x)
        self.posizione_diag = None if posizione is None else (posizione[0]+posizione[1],posizione[0]-posizione[1]) # (somma,differenza)
        #altri parametri vicinanza
        self.colori_adiacenti = set() if colori_adiacenti is None else tuple(colori_adiacenti)
        self.oggetti_adiacenti = set() if oggetti_adiacenti is None else oggetti_adiacenti
        self.tocca_bordo = None if tocca_bordo is None else tuple(tocca_bordo)     #tupla (tocca_sopra, tocca_sotto, tocca_sx, tocca_dx)
        self.tocca_un_bordo = None if tocca_bordo is None else tocca_bordo[0] or tocca_bordo[1] or tocca_bordo[2] or tocca_bordo[3]
        self.tocca_solo_bordo = None if tocca_bordo is None else (tocca_bordo[0] and not tocca_bordo[1] and not tocca_bordo[2] and not tocca_bordo[3], not tocca_bordo[0] and tocca_bordo[1] and not tocca_bordo[2] and not tocca_bordo[3], not tocca_bordo[0] and not tocca_bordo[1] and tocca_bordo[2] and not tocca_bordo[3], not tocca_bordo[0] and not tocca_bordo[1] and not tocca_bordo[2] and tocca_bordo[3])
        self.tocca_tutti_i_bordi = None if tocca_bordo is None else tocca_bordo[0] and tocca_bordo[1] and tocca_bordo[2] and tocca_bordo[3]
        #tocca un angolo?
        self.tocca_bordo_ORIZ = None if tocca_bordo is None else tocca_bordo[2] or tocca_bordo[3]
        self.tocca_bordo_VERT = None if tocca_bordo is None else tocca_bordo[0] or tocca_bordo[1]
        self.is_larghissimo = None if tocca_bordo is None else tocca_bordo[2] and tocca_bordo[3]
        self.is_lunghissimo = None if tocca_bordo is None else tocca_bordo[0] and tocca_bordo[1]
        self.is_specific_IF_form = False    #se è comune a tutti i train INPUT
        self.is_specific_IF_color = False   #se è comune a tutti i train INPUT
        self.is_specific_ASS_form = is_specific_ASS_form   #se è comune a tutti i train OUTPUT
        self.is_specific_ASS_color = is_specific_ASS_color #se è comune a tutti i train OUTPUT
        self.is_special_color = False       #se è unico nell'immagine (tranne il nero)
        self.is_special_form = False        #se è unica nell'immagine
        self.is_common_color = False        #se è il più frequente nell'immagine
        self.is_common_form = False         #se è la più frequente nell'immagine
        self.is_rarest_color = False #se è il meno frequente dell'immagine (meglio rispetto a is_special_color???)
        #gestione "più oggetti indicano lo stesso" (es: oggetto tagliato dal bordo) .... coccalma
        #self.linked = set()
        #booleani avanzati per IF
        self.is_cornice = self.find_is_cornice()
        self.is_stecca = None if forma is None else self.find_is_stecca()
        self.contiene_dueXdue = None if forma is None else self.find_contiene_dueXdue()
        self.is_rettangolo = None if forma is None else self.area == self.dimensione[0]*self.dimensione[1]
        #self.is_rettangolo_grande = None if self.is_rettangolo is None else self.is_rettangolo and self.dimensione[0]>1 and self.dimensione[1]>1
        self.is_quadratoso = None if forma is None else self.dimensione[0]==self.dimensione[1]
        self.is_bucato = None #aggiusto in Immagine
        self.buchi = set() #aggiusto in Immagine
        self.numero_buchi = None
        self.figli = set() #per ora solo gli oggetti contenuti negli oggetti "non neri"
        self.fratelli = set() 
        self.is_colorful = False #True solo per gli oggetti npn neri
        self.area_order = None
        self.area_order_rev = None
        self.area_order_for_color = None
        self.area_order_for_color_rev = None
        self.piu_vicino_di_colore_diverso = None
        self.is_shape_mult=is_shape_mult
        
        self.is_for_shape_out = False
        self.is_denoised = False
        self.is_noise = False
        
        #parametri per la gestione della ricerca di funzioni
        self.is_risolto = False
        self.is_risolvibile = False
        self.is_come_input = False
        
        #find_geo_invariants(tab):
        #@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
        #è invariante per cosa
        #è invariante per tutti
            
        
        
    def calcola_area(self):
        area = 0
        for row in self.forma:
            for el in row:
                area+=el
        return area
    def find_is_cornice(self):
        if self.forma is None:
            return False
        if self.dimensione[0]<3 or self.dimensione[1]<3:
            return False
        for x in range(self.dimensione[1]):
            if self.forma[0][x]==0 or self.forma[self.dimensione[0]-1][x]==0:
                return False
        for y in range(1,self.dimensione[0]-1):
            if self.forma[y][0]==0 or self.forma[y][self.dimensione[1]-1]==0:
                return False
        for y,x in product(range(1,self.dimensione[0]-1),range(1,self.dimensione[1]-1)):
            if self.forma[y][x]==1:
                return False
        return True
    def find_is_stecca(self):
        altezza_minima_per_chiamarlo_stecca = 2
        if self.dimensione is None:
            return False
        if self.dimensione[0]==1 and self.dimensione[1]>=altezza_minima_per_chiamarlo_stecca:
            return True
        if self.dimensione[1]==1 and self.dimensione[0]>=altezza_minima_per_chiamarlo_stecca:
            return True
        return False
    def find_contiene_dueXdue(self):
        if self.dimensione is None:
            return False
        if self.is_stecca:
            return False
        for x,y in product(range(self.dimensione[1] - 1),range(self.dimensione[0] - 1)):
            if self.forma[y][x]==1 and self.forma[y+1][x]==1 and self.forma[y][x+1]==1 and self.forma[y+1][x+1]==1:
                return True
        return False
    def find_is_bucato(self,map_tab_to_obj):
        #è bucato sse il complementare (a toro) non è connesso
        #visita gli 0 partendo dal perimetro
        visitati = [[False for _ in range(self.dimensione[1])] for _ in range(self.dimensione[0])]
        def dfs(x,y):
            x %= self.dimensione[1]
            y %= self.dimensione[0]
            if visitati[y][x] or self.forma[y][x]==1:
                return 0
            visitati[y][x]=True
            punti_visti=1
            for incr in [(1,0),(-1,0),(0,1),(0,-1)]:
                punti_visti += dfs(x+incr[1],y+incr[0])
            return punti_visti
        punti_visti = 0
        for x in range(self.dimensione[1]):
            punti_visti += dfs(x,0)
            punti_visti += dfs(x,self.dimensione[0]-1)
        for y in range(1,self.dimensione[0]-1):
            punti_visti += dfs(0,y)
            punti_visti += dfs(self.dimensione[1]-1,y)
        #se esistono 0 non visitati allora c'è un buco
        is_bucato=False
        self.numero_buchi = 0
        for x,y in product(range(self.dimensione[1]),range(self.dimensione[0])):
            if not visitati[y][x] and self.forma[y][x]==0:
                self.numero_buchi += 1
                is_bucato = True
                def dfs(x,y):
                    if visitati[y][x] or self.forma[y][x]==1:
                        return
                    visitati[y][x]=True
                    for incr in [(1,0),(-1,0),(0,1),(0,-1)]:
                        dfs(x+incr[1],y+incr[0])
                    tab_x = int(self.posizione[1] - self.dimmezzi[1] + x)
                    tab_y = int(self.posizione[0] - self.dimmezzi[0] + y)
                    self.buchi.add(map_tab_to_obj[tab_y][tab_x])
                dfs(x,y)
        #if punti_visti+self.area != self.dimensione[0]*self.dimensione[1]:
        #    print('MADONNA BUCATA DA DIO NEL SONNO')
        return is_bucato
    def contiene(self,oggetto):
        return oggetto in self.buchi
    def contenuto(self,oggetto):
        return self in oggetto.buchi
    
    def possiede(self,oggetto):
        return oggetto in self.figli
    def appartiene(self,oggetto):
        return self in oggetto.figli
        
    def tocca(self,oggetto):
        return self in oggetto.oggetti_adiacenti or oggetto in self.oggetti_adiacenti
    
    def is_allineato_vert(self,oggetto):
        if self.posizione is None or self.dimmezzi is None or oggetto.posizione is None or oggetto.dimmezzi is None:
            return False
        #if (self.tocca_bordo[0] and self.tocca_bordo[1]) or (oggetto.tocca_bordo[0] and oggetto.tocca_bordo[1]):
        #    return False #### MMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMM BOH
        distY = 100000000
        sopra=None
        for a,b in cartesian([[1,-1],[1,-1]]):
            offsetY = self.posizione[0]+a*self.dimmezzi[0] - oggetto.posizione[0]+b*oggetto.dimmezzi[0]
            if offsetY>0:
                if sopra is None:
                    sopra=False
                elif sopra is True:
                    distY=-1
            elif offsetY<0:
                if sopra is None:
                    sopra=True
                elif sopra is False:
                    distY=-1
            #distY = min(distY, abs(offsetY))
        return distY==-1
    def is_allineato_oriz(self,oggetto):
        if self.posizione is None or self.dimmezzi is None or oggetto.posizione is None or oggetto.dimmezzi is None:
            return False
        #if (self.tocca_bordo[2] and self.tocca_bordo[3]) or (oggetto.tocca_bordo[2] and oggetto.tocca_bordo[3]):
        #    return False
        distX = 100000000
        sinistra=None
        for a,b in cartesian([[1,-1],[1,-1]]):
            offsetX = self.posizione[1]+a*self.dimmezzi[1] - oggetto.posizione[1]+b*oggetto.dimmezzi[1]
            if offsetX>0:
                if sinistra is None:
                    sinistra=False
                elif sinistra is True:
                    distX=-1
            elif offsetX<0:
                if sinistra is None:
                    sinistra=True
                elif sinistra is False:
                    distX=-1
            #distX = min(distX, abs(offsetX))
            #print(distX)
        return distX==-1
    
    def distanza(self,oggetto):
        if self in oggetto.figli or oggetto in self.figli:
            return -2
        if self.tocca(oggetto):
            return -1
        if self.posizione is None or self.dimmezzi is None or oggetto.posizione is None or oggetto.dimmezzi is None:
            return 100000000
        distX = 100000000
        distY = 100000000
        sopra=None
        sinistra=None
        for a,b in cartesian([[1,-1],[1,-1]]):
            offsetX = self.posizione[1]+a*self.dimmezzi[1] - oggetto.posizione[1]+b*oggetto.dimmezzi[1]
            offsetY = self.posizione[0]+a*self.dimmezzi[0] - oggetto.posizione[0]+b*oggetto.dimmezzi[0]
            if offsetX>0:
                if sinistra is None:
                    sinistra=False
                elif sinistra is True:
                    distX=0
            elif offsetX<0:
                if sinistra is None:
                    sinistra=True
                elif sinistra is False:
                    distX=0
            if offsetY>0:
                if sopra is None:
                    sopra=False
                elif sopra is True:
                    distY=0
            elif offsetY<0:
                if sopra is None:
                    sopra=True
                elif sopra is False:
                    distY=0   
            distX = min(distX, abs(offsetX))
            distY = min(distY, abs(offsetY))
        return distX+distY
    
    def dista(self,oggetto,distanza_max):
        return self.distanza(oggetto) <= distanza_max
    
    def stampa(self):
        #print(self.dimensione)
        fig,ax = plt.subplots(1, figsize=(3,3))
        cmap = colors.ListedColormap(['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00','#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25', 'white'])
        norm = colors.Normalize(vmin=0, vmax=10)
        sfondo = 10 # if self.colore!=0 else -1
        da_stampare = [[self.colore if point==1 else sfondo for point in row] for row in self.forma]
        ax.imshow(da_stampare, cmap=cmap, norm=norm)
        ax.grid(True,which='both',color='lightgrey', linewidth=0.5)    
        ax.set_yticks([x-0.5 for x in range(1+self.dimensione[0])])
        ax.set_xticks([x-0.5 for x in range(1+self.dimensione[1])]) 
        ax.set_yticklabels([])    
        ax.set_xticklabels([])
        #ax.set_ylim(-0.5, self.dimensione[0]-0.5)
        #ax.set_xlim(-0.5, self.dimensione[1]-0.5)
        str_title = '['+str(self.colore)+']  in '+str(self.posizione)
        str_title += '   - adiacenti='+str(self.colori_adiacenti)
        specFif = ' Fif' if self.is_specific_IF_form else ''
        specCif = ' Cif' if self.is_specific_IF_color else ''
        str_title += specFif+specCif
        #str_title += str([self.lato_up, self.lato_down, self.lato_sx, self.lato_dx])
        ax.set_title(str_title)
        ax.set_xlabel(str(int(self.posizione[1]-self.dimmezzi[1]))+' - '+str(int(self.posizione[1]+self.dimmezzi[1])))
        ax.set_ylabel(str(int(self.posizione[0]+self.dimmezzi[0]))+' - '+str(int(self.posizione[0]-self.dimmezzi[0])))
        plt.tight_layout()
        plt.show() 
        
    def to_text(self):
        return str(self.posizione)+' '+str(self.colore)+' '+str(self.forma)


##################################
#  DEFINISCO LA CLASSE IMMAGINI  #
##################################
class Immagine:
    def __init__(self,tab,comp_connesse_diagonali=True, add_ogg_non_neri=False, add_ogg_shape_mul=None, solo_quadratini=False): #gli oggetti non neri vanno aggiunti solo se l'immagine è input
        self.shape = (len(tab),len(tab[0]))
        self.tab = tab
        self.list_obj = []
        self.forma = {}
        self.colore = {}
        self.posizione = {}
        self.dimensione = {}
        #self.colori_adiacenti = {}
        self.comp_connesse_diagonali = comp_connesse_diagonali
        self.solo_quadratini = solo_quadratini
        
        oggetti, map_tab_to_obj = self.trova_oggetti(comp_connesse_diagonali=comp_connesse_diagonali)
        self.map_tab_to_obj = map_tab_to_obj
        if add_ogg_non_neri:
            #print('aggiungo oggetti non neri')
            oggetti += self.trova_oggetti_non_neri(comp_connesse_diagonali=comp_connesse_diagonali) #usa map_tab_to_obj
        for oggetto in oggetti:
            self.aggiungi(oggetto) #aggiunge oggetto a list_obj e mette a posto i dizionari vari
                    
        
        for ogg in self.list_obj:
            #trova i buchi degli oggetti (parametri .is_bucato e .buchi)
            ogg.is_bucato = ogg.find_is_bucato(self.map_tab_to_obj)
            #trova oggetto per relazione (aggiusta il parametro)
            ogg.piu_vicino_di_colore_diverso = self.trova_oggetto_piu_vicino_di_colore_diverso(ogg)
        
        
        #aggiusta parametro oggetti AREA_ORDER
        i_of_areas = {}
        for index,ogg in enumerate(self.list_obj):
            if ogg.colore==0 or ogg.colore is None: #trascura gli oggetti NERI per la classifica
                continue 
            if ogg.area not in i_of_areas.keys():
                i_of_areas[ogg.area] = set([index])
            else:
                i_of_areas[ogg.area].add(index)
        i_and_areas = [(i_of_areas[area],area) for area in i_of_areas.keys()]
        i_and_areas.sort(key = lambda x : x[1], reverse=True) #decrescente
        for order,i_and_area in enumerate(i_and_areas):
            indexes = i_and_area[0]
            area = i_and_area[1]
            for index in indexes:
                if self.list_obj[index].area != area:
                    print('dio bau miao')
                self.list_obj[index].area_order = order
        i_and_areas.sort(key = lambda x : x[1], reverse=False) #crescente
        for order,i_and_area in enumerate(i_and_areas):
            indexes = i_and_area[0]
            for index in indexes:
                self.list_obj[index].area_order_rev = order
        #aggiusta parametro oggetti AREA_ORDER_for_color
        for colore in self.colore.keys():
            i_of_areas = {}
            ogg_con_quel_colore = self.colore[colore]
            if len(ogg_con_quel_colore)>1:
                for index,ogg in enumerate(ogg_con_quel_colore):
                    if ogg.area not in i_of_areas.keys():
                        i_of_areas[ogg.area] = set([index])
                    else:
                        i_of_areas[ogg.area].add(index)
                i_and_areas = [(i_of_areas[area],area) for area in i_of_areas.keys()]
                #ordine decrescente
                i_and_areas.sort(key = lambda x : x[1], reverse=True)
                for order,i_and_area in enumerate(i_and_areas):
                    indexes = i_and_area[0]
                    area = i_and_area[1]
                    for index in indexes:
                        if ogg_con_quel_colore[index].area != area:
                            print('dio bau miao')
                        ogg_con_quel_colore[index].area_order_for_color = order
                        if colore is 0:
                            ogg_con_quel_colore[index].area_order = order
                #ordine crescente
                i_and_areas.sort(key = lambda x : x[1], reverse=False)
                for order,i_and_area in enumerate(i_and_areas):
                    indexes = i_and_area[0]
                    for index in indexes:
                        ogg_con_quel_colore[index].area_order_for_color_rev = order
                        if colore is 0:
                            ogg_con_quel_colore[index].area_order_rev = order
        
        #aggiusta parametro oggetti IS_SPECIAL_FORM     (aka. quelli che compaiono solo una volta nell'immagine input)
        for forma in self.forma.keys():
            ogg_con_quella_forma = self.forma[forma]
            if len(ogg_con_quella_forma) == 1:
                for ogg in ogg_con_quella_forma:
                    ogg.is_special_form = True
        #aggiusta parametro oggetti IS_COMMON_FORM     (aka. quelle che compaiono più volte (strettamente) nell'immagine input)
        max_occ = 0
        max_occ_objs = []
        for forma in self.forma.keys():
            ogg_con_quella_forma = self.forma[forma]
            if len(ogg_con_quella_forma)>max_occ:
                max_occ_objs = ogg_con_quella_forma
                max_occ = len(ogg_con_quella_forma)
            elif len(ogg_con_quella_forma)==max_occ: #se ci sono più forme che compaiono max_occ volte allora non sono così speciali
                max_occ_objs = []
        for ogg in max_occ_objs:
            ogg.is_common_form = True
        
        
        if add_ogg_shape_mul is not None:
            print('aggiungo oggetti shape multiplier')
            if add_ogg_shape_mul!=self.shape:
                print('cazzo succe')
            for oggetto in self.trova_oggetti_shape_multiplier():
                self.aggiungi(oggetto) #aggiunge oggetto a list_obj e mette a posto i dizionari vari
                    
                    
        #aggiusta parametro oggetti IS_SPECIAL_COLOR     (aka. quelli che compaiono solo una volta nell'immagine input)
        for colore in self.colore.keys():
            if colore is 0 or colore is None: #il nero non conta e non viene contato
                continue
            ogg_con_quel_colore = self.colore[colore]
            if len(ogg_con_quel_colore) == 1: 
                for ogg in ogg_con_quel_colore:
                    ogg.is_special_color = True
        #aggiusta parametro oggetti IS_COMMON_COLOR    (aka. quelli che compaiono più volte (strettamente) nell'immagine input)
        max_occ = 0
        min_occ = 10000
        max_occ_objs = []
        min_occ_objs = []
        for colore in self.colore.keys():
            if colore is 0: # or colore is 5: # trascuro nero e grigio ??? boh
                continue
            ogg_con_quel_colore = self.colore[colore]
            
            if len(ogg_con_quel_colore)>max_occ:
                max_occ_objs = ogg_con_quel_colore
                max_occ = len(ogg_con_quel_colore)
            elif len(ogg_con_quel_colore)==max_occ: #se ci sono più forme che compaiono max_occ volte allora non sono così speciali
                max_occ_objs = []
                
            if len(ogg_con_quel_colore)<min_occ:
                min_occ_objs = ogg_con_quel_colore
                min_occ = len(ogg_con_quel_colore)
            elif len(ogg_con_quel_colore)==min_occ: #se ci sono più forme che compaiono min_occ volte allora non sono così speciali
                min_occ_objs = []
        for ogg in max_occ_objs:
            ogg.is_common_color = True
        for ogg in min_occ_objs:
            ogg.is_rarest_color = True
        
    ####################################
    # FUNZIONI PER gestire gli oggetti #
    def aggiungi(self,oggetto):
        self.list_obj.append(oggetto)
        for attributo in ['forma','colore','posizione','dimensione']:
            if oggetto.__dict__[attributo] != None:
                if oggetto.__dict__[attributo] not in self.__dict__[attributo]:
                    self.__dict__[attributo][oggetto.__dict__[attributo]] = [oggetto]
                else:
                    self.__dict__[attributo][oggetto.__dict__[attributo]].append(oggetto)
                    
    def aggiungi_oggetti_denoised(self,iterazioni=2):
        for oggetto in self.trova_oggetti_denoised(comp_connesse_diagonali=self.comp_connesse_diagonali,iterazioni=iterazioni):
            self.aggiungi(oggetto) #aggiunge oggetto a list_obj e mette a posto i dizionari vari
                      
                    
    def get_obj_with(self,forma=None, colore=None, posizione=None):
        if forma is not None:
            if forma in self.forma.keys():
                return set(self.forma[forma])
            else:
                return set()
        elif colore is not None:
            if colore in self.colore.keys():
                return set(self.colore[colore])
            else:
                return set()
        elif posizione is not None:
            if posizione in self.posizione.keys():
                return set(self.posizione[posizione])
            else:
                return set()
            
    def refresh_list_obj_from_map(self):
        oggetti = set()
        for row in self.map_tab_to_obj:
            for ogg in row:
                oggetti.add(ogg)
        self.list_obj = list(oggetti)
    
    ########################################################################
    # FUNZIONI PER trovare le componenti connesse di colore (aka. OGGETTI) #
    #trova anche map_tab_to_ogg, mappa che a ogni posizione associa l'oggetto che ci sta
    #Nota: non serve self perché usa solo la tabella di colori raw
    def trova_oggetti(self, comp_connesse_diagonali = True):
        tab = self.tab
        dimensione = [len(tab),len(tab[0])]   # [y,x]
        map_tab_to_obj = [[None for _ in range(dimensione[1])] for _ in range(dimensione[0])]
        visitato = [[False for _ in range(dimensione[1])] for _ in range(dimensione[0])]
        oggetti = []
        for x,y in product(range(dimensione[1]),range(dimensione[0])):
            if not visitato[y][x]:
                obj_colore = tab[y][x]
                obj_forma_big = [[0 for _ in range(dimensione[1])] for _ in range(dimensione[0])]
                obj_colori_adiacenti = set()
                obj_oggetti_adiacenti = set()
                obj_tocca_bordo = [False for _ in range(4)]    #up, down, sx, dx
                def dfs(x,y): #ritorna la lista di coordinate visitate
                    if tab[y][x] != obj_colore:
                        obj_colori_adiacenti.add(tab[y][x])
                        if map_tab_to_obj[y][x] is not None:
                            obj_oggetti_adiacenti.add(map_tab_to_obj[y][x])
                        return []
                    if visitato[y][x]:
                        return []
                    visitato[y][x] = True
                    obj_forma_big[y][x] = 1
                    incrementi = []
                    #orizzontali
                    if x>0:
                        incrementi.append([-1,0])
                    else:
                        obj_tocca_bordo[2] = True #tocca sinistra
                    if x+1<dimensione[1]:
                        incrementi.append([1,0])
                    else:
                        obj_tocca_bordo[3] = True #tocca destra
                    #verticali
                    if y>0:
                        incrementi.append([0,-1])
                    else:
                        obj_tocca_bordo[0] = True #tocca sopra
                    if y+1<dimensione[0]:
                        incrementi.append([0,1])
                    else:
                        obj_tocca_bordo[1] = True #tocca sotto
                    #diagonali
                    if comp_connesse_diagonali:
                        if x>0 and y>0:
                            incrementi.append([-1,-1])
                        if x>0 and y+1<dimensione[0]:
                            incrementi.append([-1,1])
                        if x+1<dimensione[1] and y>0:
                            incrementi.append([1,-1])
                        if x+1<dimensione[1] and y+1<dimensione[0]:
                            incrementi.append([1,1])
                    if self.solo_quadratini:
                        incrementi=[]
                    punti_visitati = [[y,x]]
                    for incr_x,incr_y in incrementi:
                        punti_visitati += dfs(x+incr_x, y+incr_y)
                    return punti_visitati
                punti_visitati = dfs(x,y)
                #qui obj_forma_big contiene la forma giusta in 0 e 1 ma è una tabella full-dimension
                #NOTA: il punto (x,y) appartiene all'oggetto ma non è detto che sia in uno dei 4 angoli della forma
                min_x, max_x, min_y, max_y = dimensione[1], 0, dimensione[0], 0
                for tmp_x,tmp_y in product(range(dimensione[1]),range(dimensione[0])):
                    if obj_forma_big[tmp_y][tmp_x]==1:
                        min_x = min(min_x,tmp_x)
                        max_x = max(max_x,tmp_x)
                        min_y = min(min_y,tmp_y)
                        max_y = max(max_y,tmp_y)
                obj_dim_x = max_x-min_x+1
                obj_dim_y = max_y-min_y+1
                obj_forma = [[0 for _ in range(obj_dim_x)] for _ in range(obj_dim_y)]
                for tmp_x,tmp_y in product(range(obj_dim_x),range(obj_dim_y)):
                    obj_forma[tmp_y][tmp_x] = obj_forma_big[min_y+tmp_y][min_x+tmp_x]
                obj_forma = tuple( tuple(row) for row in obj_forma)
                #qui obj_forma contiene la forma giusta in 0 e 1 ritagliata in modo da non avere colonne o righe di soli 0
                obj_posizione = (obj_dim_y/2 + min_y, obj_dim_x/2 + min_x)

                oggetto = Oggetto(forma=obj_forma, colore=obj_colore, posizione=obj_posizione,
                                  colori_adiacenti=obj_colori_adiacenti, oggetti_adiacenti=obj_oggetti_adiacenti,
                                  tocca_bordo=obj_tocca_bordo)
                oggetti.append(oggetto)
                #mette i puntatori all'oggetto nel tab
                for p in punti_visitati:
                    map_tab_to_obj[p[0]][p[1]] = oggetto
        for x,y in product(range(dimensione[1]),range(dimensione[0])):
            if map_tab_to_obj[y][x] == None:
                print('MALE MALE ho dei buchi nella fotuzza')
        return oggetti, map_tab_to_obj
    
    def trova_oggetti_non_neri(self, comp_connesse_diagonali = True): #ocho che USA la mappa map_tab_to_obj
        #@@@@@@@@@@@@@@@@@@@ EVITA OGGETTI LUNGHISSIMI O LARGHISSIMI
        tab = self.tab
        dimensione = [len(tab),len(tab[0])]   # [y,x]
        visitato = [[False for _ in range(dimensione[1])] for _ in range(dimensione[0])]
        is_da_trascurare = [[self.map_tab_to_obj[y][x].is_lunghissimo or self.map_tab_to_obj[y][x].is_larghissimo for x in range(dimensione[1])] for y in range(dimensione[0])] #metto visitati quei punti così non li considero
        #print('trasscuro',is_da_trascurare)
        oggetti = []
        for x,y in product(range(dimensione[1]),range(dimensione[0])):
            if not visitato[y][x] and tab[y][x]!=0 and not is_da_trascurare[y][x]:
                colori_visti = set()
                obj_forma_big = [[0 for _ in range(dimensione[1])] for _ in range(dimensione[0])]
                obj_tocca_bordo = [False for _ in range(4)]    #up, down, sx, dx
                def dfs(x,y): #ritorna la lista di coordinate visitate
                    if tab[y][x] == 0 or visitato[y][x] or is_da_trascurare[y][x]:
                        return []
                    visitato[y][x] = True
                    colori_visti.add(tab[y][x])
                    obj_forma_big[y][x] = 1
                    incrementi = []
                    #orizzontali
                    if x>0:
                        incrementi.append([-1,0])
                    else:
                        obj_tocca_bordo[2] = True #tocca sinistra
                    if x+1<dimensione[1]:
                        incrementi.append([1,0])
                    else:
                        obj_tocca_bordo[3] = True #tocca destra
                    #verticali
                    if y>0:
                        incrementi.append([0,-1])
                    else:
                        obj_tocca_bordo[0] = True #tocca sopra
                    if y+1<dimensione[0]:
                        incrementi.append([0,1])
                    else:
                        obj_tocca_bordo[1] = True #tocca sotto
                    #diagonali
                    if comp_connesse_diagonali:
                        if x>0 and y>0:
                            incrementi.append([-1,-1])
                        if x>0 and y+1<dimensione[0]:
                            incrementi.append([-1,1])
                        if x+1<dimensione[1] and y>0:
                            incrementi.append([1,-1])
                        if x+1<dimensione[1] and y+1<dimensione[0]:
                            incrementi.append([1,1])
                    if self.solo_quadratini:
                        incrementi = []
                    punti_visitati = [[y,x]]
                    for incr_x,incr_y in incrementi:
                        punti_visitati += dfs(x+incr_x, y+incr_y)
                    return punti_visitati
                punti_visitati = dfs(x,y)
                #qui obj_forma_big contiene la forma giusta in 0 e 1 ma è una tabella full-dimension
                #NOTA: il punto (x,y) appartiene all'oggetto ma non è detto che sia in uno dei 4 angoli della forma
                min_x, max_x, min_y, max_y = dimensione[1], 0, dimensione[0], 0
                for tmp_x,tmp_y in product(range(dimensione[1]),range(dimensione[0])):
                    if obj_forma_big[tmp_y][tmp_x]==1:
                        min_x = min(min_x,tmp_x)
                        max_x = max(max_x,tmp_x)
                        min_y = min(min_y,tmp_y)
                        max_y = max(max_y,tmp_y)
                obj_dim_x = max_x-min_x+1
                obj_dim_y = max_y-min_y+1
                obj_forma = [[0 for _ in range(obj_dim_x)] for _ in range(obj_dim_y)]
                for tmp_x,tmp_y in product(range(obj_dim_x),range(obj_dim_y)):
                    obj_forma[tmp_y][tmp_x] = obj_forma_big[min_y+tmp_y][min_x+tmp_x]
                obj_forma = tuple( tuple(row) for row in obj_forma)
                #qui obj_forma contiene la forma giusta in 0 e 1 ritagliata in modo da non avere colonne o righe di soli 0
                obj_posizione = (obj_dim_y/2 + min_y, obj_dim_x/2 + min_x)

                if len(colori_visti)>1: #se non copre almeno due colori non ha senso aggiungerlo!
                    oggetto = Oggetto(forma=obj_forma, colore=None, posizione=obj_posizione,
                                      tocca_bordo=obj_tocca_bordo)
                    #print('aggiungo multicolor',colori_visti,obj_forma)
                    #print(visitato, is_da_trascurare)
                    #mette i puntatori agli oggetti contenuti
                    for p in punti_visitati:
                        oggetto.figli.add(self.map_tab_to_obj[p[0]][p[1]])
                    for figlio in oggetto.figli:
                        figlio.fratelli = oggetto.figli
                    oggetto.is_colorful = True
                    oggetti.append(oggetto)
        return oggetti
    
    def trova_oggetti_denoised(self, comp_connesse_diagonali = True, iterazioni=2): #ocho che anche lui USA la mappa map_tab_to_obj
        tab = self.tab
        d = Denoiser(tab,iterazioni=iterazioni)
        denoised_tab = d.denoised_tab
        if denoised_tab == tab:
            return []
        dimensione = [len(tab),len(tab[0])]   # [y,x]
        visitato = [[False for _ in range(dimensione[1])] for _ in range(dimensione[0])]
        is_potenziale_denoised = [[tab[y][x]!=denoised_tab[y][x] for x in range(dimensione[1])] for y in range(dimensione[0])]
        eeei = [[1 if is_potenziale_denoised[y][x] else 0 for x in range(dimensione[1])] for y in range(dimensione[0])]  ###################################
        #mostra_tab(eeei)
        oggetti = []
        for x,y in product(range(dimensione[1]),range(dimensione[0])):
            if not visitato[y][x] and is_potenziale_denoised[y][x]:
                obj_colore = denoised_tab[y][x]
                obj_forma_big = [[0 for _ in range(dimensione[1])] for _ in range(dimensione[0])]
                obj_tocca_bordo = [False for _ in range(4)]    #up, down, sx, dx
                def dfs(x,y): #ritorna la lista di coordinate visitate
                    if denoised_tab[y][x] != obj_colore:
                        return []
                    if visitato[y][x] or not is_potenziale_denoised[y][x]:
                        return []
                    visitato[y][x] = True
                    obj_forma_big[y][x] = 1
                    incrementi = []
                    #orizzontali
                    if x>0:
                        incrementi.append([-1,0])
                    else:
                        obj_tocca_bordo[2] = True #tocca sinistra
                    if x+1<dimensione[1]:
                        incrementi.append([1,0])
                    else:
                        obj_tocca_bordo[3] = True #tocca destra
                    #verticali
                    if y>0:
                        incrementi.append([0,-1])
                    else:
                        obj_tocca_bordo[0] = True #tocca sopra
                    if y+1<dimensione[0]:
                        incrementi.append([0,1])
                    else:
                        obj_tocca_bordo[1] = True #tocca sotto
                    #diagonali
                    if comp_connesse_diagonali:
                        if x>0 and y>0:
                            incrementi.append([-1,-1])
                        if x>0 and y+1<dimensione[0]:
                            incrementi.append([-1,1])
                        if x+1<dimensione[1] and y>0:
                            incrementi.append([1,-1])
                        if x+1<dimensione[1] and y+1<dimensione[0]:
                            incrementi.append([1,1])
                    if self.solo_quadratini:
                        incrementi = []
                    punti_visitati = [[y,x]]
                    for incr_x,incr_y in incrementi:
                        punti_visitati += dfs(x+incr_x, y+incr_y)
                    return punti_visitati
                punti_visitati = dfs(x,y)
                #qui obj_forma_big contiene la forma giusta in 0 e 1 ma è una tabella full-dimension
                #NOTA: il punto (x,y) appartiene all'oggetto ma non è detto che sia in uno dei 4 angoli della forma
                min_x, max_x, min_y, max_y = dimensione[1], 0, dimensione[0], 0
                for tmp_x,tmp_y in product(range(dimensione[1]),range(dimensione[0])):
                    if obj_forma_big[tmp_y][tmp_x]==1:
                        min_x = min(min_x,tmp_x)
                        max_x = max(max_x,tmp_x)
                        min_y = min(min_y,tmp_y)
                        max_y = max(max_y,tmp_y)
                obj_dim_x = max_x-min_x+1
                obj_dim_y = max_y-min_y+1
                obj_forma = [[0 for _ in range(obj_dim_x)] for _ in range(obj_dim_y)]
                for tmp_x,tmp_y in product(range(obj_dim_x),range(obj_dim_y)):
                    obj_forma[tmp_y][tmp_x] = obj_forma_big[min_y+tmp_y][min_x+tmp_x]
                obj_forma = tuple( tuple(row) for row in obj_forma)
                #qui obj_forma contiene la forma giusta in 0 e 1 ritagliata in modo da non avere colonne o righe di soli 0
                obj_posizione = (obj_dim_y/2 + min_y, obj_dim_x/2 + min_x)

                oggetto = Oggetto(forma=obj_forma, colore=obj_colore, posizione=obj_posizione,
                                      tocca_bordo=obj_tocca_bordo)
                denoisati=set()
                for p in punti_visitati:
                    denoisati.add(self.map_tab_to_obj[p[0]][p[1]])
                #if len(denoisati)>1:
                #    print([ogg.to_text() for ogg in denoisati])
                #    print('DENOISER CHE CAZZO FAI') #################################### aggiungere un collegamento ai vecchi ogg???
                for ogg_predenoisss in denoisati:
                    ogg_predenoisss.figli.add(oggetto)
                    ogg_predenoisss.is_noise = True
                oggetto.is_denoised = True
                #print('ASTRAGGO OGGETTO',oggetto.to_text())
                oggetti.append(oggetto)
        return oggetti
    
    def trova_oggetto_piu_vicino_di_colore_diverso(self,oggetto_start):
        distanze = [[100000000 for _ in range(self.shape[1])] for _ in range(self.shape[0])]
        max_poss_dist = max(self.shape[0],self.shape[1])
        def dfs(x,y,distanza_percorsa):                  #fa schifo la DFS!! va fatta una bfs 
            if distanza_percorsa >= distanze[y][x]:
                return
            if distanza_percorsa > max_poss_dist:
                return
            distanze[y][x] = distanza_percorsa
            incrementi = []
            if x>0:
                incrementi.append([-1,0])
            if x+1<self.shape[1]:
                incrementi.append([1,0])
            if y>0:
                incrementi.append([0,-1])
            if y+1<self.shape[0]:
                incrementi.append([0,1])
            if self.comp_connesse_diagonali:
                if x>0 and y>0:
                    incrementi.append([-1,-1])
                if x>0 and y+1<self.shape[0]:
                    incrementi.append([-1,1])
                if x+1<self.shape[1] and y>0:
                    incrementi.append([1,-1])
                if x+1<self.shape[1] and y+1<self.shape[0]:
                    incrementi.append([1,1])
                
            for incr_x,incr_y in incrementi:
                dfs(x+incr_x, y+incr_y, distanza_percorsa+1)
                
        for x,y in product(range(self.shape[1]),range(self.shape[0])):
            if self.map_tab_to_obj[y][x] is oggetto_start:
                dfs(x,y,0)
        
        min_dist = 1000000
        oggetto_bbbono = None
        for x,y in product(range(self.shape[1]),range(self.shape[0])):
            if distanze[y][x]>=0 and self.map_tab_to_obj[y][x].colore!=oggetto_start.colore and not self.map_tab_to_obj[y][x].tocca(oggetto_start):  #seleziono punti NON attaccati e di colore diverso
                if distanze[y][x] < min_dist:
                    #print('nuova min_dist',min_dist,'in',y,x)
                    oggetto_bbbono = self.map_tab_to_obj[y][x]
                    min_dist = distanze[y][x]
                elif distanze[y][x] == min_dist: #se ce n'è più di uno alla stessa distanza allora non è significativo
                    #print('nane',y,x)
                    if self.map_tab_to_obj[y][x] is not oggetto_bbbono:
                        oggetto_bbbono = None
                
        return oggetto_bbbono
                
    def trova_oggetti_shape_multiplier(self):
        oggetti = []
        for y,row in enumerate(self.tab):
            for x,colore in enumerate(row):
                new_pos = (y*self.shape[0], x*self.shape[1])
                oggetti.append(Oggetto(colore=colore, posizione=new_pos, is_shape_mult=True))
                #dimmezzi = (y*self.shape[0], x*self.shape[1])
                #forma = tuple( tuple(1 for _ in range(2*dimmezzi[1])) for _ in range(2*dimmezzi[0]))
                #print(dimmezzi, forma)
                #oggetti.append(Oggetto(forma=forma, colore=colore, is_shape_mult=True))
        return oggetti
          
    
    ##################################################################
    # FUNZIONI PER vedere insieme tutte le forme (senza ripetizioni) #
    def stampa(self):
        #print(self.dimensione)
        forme = list(self.forma.keys())
        num_forme = len(forme)
        if num_forme>0:
            cmap = colors.ListedColormap(['white', '#000000'])
            norm = colors.Normalize(vmin=0, vmax=1)
            fig,ax = plt.subplots(1, num_forme, figsize=(3*num_forme,3))
            for i in range(num_forme):     
                this_forma = forme[i]
                ax_infame = ax[i] if num_forme>=2 else ax 
                ax_infame.imshow(forme[i], cmap=cmap, norm=norm)
                ax_infame.grid(True,which='both',color='lightgrey', linewidth=0.5)    
                ax_infame.set_yticks([x-0.5 for x in range(1+len(this_forma))])
                ax_infame.set_xticks([x-0.5 for x in range(1+len(this_forma[0]))]) 
                ax_infame.set_yticklabels([])    
                ax_infame.set_xticklabels([])
                colori_forma = set()
                for oggetto in self.forma[this_forma]:
                    colori_forma.add(oggetto.colore)
                ax_infame.set_title('Occorrenze = '+str(len(self.forma[this_forma])) + '   Colori = '+str(colori_forma))
            plt.tight_layout()
            plt.show()  
            
    
    ################################################
    # FUNZIONI PER iterare su oggetti da risolvere #
    def esistono_oggetti_risolvibili_non_risolti(self,input_formacolore_e_AB):
        #print('esiste? ',self.list_obj)
        for oggetto in self.list_obj:
            if oggetto.is_risolvibile and not oggetto.is_risolto and input_formacolore_e_AB[oggetto][0]!=0:
                return True
        return False
    def get_first_obj_risolvibile_non_risolto(self,input_formacolore_e_AB): 
       #prima priorita, cioè prima ogg standard e poi splittati comeinput
        #restituisce prima oggetti che hanno forma e colore dati da 1 solo ogg_input e poi quelli a cui ne servono 2
        # tra questi, prima quelli che non hanno bisogno di rotazioni
        #  tra questi prima i non neri poi i neri
        #@@@@@@@@@@@@@@@@@@ dai la priorità a oggetti 'meno comuni'
        #
        for num_input in [1,2]:
            for need_rotation in [False,True]:
                #cioè prendo in ordine: (input 1 no rot),(input 1 si rot),(input 2 no rot),(input 2 si rot)         
                for nero in [False,True]:
                    for priorita in [1,2]:
                        for oggetto in self.list_obj:
                            if oggetto.is_risolvibile and not oggetto.is_risolto and input_formacolore_e_AB[oggetto][0]==num_input and input_formacolore_e_AB[oggetto][4]==priorita:
                                if (nero and oggetto.colore==0) or (not nero and oggetto.colore!=0):
                                    if need_rotation and len(input_formacolore_e_AB[oggetto][3])==0:
                                        print('maaaaaaa NNAGGIAAAA A QUEL PORCONEEE')
                                        print(oggetto.to_text(),input_formacolore_e_AB[oggetto])
                                    if (not need_rotation and 0 in input_formacolore_e_AB[oggetto][3]) or (need_rotation and 0 not in input_formacolore_e_AB[oggetto][3]):
                                        return oggetto
                        
        return None
        
    #######################################################
    # FUNZIONI PER USARE FUNZIONI BAU (le definisco dopo) #
    '''def possibili_oggetti_input_per(self,fun):
        possibili_oggetti_input = elenca_solo_sottosequenze_di(self.list_obj, cardinalita=fun.vero_numero_input)
        if fun.is_oggB_None:
            possibili_oggetti_input = [[oggetti_input[0]] + [None] + oggetti_input[1:] for oggetti_input in possibili_oggetti_input]
        return possibili_oggetti_input'''

    def find_obj_in_tab(self,forma,colore,posizione,g,ritorna_solo_coperture_totali=True):
        #RETURN punti_indovinati, punti_sbagliati, oggetti_risolti (completati)
        # può sbagliare, cioè andare su caselle di colori diversi        ---> punti_sbagliati > 0
        # (può andare su caselle di oggetti diversi ma non sbagliare)?
        # può coprire interamente un oggetto,                        
        # può coprire parzialmente un oggetto,
        # può uscire totalmente dall'immagine,                      ---> punti_sbagliati = punti_indovinati = 0
        #@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
        #da aggiungere eventuali oggetti linked
        sbaglia = False #sbaglia se crea anche solo un pixel di colore sbagliato
        esce = False #esce se crea una forma che sta totalmente fuori da shape output
        incompleto = False
        
        dim_x_f, dim_y_f = len(forma[0]), len(forma)
        dim_y_f, dim_x_f = geometry[g](0,0,(dim_y_f,dim_x_f),to_shape=True) #shape forma eventualmente ruotata
        dim_x_t, dim_y_t = self.shape[1], self.shape[0]
        x_upsx = posizione[1] - dim_x_f/2
        y_upsx = posizione[0] - dim_y_f/2
        if x_upsx != int(x_upsx) or y_upsx != int(y_upsx):
            #print('MANNAGGIA A GESU E ALLA POSIZIONE NON INTERA') #######################################LO VISITA SPESSISSIMO ocho anche al verifica_if
            return 0, 1, []     #la posizione non torna con la dimensione della forma 
        else:
            x_upsx, y_upsx = int(x_upsx), int(y_upsx)
        
        punti_visti = 0
        punti_sbagliati = 0
        punti_indovinati = 0
        oggetti_intersecati = set()
        xs = range(max(x_upsx,0), min(x_upsx+dim_x_f, dim_x_t))
        ys = range(max(y_upsx,0), min(y_upsx+dim_y_f, dim_y_t))
        for x,y in product(xs,ys): #coordinate di tab nell'intersezione con forma ruotata
            point_in_rot_form = (y-y_upsx,x-x_upsx)
            point_in_orig_form = geometry_inv[g](point_in_rot_form[0],point_in_rot_form[1],(dim_y_f,dim_x_f))
            if forma[point_in_orig_form[0]][point_in_orig_form[1]]==1:
                punti_visti += 1
                ogg_in_quel_punto = self.map_tab_to_obj[y][x]
                if ogg_in_quel_punto.colore!=colore:
                    sbaglia=True
                    punti_sbagliati += 1
                else:
                    oggetti_intersecati.add(ogg_in_quel_punto)
                    punti_indovinati += 1
        oggetti_intersecati = list(oggetti_intersecati) #con lo stesso colore
        
        #print(punti_visti,punti_sbagliati,punti_indovinati,oggetti_intersecati)
        
        if punti_visti==0: #esce
            return 0,0,[]
        if len(oggetti_intersecati)==0: #non indovina nemmeno un punto
            return 0,punti_sbagliati,[]
        if len(oggetti_intersecati)==1:
            if punti_indovinati==oggetti_intersecati[0].area: #copre un intero oggetto e non indovina altri punti
                return punti_indovinati, punti_sbagliati, oggetti_intersecati #ocho punti_sbagliati può essere >0
            elif punti_indovinati<oggetti_intersecati[0].area: #copre parzialmente un oggetto e non indovina altri punti
                if ritorna_solo_coperture_totali:
                    #print('check aree',punti_indovinati,oggetti_intersecati[0].area, oggetti_intersecati[0].to_text())
                    return punti_indovinati, punti_sbagliati, []
                else:
                    return punti_indovinati, punti_sbagliati, oggetti_intersecati
            else:
                print('GROSSO PROBLEMA O ALLA MAPPA TAB_TO_OBJ O A QUALCHE ALTRO GESU CRISTO')
        if len(oggetti_intersecati)>1:
            somma_aree = 0
            for ogg_inter in oggetti_intersecati:
                somma_aree += ogg_inter.area
            if punti_indovinati==somma_aree:
                return punti_indovinati, punti_sbagliati, oggetti_intersecati #ocho punti_sbagliati può essere >0
            elif punti_indovinati<somma_aree:
                if ritorna_solo_coperture_totali:
                    return punti_indovinati, punti_sbagliati, [] #in realtà potrebbe esserci qualcuno tra questi oggetti coperto totalmente
                else:
                    return punti_indovinati, punti_sbagliati, oggetti_intersecati
            else:
                print('GROSSO PROBLEMA O ALLA MAPPA TAB_TO_OBJ O A QUALCHE ALTRO GESU CRISTO')
            
        
    
    
##############################
#  DEFINISCO LA CLASSE TASK  #
##############################
class Task:
    def __init__(self,task,comp_connesse_diagonali=True,reflected=False,solo_quadratini=False):
        self.original_task = task
        self.solo_quadratini = solo_quadratini
        
        self.train_size = len(task['train'])
        self.train = task['train']
        self.test_size = len(task['test'])
        self.test = task['test']
        
        shape_mult_train, shape_mult_test = self.find_shape_mult_se_esistono()
        
        self.trainIMM = []      #lista di dizionari con key 'input' e 'output' con valori in class_Immagine
        for n_train in range(self.train_size):
            self.trainIMM.append({})
            for in_or_out in ['input', 'output']:
                tab = task['train'][n_train][in_or_out]     #tabelle raw
                is_input = in_or_out=='input'
                if not reflected:
                    self.trainIMM[n_train][in_or_out] = Immagine(tab, comp_connesse_diagonali=comp_connesse_diagonali, solo_quadratini=solo_quadratini,
                                                                 add_ogg_non_neri=is_input and not solo_quadratini, add_ogg_shape_mul=shape_mult_train[n_train] if is_input else None)
                else:
                    reflected_tab = apply_transf(geometry[5],tab)
                    self.trainIMM[n_train][in_or_out] = Immagine(reflected_tab, comp_connesse_diagonali=comp_connesse_diagonali, solo_quadratini=solo_quadratini,
                                                                 add_ogg_non_neri=is_input and not solo_quadratini, add_ogg_shape_mul=shape_mult_train[n_train] if is_input else None)
                
        self.testIMM = []      #lista di dizionari con key 'input' e 'output' con valori in class_Immagine
        for n_test in range(self.test_size):
            self.testIMM.append({})
            for in_or_out in ['input', 'output']:
                if in_or_out in task['test'][n_test].keys():
                    tab = task['test'][n_test][in_or_out]     #tabelle raw
                    is_input = in_or_out=='input'
                    if not reflected:
                        self.testIMM[n_test][in_or_out] = Immagine(tab, comp_connesse_diagonali=comp_connesse_diagonali, solo_quadratini=solo_quadratini,
                                                                   add_ogg_non_neri=is_input and not solo_quadratini, add_ogg_shape_mul=shape_mult_test[n_test] if is_input else None)
                    else:
                        reflected_tab = apply_transf(geometry[5],tab)
                        self.testIMM[n_test][in_or_out] = Immagine(reflected_tab, comp_connesse_diagonali=comp_connesse_diagonali, solo_quadratini=solo_quadratini,
                                                                   add_ogg_non_neri=is_input and not solo_quadratini, add_ogg_shape_mul=shape_mult_test[n_test] if is_input else None)
            
        #aggiusto parametro .is_come_input degli oggetti
        for immagine_in,immagine_out in [(self.trainIMM[i]['input'],self.trainIMM[i]['output']) for i in range(self.train_size)]: #+[(self.testIMM[i]['input'],self.testIMM[i]['output']) for i in range(self.test_size)]:
            for oggetto_out in immagine_out.list_obj:
                _,errori,_ = immagine_in.find_obj_in_tab(oggetto_out.forma,oggetto_out.colore,oggetto_out.posizione,0,ritorna_solo_coperture_totali=False)
                if errori == 0:
                    oggetto_out.is_come_input = True
                
        #forme e colori comuni e tutti i TRAIN (ma poi li flaggo anche nel TEST)
        self.specific = {}
        self.specific['IF'] = {}
        self.specific['IF']['forms'] = self.find_common_forms('input')
        self.specific['IF']['colors'] = self.find_common_colors('input')
        self.specific['ASS'] = {}
        self.specific['ASS']['forms'] = self.find_common_forms('output')
        self.specific['ASS']['colors'] = self.find_common_colors('output')
        
        immagini_input_train_test = [self.trainIMM[i]['input'] for i in range(self.train_size)] + [self.testIMM[i]['input'] for i in range(self.test_size)]
        #flaggo nell'immagine le forme e i colori specifici degli input
        for forma_specifica in self.specific['IF']['forms']:
            for immagine in immagini_input_train_test:
                for oggetto in immagine.get_obj_with(forma=forma_specifica):
                    oggetto.is_specific_IF_form=True
        for colore_specifico in self.specific['IF']['colors']:
            for immagine in immagini_input_train_test:
                for oggetto in immagine.get_obj_with(colore=colore_specifico):
                    oggetto.is_specific_IF_colors=True
        #flaggo nell'immagine le forme e i colori specifici degli output, aggiungendoli all'input se non sono già presenti
        for forma_specifica in self.specific['ASS']['forms']:
            colori_di_questa_forma_specifica = list(self.find_colors_of(forma_specifica,'output'))
            colore = colori_di_questa_forma_specifica[0] if len(colori_di_questa_forma_specifica)==1 else None
            posizioni_di_questa_forma_specifica = list(self.find_positions_of(forma_specifica,'output'))
            posizione = posizioni_di_questa_forma_specifica[0] if len(posizioni_di_questa_forma_specifica)==1 else None
            for immagine in immagini_input_train_test:
                oggetti_con_quella_forma = immagine.get_obj_with(forma=forma_specifica)                    
                if len(oggetti_con_quella_forma)==0:
                    immagine.aggiungi(Oggetto(forma=forma_specifica,colore=colore,posizione=posizione,is_specific_ASS_form=True))
                else:
                    for oggetto in oggetti_con_quella_forma:
                        oggetto.is_specific_ASS_form=True
                    if posizione is not None:
                        immagine.aggiungi(Oggetto(forma=forma_specifica,colore=colore,posizione=posizione,is_specific_ASS_form=True)) # non so se è una buona ideaaaa @@@@@@@@@@@@@@@@@@@@@@@@@@
        for colore_specifico in self.specific['ASS']['colors']:
            #forme_di_questo_colore_specifico = list(self.find_forms_of(colore_specifico,'output'))
            #forma = forme_di_questo_colore_specifico[0] if len(forme_di_questo_colore_specifico)==1 else None
            for immagine in immagini_input_train_test:
                oggetti_con_quel_colore = immagine.get_obj_with(colore=colore_specifico)
                if len(oggetti_con_quel_colore)==0:
                    immagine.aggiungi(Oggetto(colore=colore_specifico,is_specific_ASS_color=True))
                else:
                    for oggetto in oggetti_con_quel_colore:
                        oggetto.is_specific_ASS_color=True
        self.numero_totale_oggetti_out = 0
        for immagine in [self.trainIMM[i]['output'] for i in range(self.train_size)]:
            self.numero_totale_oggetti_out += len(immagine.list_obj)
        self.numero_totale_oggetti_in = 0
        for immagine in [self.trainIMM[i]['input'] for i in range(self.train_size)]:
            self.numero_totale_oggetti_in += len(immagine.list_obj)
        self.somma_totale_aree_oggetti_in = 0
        for immagine in [self.trainIMM[i]['input'] for i in range(self.train_size)]:
            for ogg in immagine.list_obj:
                if ogg.area is not None:
                    self.somma_totale_aree_oggetti_in += ogg.area
            
        immagini_input_num_ogg = [(i, len(self.trainIMM[i]['input'].list_obj)) for i in range(self.train_size)]
        immagini_input_num_ogg.sort(key = lambda x: x[1])
        self.train_input_num_ogg_sorted = [a[0] for a in immagini_input_num_ogg]
        
    def aggiungi_oggetti_denoised(self,iterazioni=2):
        for immagine_in in [self.trainIMM[i]['input'] for i in range(self.train_size)]+[self.testIMM[i]['input'] for i in range(self.test_size)]:
            immagine_in.aggiungi_oggetti_denoised(iterazioni=iterazioni)
    
    def find_shape_mult_se_esistono(self): #prima di creare le Immagini
        poca_nana = ([None for _ in range(self.train_size)], [None for _ in range(self.test_size)])
        shapes_mult_boni_train = []
        for n in range(self.train_size):
            shape_in = (len(self.train[n]['input']),len(self.train[n]['input'][0]))
            shape_out = (len(self.train[n]['output']),len(self.train[n]['output'][0]))
            shape_mult = (int(shape_out[0]/shape_in[0]), int(shape_out[1]/shape_in[1])) 
            if shape_mult==shape_in:
                shapes_mult_boni_train.append(shape_mult)
            else:
                return poca_nana
        shapes_mult_boni_test = []
        for n in range(self.test_size):
            shape_in = (len(self.test[n]['input']),len(self.test[n]['input'][0])) #non devo usare l'output perché non ce l'ho!!
            shapes_mult_boni_test.append(shape_in)
        return shapes_mult_boni_train, shapes_mult_boni_test
            
    ##########################################################################################
    # trova COLORI e FORME comuni a tutti gli input e comuni a tutti gli output    nel TRAIN #
    def find_common_colors(self,in_or_out):
        common_colors=set(range(0,10))
        for i in range(self.train_size):
            tab = self.train[i][in_or_out]
            case_colors = set()
            for row in tab:
                for element in row:
                    case_colors.add(element)
            common_colors = common_colors.intersection(case_colors)
        return common_colors
    def find_common_forms(self,in_or_out): 
        #@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
        #da aggiungere -------> forme comuni a meno di rotazioni <---------
        immagine = self.trainIMM[0][in_or_out]
        common_forms=set(immagine.forma.keys())
        for i in range(1,self.train_size):
            immagine = self.trainIMM[i][in_or_out]
            common_forms = common_forms.intersection(set(immagine.forma.keys()))
        return common_forms
    def find_colors_of(self,forma,in_or_out):
        colori_di_questa_forma_specifica = set()
        for immagine in [self.trainIMM[i][in_or_out] for i in range(self.train_size)]:
            oggetti_con_quella_forma = immagine.get_obj_with(forma=forma)
            for oggetto in oggetti_con_quella_forma:
                colori_di_questa_forma_specifica.add(oggetto.colore)
        return colori_di_questa_forma_specifica
    def find_positions_of(self,forma,in_or_out):
        posizioni_di_questa_forma_specifica = set()
        for immagine in [self.trainIMM[i][in_or_out] for i in range(self.train_size)]:
            oggetti_con_quella_forma = immagine.get_obj_with(forma=forma)
            for oggetto in oggetti_con_quella_forma:
                posizioni_di_questa_forma_specifica.add(oggetto.posizione)
        return posizioni_di_questa_forma_specifica
    def find_forms_of(self,colore,in_or_out):
        forme_di_questo_colore_specifico = set()
        for immagine in [self.trainIMM[i][in_or_out] for i in range(self.train_size)]:
            oggetti_con_quel_colore = immagine.get_obj_with(colore=colore)
            for oggetto in oggetti_con_quel_colore:
                forme_di_questo_colore_specifico.add(oggetto.forma)
        return forme_di_questo_colore_specifico
    
    
    ################################################
    # FUNZIONI PER iterare su oggetti da risolvere #
    def metti_tutti_oggetti_risolvibili(self):
        for i in range(self.train_size):
            immagine_out = self.trainIMM[i]['output']
            for oggetto in immagine_out.list_obj:
                oggetto.is_risolvibile=True
    def get_lista_colori_non_risolti(self):
        oggetti = []
        colori = set()
        for i in range(self.train_size):
            immagine_out = self.trainIMM[i]['output']
            for oggetto in immagine_out.list_obj:
                if not oggetto.is_risolto:
                    colori.add(oggetto.colore)
                    oggetti.append(oggetto)
        return list(colori)
            
    
    ####################################################
    # FUNZIONE PER stampare tutte le immagini del task #
    def stampa(self):
        #stampa Input
        da_stampare = 'Input'
        if len(self.specific['IF']['colors'])>0:
            da_stampare += '    - colori specifici ='+str(self.specific['IF']['colors'])
        if len(self.specific['IF']['forms'])>0:
            da_stampare += '    - forme specifiche ='+str(self.specific['IF']['forms'])
        print(da_stampare)
        for i in range(self.train_size):
            self.trainIMM[i]['input'].stampa()
        #stampa Output
        da_stampare = 'Output'
        if len(self.specific['ASS']['colors'])>0:
            da_stampare += '    - colori specifici ='+str(self.specific['ASS']['colors'])
        if len(self.specific['ASS']['forms'])>0:
            da_stampare += '    - forme specifiche ='+str(self.specific['ASS']['forms'])
        print(da_stampare)
        for i in range(self.train_size):
            self.trainIMM[i]['output'].stampa()
    
    ############################################################
    # FUNZIONI PER VERIFICARE FUNZIONI BAU (le definisco dopo) #
    def check_funzione_copertura(self,fun,test=False,solo_coperture_totali=True, with_modulo=False):
        oggetti_trovati = set()
        errori_fatti = 0
        caselle_coperte = 0
        immagini_coperte = 0
        numero_coppie = self.train_size if not test else self.test_size
        for i in range(numero_coppie):
            immagine_coperta = False #coperta significa che ci ho risolto almeno un oggetto
            immagine_in = self.trainIMM[i]['input'] if not test else self.testIMM[i]['input']
            immagine_out = self.trainIMM[i]['output'] if not test else self.testIMM[i]['output']
            for forma,colore,posizione in fun.applica_a_immagine(immagine_in):
                if with_modulo:
                    posizione = (posizione[0] % immagine_out.shape[0], posizione[1] % immagine_out.shape[1])
                #print('         cago',posizione,colore,forma)
                punti_indovinati, punti_sbagliati, oggetti_output = immagine_out.find_obj_in_tab(forma,colore,posizione,fun.geom,
                                                                                ritorna_solo_coperture_totali=solo_coperture_totali)
                #fatti ritornare quali sono i punti indovinati, magari copre un intero oggetto in più parti!! @@@@@@@@@@@@@@@@@@@@@@@@@@@@ (51)
                caselle_coperte += punti_indovinati
                errori_fatti += punti_sbagliati
                #if errori_fatti>0:
                #    return errori_fatti, caselle_coperte, immagini_coperte, list(oggetti_trovati)
                if len(oggetti_output)>0:
                    immagine_coperta=True
                    for oggetto_output in oggetti_output:
                        oggetti_trovati.add(oggetto_output)
            if immagine_coperta:
                immagini_coperte += 1
        fun.errori_fatti = errori_fatti
        fun.caselle_coperte = caselle_coperte
        fun.immagini_coperte = immagini_coperte
        fun.oggetti_trovati = oggetti_trovati
        return errori_fatti, caselle_coperte, immagini_coperte, list(oggetti_trovati)
    
    

> stampa tutti gli oggetti del task "move red":
> > immagine 0, output

In [ ]:
task_file = str(training_path / training_tasks[7])
with open(task_file, 'r') as f:
    task = json.load(f)
plot_task(task)

t = Task(task,comp_connesse_diagonali=True)

immagine = t.trainIMM[0]['output']
for oggetto in immagine.list_obj:
    oggetto.stampa()

> stampa tutti gli oggetti del task "fill yellow":
> > tutte le immagini, output

In [ ]:
task_file = str(training_path / training_tasks[1])
with open(task_file, 'r') as f:
    task = json.load(f)
plot_task(task)

t = Task(task,comp_connesse_diagonali=True)

for n,immagine in enumerate([t.trainIMM[i]['output'] for i in range(5)] + [t.testIMM[0]['output']]):
    print('Immagine ',n)
    immagine.stampa()


___


<br><br><br>

# Regole

La soluzione finale ad un Task sarà un'insieme di **regole**.


_Ad esempio_ 
> Per il task "move red":
> * "muovi l'oggetto rosso e fagli toccare l'oggetto blu"
> * "lascia l'oggetto blu fermo"
> * "aggiusta lo sfondo"

> Per il task "fill yellow":
> * "colora di giallo l'oggetto nero contenuto nell'oggetto verde"
> * "lascia nero l'oggetto nero non contenuto nell'oggetto verde"
> * "lascia l'oggetto verde fermo"

<h2>Classe Predicato</h2>
Ogni istanza possiede il metodo _.is\_true()_ che prende in input **un** oggetto e restituisce vero/falso

<h2>Classe Relazione</h2>
Ogni istanza possiede il metodo _.is\_true()_ che prende in input **due** oggetti e restituisce vero/falso

<h2>Classe Funzione</h2>
Una **regola** sarà un'istanza della classe Funzione.  
Una **regola** possiede il metodo _.apply()_ che:
* in input -> prende una lista di oggetti
* in output -> restituisce un oggetto (aka. forma,colore,posizione) 

L'applicazione di una regola è divisa in due fasi:
1. <h4>Attivazione</h4>
> <h6>decide se gli oggetti in input sono "ok"</h6>
> * lista di liste di predicati - una lista per ogni oggetto in input
> * lista di liste di relazioni - una lista oer ogni coppia di oggetti in input  
>
> l'attivazione avviene per *congiunzione*
2. <h4>Azione</h4>
> <h6>decide come produrre l'oggetto in output</h6>
> * da quale input prendere la forma
> * da quale input prendere il colore
> * da quali input prendere la posizione (e come!)

numero di input, numero di predicati e relazioni, complessità dell'azione -> **complessità** di una regola


In [ ]:
###########################
#  DEFINISCO LA CLASSE IF #
###########################
class Predicato:
    def __init__(self, attributo='', index=None, val=None, applicazione=None, appl_to_text='',complexity=1):
        self.attributo = attributo
        self.index = index
        self.val = val
        self.applicazione = applicazione
        self.appl_to_text = appl_to_text
        self.complexity = complexity
    def is_True(self,oggetto):
        if oggetto is None: #i None ritornano sempre False
            return False
        attr = getattr(oggetto, self.attributo)
        if attr is None: #i None ritornano sempre False
            return False
        if self.index is not None:
            attr = attr[self.index]
            
        if self.applicazione is None: #cioè attr è già un booleano
            return attr
        
        if self.val is None:
            statement = self.applicazione(attr)
        else:
            statement = self.applicazione(attr,self.val)
        return statement
    def to_text(self):
        ogg_to_text = 'oggetto.'+self.attributo+' ' if self.index is None else 'oggetto.'+self.attributo+'['+str(self.index)+'] '
        val_to_text = '' if self.val is None else ' '+str(self.val)
        return ogg_to_text + self.appl_to_text + val_to_text
    def stampa(self):
        print(self.to_text())
    
    @staticmethod
    def genera(t):
        #GENERA IF SINGOLI
        predicati = []

        #colori
        #predicati += [Predicato(attributo='colore', val=colore, applicazione=lambda x,val: x is val, appl_to_text='is',complexity=1.9) for colore in range(10)]
        predicati.append(Predicato(attributo='colore', applicazione=lambda x: x!=0, appl_to_text='non nero'))  # non nero
        #predicati.append(Predicato(attributo='colore', applicazione=lambda x: x!=5, appl_to_text='non grigio'))  # non nero
        predicati.append(Predicato(attributo='is_rarest_color'))   #predicati.append(Predicato(attributo='is_special_color'))
        predicati.append(Predicato(attributo='is_common_color'))
        #predicati += [Predicato(attributo='area_order_for_color',val=order, applicazione=lambda x,val: x is val, appl_to_text='is',complexity=1.5) for order in range(1)]
        #predicati += [Predicato(attributo='area_order_for_color_rev',val=order, applicazione=lambda x,val: x is val) for order in range(1)]
        predicati.append(Predicato(attributo='is_colorful'))

        #posizioni
        #predicati.append(Predicato(attributo='tocca_un_bordo', applicazione=lambda x: x is True, appl_to_text='is True',complexity=1.5))
        #predicati.append(Predicato(attributo='tocca_un_bordo', applicazione=lambda x: x is False, appl_to_text='is False',complexity=1.5))
        predicati.append(Predicato(attributo='tocca_tutti_i_bordi', applicazione=lambda x: x is True, appl_to_text='is True',complexity=0.9))
        predicati.append(Predicato(attributo='tocca_tutti_i_bordi', applicazione=lambda x: x is False, appl_to_text='is False',complexity=1.5))
        #predicati += [Predicato(attributo='tocca_solo_bordo', index=b,complexity=3) for b in range(4)]
        #predicati.append(Predicato(attributo='tocca_bordo_ORIZ',complexity=3))
        #predicati.append(Predicato(attributo='tocca_bordo_VERT',complexity=3))
        predicati.append(Predicato(attributo='is_lunghissimo',complexity=1))
        predicati.append(Predicato(attributo='is_larghissimo',complexity=1))
        
        #predicati += [Predicato(attributo='posizione', index=0, val=pos, applicazione=lambda x,val: x is val) for pos in range(3)]
        #predicati += [Predicato(attributo='posizione', index=1, val=pos, applicazione=lambda x,val: x is val) for pos in range(3)]

        #forme
        #predicati += [Predicato(attributo='forma',val=forma, applicazione=lambda x,val: x==val, appl_to_text='==',complexity=1.5) for forma in t.specific['IF']['forms'].union(t.specific['ASS']['forms'])]
        #predicati.append(Predicato(attributo='contiene_dueXdue', applicazione=lambda x: x is False, appl_to_text='is False',complexity=3)) # come dire is_linea
        #predicati.append(Predicato(attributo='is_stecca'))
        #predicati.append(Predicato(attributo='is_cornice',complexity=3))
        predicati.append(Predicato(attributo='is_rettangolo',complexity=1.5))
        predicati.append(Predicato(attributo='is_rettangolo', applicazione=lambda x: x is False, appl_to_text='is False',complexity=1.5))
        #predicati.append(Predicato(attributo='is_quadratoso',complexity=1.5))
        #predicati.append(Predicato(attributo='is_bucato',complexity=1.5))
        #predicati += [Predicato(attributo='numero_buchi', val=n, applicazione=lambda x,val: x is val, appl_to_text='is',complexity=3) for n in range(1,4)]
        predicati.append(Predicato(attributo='is_common_form',complexity=1.5))
        #predicati.append(Predicato(attributo='is_common_form', applicazione=lambda x: x is False, appl_to_text='is False',complexity=3))
        ##predicati += [Predicato(attributo='area',val=rang, applicazione=lambda x,val: x in val, appl_to_text='in') for rang in [range(2,5), range(4,8), range(8,16), range(6,20)]] ##########
        #predicati += [Predicato(attributo='area_order',val=order, applicazione=lambda x,val: x is val, appl_to_text='is',complexity=1.6) for order in range(4)]
        #predicati += [Predicato(attributo='area_order_rev',val=order, applicazione=lambda x,val: x is val, appl_to_text='is',complexity=1.6) for order in range(1)]
        
        #predicati.append(Predicato(attributo='is_denoised',complexity=0.9))
        #predicati.append(Predicato(attributo='is_noise',complexity=0.9))
        #predicati.append(Predicato(attributo='is_noise', applicazione=lambda x: x is False, appl_to_text='is False',complexity=0.9)) 
        
        predicati.append(Predicato(attributo='is_shape_mult',complexity=0.9))
        
        #is specific IF ??  (gli specific ASS li assegno dopo solo a oggA e oggB)
        #@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
        # is L
        # is C
        # is cornice
        # non contiene snodi, cioè è un percorso da uno a un altro
        # is convesso e is not convesso (ma non contando i bucato)
        # is convesso up/down/sx/dx
        #  frequenza order of forms 
        #  frequenza order of forms per color
        #  frequenza order of colors per forms (not None solo se una forma compare più di 3 volte?)   task 390
        
        # order per numero di buchi
        
        #predicati.append(Predicato(attributo='is_for_shape_out')) #is oggetto per shape output #GRAVISSIMO ERRORE! CE L'HO SOLO PER I TRAIN
        
        
        def elimina_predicati_inutili(predicati,t):
            #elimina i predicati che valgono sempre e mai     e solo una volta
            new_predicati = []
            for predicato in predicati:
                vale_sempre=True
                vale_mai=True
                vale_massimo_una_volta=True
                for immagine_in in [t.trainIMM[i]['input'] for i in range(t.train_size)]:
                    for oggetto in immagine_in.list_obj:
                        if predicato.is_True(oggetto):
                            if not vale_mai:
                                vale_massimo_una_volta=False
                            vale_mai = False
                        else:
                            vale_sempre = False
                if not vale_sempre and not vale_mai and not vale_massimo_una_volta:
                    new_predicati.append(predicato)
            return new_predicati
        
        def elimina_predicati_uguali(predicati,t):
            new_predicati = []
            for i in range(len(predicati)):
                ne_esiste_uno_uguale = False
                for j in range(i+1,len(predicati)):
                    p1 = predicati[i]
                    p2 = predicati[j]
                    diversi = False
                    for immagine_in in [t.trainIMM[n]['input'] for n in range(t.train_size)] + [t.testIMM[n]['input'] for n in range(t.test_size)]:
                        for oggetto in immagine_in.list_obj:
                            if (p1.is_True(oggetto) and not p2.is_True(oggetto)) or (not p1.is_True(oggetto) and p2.is_True(oggetto)):
                                diversi=True
                    if not diversi:
                        #print(predicati[i].to_text(),'  è uguale AAAAAAAAAAAAAAAAAAAAa  ',predicati[j].to_text())
                        ne_esiste_uno_uguale = True
                if not ne_esiste_uno_uguale:
                    #print(predicati[i].to_text(),'    è unico')
                    new_predicati.append(predicati[i])
            return new_predicati
        
        predicati = elimina_predicati_inutili(predicati, t)
        predicati = elimina_predicati_uguali(predicati, t)
        return predicati

predicato_is_specific_ASS_form = Predicato(attributo='is_specific_ASS_form')
predicato_is_specific_ASS_color = Predicato(attributo='is_specific_ASS_color')
    
    
class Relazione:
    def __init__(self, attributo1='', index1=None, attributo2='', index2=None, val=None, applicazione=None, appl_to_text='', complexity=1):
        self.attributo1 = attributo1
        self.index1 = index1
        self.attributo2 = attributo2
        self.index2 = index2
        self.val = val
        self.applicazione = applicazione
        self.appl_to_text = appl_to_text
        self.complexity = complexity
    def is_True(self,ogg1,ogg2):
        if ogg1 is None or ogg2 is None: # i None ritornano sempre False
            return False
        #trovo attr1
        if self.attributo1=='':
            attr1 = ogg1
        else:
            attr1 = getattr(ogg1, self.attributo1)
            if attr1 is None: # i None ritornano sempre False
                return False
            if self.index1 is not None:
                attr1 = attr1[self.index1]
        #trovo attr2
        if self.attributo2=='':
            attr2 = ogg2
        else:
            attr2 = getattr(ogg2, self.attributo2)
            if attr2 is None: # i None ritornano sempre False
                return False
            if self.index2 is not None:
                attr2 = attr2[self.index2]
        
        if self.val is None:
            statement = self.applicazione(attr1, attr2)
        else:
            statement = self.applicazione(attr1, attr2, self.val)
        return statement
    def to_text(self):
        ogg1_to_text = 'ogg1.'+self.attributo1 if self.index1 is None else 'ogg1.'+self.attributo1+'['+str(self.index1)+']'
        ogg2_to_text = 'ogg2.'+self.attributo2 if self.index2 is None else 'ogg2.'+self.attributo2+'['+str(self.index2)+']'
        val_to_text = '' if self.val is None else ' '+str(self.val)
        return ogg1_to_text+' '+ogg2_to_text+' '+self.appl_to_text+val_to_text
    def stampa(self):
        print(self.to_text())

    @staticmethod
    def genera(t, costanti_posizione=[]):
        #GENERA IF DOPPI
        relazioni = []

        relazioni.append(Relazione(attributo1='forma', attributo2='forma', applicazione=lambda x,y: x==y, appl_to_text='=='))
        relazioni.append(Relazione(attributo1='colore', attributo2='colore', applicazione=lambda x,y: x is y, appl_to_text='is'))
        relazioni.append(Relazione(attributo1='colore', attributo2='colore', applicazione=lambda x,y: x is not y, appl_to_text='is not'))
        relazioni.append(Relazione(attributo1='posizione', attributo2='posizione', applicazione=lambda x,y: x==y, appl_to_text='=='))
        #relazioni.append(Relazione(attributo1='posizione',index1=0, attributo2='posizione',index2=0, applicazione=lambda x,y: x==y, appl_to_text='==')) #troppo generico, forse ci sta metterlo con anche controllo dimmezzi==
        #relazioni.append(Relazione(attributo1='posizione',index1=1, attributo2='posizione',index2=1, applicazione=lambda x,y: x==y, appl_to_text='=='))
        #relazioni.append(Relazione(attributo1='posizione_diag',index1=0, attributo2='posizione_diag',index2=0, applicazione=lambda x,y: x==y, appl_to_text='==',complexity=3))
        #relazioni.append(Relazione(attributo1='posizione_diag',index1=1, attributo2='posizione_diag',index2=1, applicazione=lambda x,y: x==y, appl_to_text='==',complexity=3))
        relazioni.append(Relazione(applicazione=lambda x,y: x.tocca(y), appl_to_text='tocca(.)',complexity=0.9))
        relazioni.append(Relazione(applicazione=lambda x,y: not x.tocca(y), appl_to_text='non_tocca(.)'))
        relazioni.append(Relazione(applicazione=lambda x,y: x.contiene(y), appl_to_text='contiene(.)',complexity=0.9))
        relazioni.append(Relazione(applicazione=lambda x,y: x.contenuto(y), appl_to_text='contenuto(.)',complexity=0.9))
        relazioni.append(Relazione(applicazione=lambda x,y: x.possiede(y), appl_to_text='possiede(.)',complexity=0.9))
        relazioni.append(Relazione(applicazione=lambda x,y: x.appartiene(y), appl_to_text='appartiene(.)',complexity=0.9))
        relazioni.append(Relazione(applicazione=lambda x,y: x in y.fratelli, appl_to_text='fratelli(.)',complexity=0.9))
        #relazioni.append(Relazione(applicazione=lambda x,y: x.dista(y,0), appl_to_text='dista0',complexity=0.9))      #solo quadratini
        #relazioni.append(Relazione(applicazione=lambda x,y: x.dista(y,1), appl_to_text='dista1',complexity=3))      #compl alta
        #relazioni.append(Relazione(applicazione=lambda x,y: x.dista(y,2), appl_to_text='dista2',complexity=3))
        #relazioni.append(Relazione(applicazione=lambda x,y: x.dista(y,3), appl_to_text='dista3',complexity=3))
        relazioni.append(Relazione(applicazione=lambda x,y: x.is_allineato_vert(y), appl_to_text='allineato_vert'))
        relazioni.append(Relazione(applicazione=lambda x,y: x.is_allineato_oriz(y), appl_to_text='allineato_oriz'))

        #relazioni += [Relazione(attributo1='posizione',index1=0, attributo2='posizione',index2=0, val=cost_pos[0], applicazione=lambda x,y,val: x==y+val, appl_to_text='== +') for cost_pos in costanti_posizione]
        #relazioni += [Relazione(attributo1='posizione',index1=1, attributo2='posizione',index2=1, val=cost_pos[1], applicazione=lambda x,y,val: x==y+val, appl_to_text='== +') for cost_pos in costanti_posizione]

        relazioni.append(Relazione(attributo1='area', attributo2='area', applicazione=lambda x,y: x<y, appl_to_text='<',complexity=1.9))
        relazioni.append(Relazione(attributo1='area', attributo2='area', applicazione=lambda x,y: x>y, appl_to_text='>',complexity=1.9))
        
        relazioni.append(Relazione(attributo1='posizione',index1=0, attributo2='posizione',index2=0, applicazione=lambda x,y: x>y, appl_to_text='>',complexity=3))
        relazioni.append(Relazione(attributo1='posizione',index1=1, attributo2='posizione',index2=1, applicazione=lambda x,y: x>y, appl_to_text='>',complexity=3))
        relazioni.append(Relazione(attributo1='posizione',index1=0, attributo2='posizione',index2=0, applicazione=lambda x,y: x<y, appl_to_text='<',complexity=3))
        relazioni.append(Relazione(attributo1='posizione',index1=1, attributo2='posizione',index2=1, applicazione=lambda x,y: x<y, appl_to_text='<',complexity=3))
       
        # è oggetto più vicino di colore diverso (che non tocca!!)
        relazioni.append(Relazione(applicazione=lambda ogg1,ogg2: ogg2 is ogg1.piu_vicino_di_colore_diverso, appl_to_text='più vicino col diverso',complexity=1.5))
        
        #potrebbe essere contenuto in forma
        #potrebbe essere contenuto in buchi della forma
        
        
        def elimina_relazioni_inutili(relazioni,t):
            #elimina le relazioni che valgono sempre e mai     e solo una volta
            new_relazioni = []
            for relazione in relazioni:
                vale_sempre=True
                vale_mai=True
                vale_massimo_una_volta=True
                for immagine_in in [t.trainIMM[i]['input'] for i in range(t.train_size)]:
                    for ogg1 in immagine_in.list_obj:
                        for ogg2 in immagine_in.list_obj:
                            if ogg1 is not ogg2:
                                if relazione.is_True(ogg1,ogg2):
                                    if not vale_mai:
                                        vale_massimo_una_volta=False
                                    vale_mai = False
                                else:
                                    vale_sempre = False
                if not vale_sempre and not vale_mai and not vale_massimo_una_volta:
                    new_relazioni.append(relazione)
            return new_relazioni
        def elimina_relazioni_uguali(relazioni,t):
            new_relazioni = []
            for i in range(len(relazioni)):
                ne_esiste_uno_uguale = False
                for j in range(i+1,len(relazioni)):
                    r1 = relazioni[i]
                    r2 = relazioni[j]
                    diversi = False
                    for immagine_in in [t.trainIMM[n]['input'] for n in range(t.train_size)] + [t.testIMM[n]['input'] for n in range(t.test_size)]:
                        for ogg1 in immagine_in.list_obj:
                            for ogg2 in immagine_in.list_obj:
                                if ogg1 is not ogg2:
                                    if (r1.is_True(ogg1,ogg2) and not r2.is_True(ogg1,ogg2)) or (not r1.is_True(ogg1,ogg2) and r2.is_True(ogg1,ogg2)):
                                        diversi=True
                    if not diversi:
                        #print(r1.to_text(),'  è uguale AAAAAAAAAAAAAAAAAAAAa  ',r2.to_text())
                        ne_esiste_uno_uguale = True
                if not ne_esiste_uno_uguale:
                    #print(relazioni[i].to_text(),'    è unico')
                    new_relazioni.append(relazioni[i])
            return new_relazioni
        
        relazioni = elimina_relazioni_inutili(relazioni, t)
        #relazioni = elimina_relazioni_uguali(relazioni, t)
        return relazioni

        
        
#IF TRIPLI??  t 130, 147
#  @ pos1 sta tra a pos2 e pos3

#IF QUADRUPLI?
#  @ pos1 - pos2 == pos3 - pos4 (posizione relativa uguale)
        
        
        
        
        
        
        

        
        
##################################
#  DEFINISCO LA CLASSE FUNZIONI  #
##################################
class FUN:
    def __init__(self,numero_input, IFs, indexes_supporto_fpos, coefficienti_f_pos_X, coefficienti_f_pos_Y, fpos_const=(0,0), geom=0, is_oggB_None=False, sgn_pos=False):

        if numero_input*3-2 != len(IFs):
            print('dio can qualcosa non va')
        self.numero_input = numero_input
        self.if_singoli = IFs[:numero_input]                          #size = numero_input
        self.if_doppi_A = IFs[numero_input: 2*numero_input-1]         #size = numero_input - 1
        self.if_doppi_B = IFs[2*numero_input-1:numero_input*3-3]                      #size = numero_input - 2
        self.if_doppi_12 = IFs[numero_input*3-3]
        
        self.indexes_supporto_fpos = indexes_supporto_fpos
        self.coefficienti_f_pos = {'x': coefficienti_f_pos_X , 'y': coefficienti_f_pos_Y}  #diz di liste di liste di interi     size=numero_supporto_input x 2
        self.fpos_const = {'x':fpos_const[1], 'y':fpos_const[0]}  #fpos_const = (y,x)
        self.sgn_pos = sgn_pos
        #self.is_simmetric = True
        #for x,y in zip(self.coefficienti_f_pos['x'],self.coefficienti_f_pos['y']):
        #    if x!=y:
        #        self.is_simmetric = False
    
        self.geom = geom
        self.is_oggB_None = is_oggB_None
        for relazioni in self.if_doppi_B:
            if len(relazioni)>0:
                self.is_oggB_None = False
        self.vero_numero_input = numero_input-1 if self.is_oggB_None else numero_input 
        
        #calcola COMPLESSITA funzione
        # prima numero di oggetti in input
        self.complessita = self.vero_numero_input
        self.complessita *= 100
        # poi numero di if totali
        for ifs in IFs:
            for condizione in ifs:
                self.complessita += condizione.complexity
        self.complessita *= 100
        #self.complessita *= 3   #così gli IF valgono quanto 3 coeff fpos non nulli
        # poi numero di coefficienti fpos non nulli
        for coeffs in coefficienti_f_pos_X+coefficienti_f_pos_Y:
            for coeff in coeffs:
                if coeff!=0:
                    self.complessita +=2
            if coeffs==(1,0):
                self.complessita -=1
                    
        #parametri assegnati dopo la valutazione con t.check_funzione_copertura
        self.errori_fatti = 0
        self.caselle_coperte = 0
        self.immagini_coperte = 0
        self.oggetti_trovati = []
        
    def get_IFs(self):
        return self.if_singoli + self.if_doppi_A + self.if_doppi_B + [self.if_doppi_12]
    
    def stampa(self):
        print('fun: di',self.vero_numero_input,'oggetti, complessita=',self.complessita,'       imm=',self.immagini_coperte,'   ogg=',len(self.oggetti_trovati))
        #stampa oggetto A
        oggA = '   ogg A'
        oggA += ' (P)' if (0 in self.indexes_supporto_fpos) else '' 
        for predicato in self.if_singoli[0]:
            oggA += '   '+predicato.to_text()
        print(oggA)
        for relazioni in self.if_doppi_A:
            con_ogg = '      '
            for relazione in relazioni:
                con_ogg += relazione.to_text() + ' '
            print(con_ogg)
        #stampa oggetto B
        if self.is_oggB_None == False:
            oggB = '   ogg B'
            oggB += ' (P)' if (1 in self.indexes_supporto_fpos) else '' 
            for predicato in self.if_singoli[1]:
                oggB += '   '+predicato.to_text()
            print(oggB)
            for relazioni in self.if_doppi_B:
                con_ogg = '      '
                for relazione in relazioni:
                    con_ogg += relazione.to_text() + ' '
                print(con_ogg)
        #stampa altri oggetti
        for i in range(2, self.numero_input):
            ogg = '   ogg '+str(i)
            ogg += ' (P)' if (i in self.indexes_supporto_fpos) else '' 
            for predicato in self.if_singoli[i]:
                ogg += '   '+predicato.to_text()
            print(ogg)
        #stampa coefficienti fposizioni
        print('   supp coeff',self.indexes_supporto_fpos,'  sgn_pos',self.sgn_pos,'  geometry=',geometry_name[self.geom])
        print('   coefficienti X =',self.coefficienti_f_pos['x'], ' +',self.fpos_const['x'])
        print('   coefficienti Y =',self.coefficienti_f_pos['y'], ' +',self.fpos_const['y'])
    
    def f_pos(self,x_or_y,oggetti_input):
        pos = self.fpos_const[x_or_y]
        for i, index_ogg in enumerate(self.indexes_supporto_fpos):
            pos += self.f_pos_singolo(x_or_y, oggetti_input[index_ogg], self.coefficienti_f_pos[x_or_y][i])
        return pos
    def f_pos_singolo(self,x_or_y,oggetto,coeff):
        if oggetto==None:
            return 0
        else:
            if coeff[0]==0 or oggetto.posizione is None:
                if x_or_y == 'y':
                    pos = oggetto.dimmezzi[0]*coeff[1]
                elif x_or_y == 'x':
                    pos = oggetto.dimmezzi[1]*coeff[1]
            else:
                if x_or_y == 'y':
                    pos = oggetto.posizione[0]*coeff[0] + oggetto.dimmezzi[0]*coeff[1]
                elif x_or_y == 'x':
                    pos = oggetto.posizione[1]*coeff[0] + oggetto.dimmezzi[1]*coeff[1]
            return pos
            
    #FUNZIONI PER UTILIZZO
    def possibili_oggetti_input_per(self,immagine_in):  #prende tutti i possibili oggetti che verificano IF singoli e hanno la posizione se serve
        possibili_oggetti_input = [[] for _ in range(self.numero_input)]
        for i in range(self.numero_input):
            if i==1 and self.is_oggB_None:
                possibili_oggetti_input[1] = [None]
                continue
            predicati = self.if_singoli[i]  
            for oggetto in immagine_in.list_obj:
                verifica_tutti_gli_if = True
                for predicato in predicati:          # controlla if singoli
                    if not predicato.is_True(oggetto):
                        verifica_tutti_gli_if = False
                        break
                for index, coeffY, coeffX in zip(self.indexes_supporto_fpos, self.coefficienti_f_pos['y'], self.coefficienti_f_pos['x']):
                    if i==index:
                        if oggetto.dimmezzi is None and (coeffX[1]!=0 or coeffY[1]!=0):
                            verifica_tutti_gli_if = False
                            break
                        elif oggetto.posizione is None and (coeffX[0]!=0 or coeffY[0]!=0):
                            verifica_tutti_gli_if = False
                            break
                        elif oggetto.posizione is None and oggetto.dimmezzi is None: # ma non dovrebbe servire davverooooooo
                            verifica_tutti_gli_if = False
                            break
                #if i in self.indexes_supporto_fpos:  # controlla fposizione
                #    if oggetto.dimmezzi is None:
                #        verifica_tutti_gli_if = False
                if verifica_tutti_gli_if:
                    possibili_oggetti_input[i].append(oggetto)
        #controlla che forma e colore siano presenti dove servono
        if self.is_oggB_None:
            possibili_oggetti_input[0] = [oggA for oggA in possibili_oggetti_input[0] if oggA.forma is not None and oggA.colore is not None]
        else:
            possibili_oggetti_input[0] = [oggA for oggA in possibili_oggetti_input[0] if oggA.forma is not None]
            possibili_oggetti_input[1] = [oggB for oggB in possibili_oggetti_input[1] if oggB.colore is not None]
        #restano da controllare solo gli if doppi  # e di non aver preso due volte lo stesso oggetto!!
        return cartesian(possibili_oggetti_input)
    
    def verificano_if_doppi(self,oggetti_input):
        oggA = oggetti_input[0]
        for relazioni,oggetto in zip(self.if_doppi_A,oggetti_input[1:]):
            for relazione in relazioni:
                if not relazione.is_True(oggA,oggetto):
                    return False
        if not self.is_oggB_None:
            oggB = oggetti_input[1]
            if oggB.colore==oggA.colore: ############################################ sarà una buona idea? boh
                return False
            for relazioni,oggetto in zip(self.if_doppi_B,oggetti_input[2:]):
                for relazione in relazioni:
                    if not relazione.is_True(oggB,oggetto):
                        return False
        if len(oggetti_input)>=4:
            for relazione in self.if_doppi_12:
                if not relazione.is_True(oggetti_input[2],oggetti_input[3]):
                    return False
        return True
    
    def applica_a_immagine(self,immagine_in):
        for oggetti_input in self.possibili_oggetti_input_per(immagine_in):
            #verifica che non ci siano oggetti doppi:
            if len(oggetti_input)!=len(set(oggetti_input)):
                continue
            #verifica if doppi
            if self.verificano_if_doppi(oggetti_input):
                forma = oggetti_input[0].forma
                colore = oggetti_input[0].colore if self.is_oggB_None else oggetti_input[1].colore
                #posizione = (self.f_pos('y',oggetti_input),self.f_pos('x',oggetti_input))
                ogg_in_fpos = [oggetti_input[i] for i in self.indexes_supporto_fpos]
                #posizione = ( valuta_fpos(x_or_y, ogg_in_fpos, self.coefficienti_f_pos[x_or_y], self.fpos_const[x_or_y]) for x_or_y in ['y','x'] )
                
                shape_in_plus = ( immagine_in.shape[0]+1,immagine_in.shape[1]+1 )
                posizione = ( valuta_fpos('y', ogg_in_fpos, self.coefficienti_f_pos['y'], self.fpos_const['y'],
                                          sgn_pos=self.sgn_pos, geom=self.geom, shape_geom=shape_in_plus) ,
                              valuta_fpos('x', ogg_in_fpos, self.coefficienti_f_pos['x'], self.fpos_const['x'],
                                          sgn_pos=self.sgn_pos, geom=self.geom, shape_geom=shape_in_plus)  )
                yield forma,colore,posizione
    
    
    


def valuta_fpos(x_or_y,oggetti_input,coefficienti,const, modulo=None, sgn_pos=False, geom=0, shape_geom=(1,1)):
    ans=const
    sgn = 1
    d = 0 if x_or_y=='y' else 1
    if sgn_pos and len(oggetti_input)==2 and oggetti_input[0].posizione is not None and oggetti_input[1].posizione is not None:
        if oggetti_input[0].posizione[d] - oggetti_input[1].posizione[d] < 0:
            sgn = -1
        elif oggetti_input[0].posizione[d] - oggetti_input[1].posizione[d] == 0: #è un caso di merda da trattare
            sgn = 0
    for ogg_input,coeff in zip(oggetti_input,coefficienti):
        rotated_pos = geometry[geom](ogg_input.posizione[0],ogg_input.posizione[1],shape_geom) if ogg_input.posizione is not None else None
        if coeff[0]==0 and ogg_input.dimmezzi is not None:
            ans += ogg_input.dimmezzi[d]*coeff[1]*sgn
        elif coeff[1]==0 and ogg_input.posizione is not None:
            ans += rotated_pos[d]*coeff[0]
        elif ogg_input.posizione is not None and ogg_input.dimmezzi is not None:
            ans += rotated_pos[d]*coeff[0] + ogg_input.dimmezzi[d]*coeff[1]*sgn
        else:
            return None #come ritornare False poi al controllo ans==
    if modulo is not None:
        ans = ans % modulo
    return ans


<h3>Definisco funzioni a mano e le provo</h3>

In [ ]:
def aggiungi_forcolpos_in_tab(forma,colore,posizione,tab,g,reflect_XY=False):
    dim_tab = (len(tab),len(tab[0]))
    dim_f = (len(forma),len(forma[0]))
    dim_rot_f = geometry[g](0,0,dim_f,to_shape=True) #shape forma eventualmente ruotata
    upsx = (int(posizione[0]-dim_rot_f[0]/2), int(posizione[1]-dim_rot_f[1]/2))
    for x,y in product(range(dim_rot_f[1]), range(dim_rot_f[0])):
        orig_form_y, orig_form_x = geometry_inv[g](y,x,dim_rot_f)
        if forma[orig_form_y][orig_form_x]==1:
            tab_x = upsx[1] + x
            tab_y = upsx[0] + y
            if reflect_XY:
                tab_x,tab_y = tab_y,tab_x
            if 0<=tab_x<dim_tab[1] and 0<=tab_y<dim_tab[0]:
                tab[tab_y][tab_x] = colore
    return tab

def applica(funzione,immagine,old_tab):
    tab = deepcopy(old_tab)
    for forma,col,pos in funzione.applica_a_immagine(immagine):
        tab = aggiungi_forcolpos_in_tab(forma,col,pos,tab,0)
    return tab

In [ ]:
task_da_fare = 242
print('Esempio Task numero: ',task_da_fare)
task_file = str(training_path / training_tasks[task_da_fare])
with open(task_file, 'r') as f:
    task = json.load(f)
plot_task(task)

t = Task(task,comp_connesse_diagonali=False)

#FUNZIONE 1
iffA = [Predicato(attributo='colore', val=0, applicazione=lambda x,val: x is not val, appl_to_text='is not')]
supp_fpos = [0]
fposX=[(1,0)]
fposY=[(1,0)]
funzione1 = FUN(1, [iffA], supp_fpos, fposX, fposY, fpos_const=(0,0), geom=0, is_oggB_None=True, sgn_pos=True)

#FUNZIONE 2
iffA = [Predicato(attributo='colore', val=0, applicazione=lambda x,val: x is val, appl_to_text='is')]
iffB = [Predicato(attributo='colore', val=1, applicazione=lambda x,val: x is val, appl_to_text='is')]
iffAB = [Relazione(applicazione=lambda x,y: x.tocca(y), appl_to_text='tocca')]
supp_fpos = [0]
fposX=[(1,0)]
fposY=[(1,0)]
funzione2 = FUN(2, [iffA,iffB,iffAB,[]], supp_fpos, fposX, fposY, fpos_const=(0,0), geom=0, is_oggB_None=False, sgn_pos=True)

#APPLICAZIONE
tabs = []
for immagineIO in t.trainIMM + t.testIMM:
    immagine = immagineIO['input']
    tabs.append([[0 for _ in range(immagine.shape[1])] for _ in range(immagine.shape[0])])
mostra_tabs(tabs)
for funzione in [funzione1,funzione2]:
    funzione.stampa()
    
    for n,immagineIO in enumerate(t.trainIMM + t.testIMM):
        immagine = immagineIO['input']
        tabs[n] = applica(funzione,immagine,tabs[n])
    mostra_tabs(tabs)
mostra_tabs([immagineIO['output'].tab for immagineIO in t.trainIMM + t.testIMM])

In [ ]:
task_da_fare = 205
print('Esempio Task numero: ',task_da_fare)
task_file = str(training_path / training_tasks[task_da_fare])
with open(task_file, 'r') as f:
    task = json.load(f)
plot_task(task)

t = Task(task,comp_connesse_diagonali=False)

#FUNZIONE 1
iffA = [Predicato(attributo='colore', val=5, applicazione=lambda x,val: x is not val, appl_to_text='is not')]
supp_fpos = [0]
fposX=[(1,0)]
fposY=[(1,0)]
funzione1 = FUN(1, [iffA], supp_fpos, fposX, fposY, fpos_const=(0,0), geom=0, is_oggB_None=True, sgn_pos=True)

#FUNZIONE 2
iffA = [Predicato(attributo='colore', val=0, applicazione=lambda x,val: x is not val, appl_to_text='is not')]
iff2 = [Predicato(attributo='is_colorful')]
iff3 = [Predicato(attributo='colore', val=5, applicazione=lambda x,val: x is val, appl_to_text='is')]
iffA2 = [Relazione(applicazione=lambda x,y: x.appartiene(y), appl_to_text='appartiene')]
supp_fpos = [0,2,3]
fposX=[(1,0),(-1,0),(1,0)]
fposY=[(1,0),(-1,0),(1,0)]
funzione2 = FUN(4, [iffA,[],iff2,iff3,[],iffA2,[],[],[],[]], supp_fpos, fposX, fposY, fpos_const=(0,0), geom=0, is_oggB_None=True, sgn_pos=False)

#APPLICAZIONE
tabs = []
for immagineIO in t.trainIMM + t.testIMM:
    immagine = immagineIO['input']
    tabs.append([[0 for _ in range(immagine.shape[1])] for _ in range(immagine.shape[0])])
mostra_tabs(tabs)
for funzione in [funzione1,funzione2]:
    funzione.stampa()
    
    for n,immagineIO in enumerate(t.trainIMM + t.testIMM):
        immagine = immagineIO['input']
        tabs[n] = applica(funzione,immagine,tabs[n])
    mostra_tabs(tabs)
mostra_tabs([immagineIO['output'].tab for immagineIO in t.trainIMM + t.testIMM])

In [ ]:
task_da_fare = 4
print('Esempio Task numero: ',task_da_fare)
task_file = str(training_path / training_tasks[task_da_fare])
with open(task_file, 'r') as f:
    task = json.load(f)
plot_task(task)

t = Task(task,comp_connesse_diagonali=True)

#FUNZIONE 1
iffA = [Predicato(attributo='area', val=4, applicazione=lambda x,val: x>=val, appl_to_text='>'),
        Predicato(attributo='colore', applicazione=lambda x: x!=0, appl_to_text='non nero')]
supp_fpos = [0]
fposX=[(1,0)]
fposY=[(1,0)]
funzione1 = FUN(1, [iffA], supp_fpos, fposX, fposY, fpos_const=(0,0), geom=0, is_oggB_None=True, sgn_pos=True)

#FUNZIONE 2
iffA = [Predicato(attributo='area', val=4, applicazione=lambda x,val: x>val, appl_to_text='>'),
        Predicato(attributo='colore', applicazione=lambda x: x!=0, appl_to_text='non nero')]
iffB = [Predicato(attributo='area', val=4, applicazione=lambda x,val: x<=val, appl_to_text='<='),
        Predicato(attributo='colore', applicazione=lambda x: x!=0, appl_to_text='non nero')]
supp_fpos = [0,1]
fposX=[(0,-1),(1,1)]
fposY=[(0,-1),(1,1)]
funzione2 = FUN(2, [iffA,iffB,[],[]], supp_fpos, fposX, fposY, fpos_const=(0,0), geom=0, is_oggB_None=False, sgn_pos=True)

#FUNZIONE 3
fposX=[(-1,-2),(2,2)]
fposY=[(-1,-2),(2,2)]
funzione3 = FUN(2, [iffA,iffB,[],[]], supp_fpos, fposX, fposY, fpos_const=(0,0), geom=0, is_oggB_None=False, sgn_pos=True)

#FUNZIONE 4
fposX=[(-2,-3),(3,3)]
fposY=[(-2,-3),(3,3)]
funzione4 = FUN(2, [iffA,iffB,[],[]], supp_fpos, fposX, fposY, fpos_const=(0,0), geom=0, is_oggB_None=False, sgn_pos=True)


#APPLICAZIONE
tabs = []
for immagineIO in t.trainIMM + t.testIMM:
    immagine = immagineIO['input']
    tabs.append([[0 for _ in range(immagine.shape[1])] for _ in range(immagine.shape[0])])
mostra_tabs(tabs)
for funzione in [funzione1,funzione2,funzione3,funzione4]:
    funzione.stampa()
    
    for n,immagineIO in enumerate(t.trainIMM + t.testIMM):
        immagine = immagineIO['input']
        tabs[n] = applica(funzione,immagine,tabs[n])
    mostra_tabs(tabs)
mostra_tabs([immagineIO['output'].tab for immagineIO in t.trainIMM + t.testIMM])

<br><br><br><br>
Una regola _buona_ deve essere:
* **corretta**, cioè non produrre mai oggetti sbagliati
* **generale**, cioè produrre oggetti giusti in almeno due train point (coppie I/O)

> <h4>Lemma(?)</h4>
> Aumentare la complessità dell'*attivazione* -> Fa diminuire gli oggetti prodotti  
>
> Aumentare la complessità dell'*azione* -> Fa aumentare gli oggetti prodotti
<br><br>

# Solver



<h2>Classe Solver</h2>
Trova le regole *buone*. <br>

<h6>PSEUDOCODICE</h6>
**main**
> complessitàMAX = 1
>
> while (c'è ancora tempo):
> > provando con complessita attivazione = 0
> > 
> > aumenta complessità azione finchè non ho **generalità**
> > 
> > restando sotto complessitàMAX
> > 
> > aumenta complessità attivazione finchè non ho **correttezza**
> > 
> > if (non ho perso generalità nel mentre):
> > > nuova regola *buona*!!!
> > >
> > > if (ho risolto tutti i train):
> > > > quit
> > 
> > complessitàMAX += 1

<br>
**def** aumenta_complessita_attivazione(regola):
> for predicato in all_predicati:
> > if (regola + predicato >> regola): #parecchio mejo
> > > aumenta_comlessità_attivazione(regola + predicato)

<br><br>
NOTA: sì, è tutto esponenziale 

In [ ]:
###############################
#  DEFINISCO LA CLASSE SOLVER #
###############################

class Solver:
    def __init__(self,task,BIG_TEMPO,task_ref=None,with_modulo=False,split_come_input=False, add_denoise=None):
        print('Inizializzo solver')
        self.t = task  #classe Task
        self.t_ref = task_ref
        self.BIG_TEMPO = BIG_TEMPO
        self.with_modulo = with_modulo
        
        self.immagini_train_input = [task.trainIMM[i]['input'] for i in range(task.train_size)]
        self.immagini_train_output = [task.trainIMM[i]['output'] for i in range(task.train_size)]
        self.immagini_test_input = [task.testIMM[i]['input'] for i in range(task.test_size)]
        self.immagini_test_output = [task.testIMM[i]['output'] for i in range(task.test_size)] if 'output' in task.testIMM[0].keys() else []
        
        self.Best_Dict = -1
        self.Best_Q1 = -1
        self.Best_Q2 = -1
        self.Best_alias = None
        
        self.shape_input_costante = True
        self.trova_shape_input_type()
        self.shape_output_comeinput = True
        self.shape_output_costante = True
        self.shape_output_comeoggetto = True
        self.shape_output_comeinput_per = True
        self.shape_multiplier = None
        self.shape_output_IGNOTA = False  #    <------------------------------------------------se è True non risolverò mai
        self.trova_shape_output_type()
        self.stampa_shape_type()
        
        
        
        self.magico_ELSE_COLORE = None
        self.magico_ELSE_COLORE_bool = False #vero se DEVO usarlo
        self.magico_ELSE_INPUT = None
        #self.magico_ELSE_INPUT = None
        self.magico_ELSE_INPUT_bool = False  #vero se DEVO usarlo 
        
        if add_denoise is not None:
            self.t.aggiungi_oggetti_denoised(iterazioni=add_denoise)
        
        self.n_coppie_AB_out_iniziali = 0
        self.input_formacolore_e_AB = {} #[oggetto_out]     è (int, listaAB, bool_ho_la_posizione)
        self.occ_input_formacolore = {}  #[immagine][0/1/2] è  int
        oggetti_senza_forma = self.trova_oggetti_AB()
        
        if split_come_input and not self.t.solo_quadratini:
            for oggetto, n_immagine in oggetti_senza_forma:
                immagine_out = self.t.trainIMM[n_immagine]['output']
                pezzi = self.splitta_oggetto_come_input_e_non(oggetto, n_immagine)
                for nuovo_oggetto in pezzi:
                    immagine_out.aggiungi(nuovo_oggetto)
                #print('da (colore',oggetto.colore,')')
                #mostra_tab(oggetto.forma)
                #print('ottengo')
                #print([p.to_text() for p in pezzi])
                #mostra_tabs([p.forma for p in pezzi])
            for immagine_out in self.immagini_train_output:
                immagine_out.refresh_list_obj_from_map()
            oggetti_senza_forma = self.trova_oggetti_AB(priorita=2)
            
        #splitta anche sui diversi colori in input?? boh
            
        self.forme_output_IGNOTE = False #len(oggetti_senza_forma)>0  #    <------------------------------------------------se è True non risolverò mai
        
        
        #genera IF (dopo aver trovato oggetti AB)
        self.costanti_posizione_IF = self.trova_costanti_posizione_IF()
        print('Aggiungo costanti posizioni agli IF',self.costanti_posizione_IF)
        
        self.IFsingoli_list = Predicato.genera(task)
        self.IFdoppi_list = Relazione.genera(task, costanti_posizione=self.costanti_posizione_IF)
        print('Ho',len(self.IFsingoli_list),'predicati e',len(self.IFdoppi_list),'relazioni in TOTALE')
            
        self.is_fattibile = not self.forme_output_IGNOTE and not self.shape_output_IGNOTA
        self.ogg_che_ho_risolto = set()
        self.funzioni = []
        print('è fattibile?',self.is_fattibile)
        print()
        
    def splitta_oggetto_come_input_e_non(self, oggetto, n_imm): #only_same_color True/False
        immagine_in,immagine_out = self.t.trainIMM[n_imm]['input'], self.t.trainIMM[n_imm]['output']
        dimensione = immagine_out.shape
        #print('shape in',immagine_in.shape,' - shape out',immagine_out.shape,'  -> prendo out ',dimensione)
        comp_connesse_diagonali = immagine_in.comp_connesse_diagonali
        visitato = [[False for _ in range(dimensione[1])] for _ in range(dimensione[0])]
        this_oggettoso = [[immagine_out.map_tab_to_obj[y][x] is oggetto for x in range(dimensione[1])] for y in range(dimensione[0])]
        n_pezzi = 0
        pezzi = []
        for x,y in product(range(dimensione[1]),range(dimensione[0])):
            if not visitato[y][x] and this_oggettoso[y][x]:
                #print('nuovo pezzo, parto da',y,x)
                n_pezzi+=1
                obj_forma_big = [[0 for _ in range(dimensione[1])] for _ in range(dimensione[0])]
                #obj_colori_adiacenti = set()
                #obj_oggetti_adiacenti = set()
                obj_tocca_bordo = [False for _ in range(4)]    #up, down, sx, dx     
                def dfs(x,y,come_input):
                    if visitato[y][x] or not this_oggettoso[y][x] or self.magico_ELSE_INPUT[n_imm][y][x]!=come_input:
                        return []
                    visitato[y][x] = True
                    obj_forma_big[y][x] = 1
                    incrementi = []
                    #orizzontali
                    if x>0:
                        incrementi.append([-1,0])
                    else:
                        obj_tocca_bordo[2] = True #tocca sinistra
                    if x+1<dimensione[1]:
                        incrementi.append([1,0])
                    else:
                        if x+1==immagine_out.shape[1]:
                            obj_tocca_bordo[3] = True #tocca destra
                    #verticali
                    if y>0:
                        incrementi.append([0,-1])
                    else:
                        obj_tocca_bordo[0] = True #tocca sopra
                    if y+1<dimensione[0]:
                        incrementi.append([0,1])
                    else:
                        if y+1==immagine_out.shape[0]:
                            obj_tocca_bordo[1] = True #tocca sotto
                    #diagonali
                    if comp_connesse_diagonali:
                        if x>0 and y>0:
                            incrementi.append([-1,-1])
                        if x>0 and y+1<dimensione[0]:
                            incrementi.append([-1,1])
                        if x+1<dimensione[1] and y>0:
                            incrementi.append([1,-1])
                        if x+1<dimensione[1] and y+1<dimensione[0]:
                            incrementi.append([1,1])
                    punti_visitati = [[y,x]]
                    for incr_x,incr_y in incrementi:
                        punti_visitati += dfs(x+incr_x, y+incr_y, come_input)
                    return punti_visitati
                punti_visitati = dfs(x, y, self.magico_ELSE_INPUT[n_imm][y][x])
                if 0<len(punti_visitati)<oggetto.area:
                    #qui obj_forma_big contiene la forma giusta in 0 e 1 ma è una tabella full-dimension
                    #NOTA: il punto (x,y) appartiene all'oggetto ma non è detto che sia in uno dei 4 angoli della forma
                    min_x, max_x, min_y, max_y = dimensione[1], 0, dimensione[0], 0
                    for tmp_x,tmp_y in product(range(dimensione[1]),range(dimensione[0])):
                        if obj_forma_big[tmp_y][tmp_x]==1:
                            min_x = min(min_x,tmp_x)
                            max_x = max(max_x,tmp_x)
                            min_y = min(min_y,tmp_y)
                            max_y = max(max_y,tmp_y)
                    obj_dim_x = max_x-min_x+1
                    obj_dim_y = max_y-min_y+1
                    obj_forma = [[0 for _ in range(obj_dim_x)] for _ in range(obj_dim_y)]
                    for tmp_x,tmp_y in product(range(obj_dim_x),range(obj_dim_y)):
                        obj_forma[tmp_y][tmp_x] = obj_forma_big[min_y+tmp_y][min_x+tmp_x]
                    obj_forma = tuple( tuple(row) for row in obj_forma)
                    #qui obj_forma contiene la forma giusta in 0 e 1 ritagliata in modo da non avere colonne o righe di soli 0
                    obj_posizione = (obj_dim_y/2 + min_y, obj_dim_x/2 + min_x)

                    new_oggetto = Oggetto(forma=obj_forma, colore=oggetto.colore, posizione=obj_posizione,   #MANCANO OGGETTI ADIACENTI!!!!
                                      tocca_bordo=obj_tocca_bordo)
                    pezzi.append(new_oggetto)
                    #mette i puntatori all'oggetto nel tab
                    for p in punti_visitati:
                        immagine_out.map_tab_to_obj[p[0]][p[1]] = new_oggetto #sostituisco i puntatori vecchi
        print('Ho trovato',n_pezzi,'pezzi di colore',oggetto.colore )
        return pezzi
    
    
    def filter_function(self):
        not_bad_fun = []
        for fun in self.funzioni:
            errori, caselle_coperte, immagini_coperte, oggetti_trovati = self.t.check_funzione_copertura(fun,
                                                                                solo_coperture_totali=True, with_modulo=self.with_modulo)
            if errori==0:
                not_bad_fun.append(fun)
        #not_bad_fun.sort(key=lambda fun: 100000000*(100*x[3]+len(x[4]))+x[2] , reverse=True)  #lessicografico per imm_coperte,ogg_trovati,caselle_coperte
        not_bad_fun.sort(key=lambda fun: 10000000*fun.vero_numero_input - 10000*fun.immagini_coperte - len(fun.oggetti_trovati)) 
        #print('FUNZIONI SORTATE')
        #for fun in not_bad_fun:
        all_ogg_trovati = set()
        good_fun = []
        for fun in not_bad_fun:
            old_len = len(all_ogg_trovati)
            for ogg in fun.oggetti_trovati:
                all_ogg_trovati.add(ogg)
            if len(all_ogg_trovati)>old_len:
                good_fun.append(fun)
            #else:
            #    print('SCARTO')
            #    fun.stampa()
            #    print()
        good_fun.sort(key=lambda fun: 10000000*fun.vero_numero_input - 10000*fun.immagini_coperte - len(fun.oggetti_trovati), reverse=True) 
        self.funzioni = good_fun
        
        
    
    def reset_oggetti_risolti_risolvibili(self):
        self.Best_Dict = -1
        self.Best_Q1 = -1
        self.Best_Q2 = -1
        self.Best_alias = None
        self.trova_oggetti_AB()
        #crederci sempre arrendersi mai
        self.t.metti_tutti_oggetti_risolvibili()
    def reset_all(self):
        print('lancio reset all')
        self.ogg_che_ho_risolto = set()
        self.funzioni = []
        for immagine_out in self.immagini_train_output:
            for oggetto in immagine_out.list_obj:
                oggetto.is_risolto = False
        self.input_formacolore_e_AB = {}
        self.reset_oggetti_risolti_risolvibili()
        
    def train_score(self, add_reflected=False):
        def similitudine(tab1,tab2):
            if len(tab1)!=len(tab2) or len(tab1[0])!=len(tab2[0]):
                return 0
            tot,uguali = 0,0
            for line1,line2 in zip(tab1,tab2):
                for p1,p2 in zip(line1,line2):
                    tot+=1
                    if p1==p2:
                        uguali+=1
            #print('AAAAAAAA',uguali,tot)
            return uguali/tot
        score = 0
        for n_train, tab_out in [(i, self.t.trainIMM[i]['output'].tab) for i in range(self.t.train_size)]:
            guessed_tab_out = self.applica(n_train, is_train=True, add_reflected=add_reflected)
            score += similitudine(tab_out,guessed_tab_out) / self.t.train_size
            #print('BBBBBB',score)
        return score
                                      
    def applica(self,where=0, what=None, starting_from=None, is_train=False, add_reflected=False):
        if what is None:
            what=self.funzioni
        n_immagine = where
        immagine_in = self.t.testIMM[n_immagine]['input'] if not is_train else self.t.trainIMM[n_immagine]['input']
        #trova dimensioni tabella
        final_shape = self.trova_shape_output_for(immagine_in)
        #genera tabella
        final_tab = [[0 for _ in range(final_shape[1])] for _ in range(final_shape[0])]
        self.use_magico_ELSE_z(immagine_in,final_tab)
        '''
        if starting_from is None:
            if self.magico_ELSE_COLORE_bool:
                final_tab = [[self.magico_ELSE_COLORE for _ in range(final_shape[1])] for _ in range(final_shape[0])]
            elif self.shape_output_comeinput and self.magico_ELSE_INPUT_bool:
                final_tab = deepcopy(immagine_in.tab)
            else:
                final_tab = [[0 for _ in range(final_shape[1])] for _ in range(final_shape[0])]
        else:
            final_tab = starting_from'''
        def aggiungi_forcolpos_in_tab(forma,colore,posizione,tab,g,reflect_XY=False):
            dim_tab = (len(tab),len(tab[0]))
            dim_f = (len(forma),len(forma[0]))
            dim_rot_f = geometry[g](0,0,dim_f,to_shape=True) #shape forma eventualmente ruotata
            upsx = (int(posizione[0]-dim_rot_f[0]/2), int(posizione[1]-dim_rot_f[1]/2))
            for x,y in product(range(dim_rot_f[1]), range(dim_rot_f[0])):
                orig_form_y, orig_form_x = geometry_inv[g](y,x,dim_rot_f)
                if forma[orig_form_y][orig_form_x]==1:
                    tab_x = upsx[1] + x
                    tab_y = upsx[0] + y
                    if reflect_XY:
                        tab_x,tab_y = tab_y,tab_x
                    if 0<=tab_x<dim_tab[1] and 0<=tab_y<dim_tab[0]:
                        tab[tab_y][tab_x] = colore
            return tab
        #applica funzioni
        for fun in what:
            for forma,colore,posizione in fun.applica_a_immagine(immagine_in):
                if self.with_modulo:
                    posizione = (posizione[0]%final_shape[0], posizione[1]%final_shape[1])
                final_tab = aggiungi_forcolpos_in_tab(forma,colore,posizione,final_tab,fun.geom)
            if add_reflected:
                immagine_in_ref = self.t_ref.testIMM[n_immagine]['input'] if not is_train else self.t_ref.trainIMM[n_immagine]['input']
                for forma,colore,posizione in fun.applica_a_immagine(immagine_in_ref):
                    if self.with_modulo:
                        posizione = (posizione[0]%final_shape[0], posizione[1]%final_shape[1])
                    final_tab = aggiungi_forcolpos_in_tab(forma,colore,posizione,final_tab,fun.geom,reflect_XY=True)
        return final_tab
            #reflected_tab = apply_transf(geometry[6],final_tab) #riflessione x,y  #,inplace=True un giorno lo scrivo
    
    def num_oggetti_non_risolti_con_formacolore(self,immagine_out):
        quanti_oggetti_non_risolti_hanno_formacolore=0
        for ogg_out in immagine_out.list_obj:
            if not ogg_out.is_risolto and ogg_out.is_risolvibile and len(self.input_formacolore_e_AB[ogg_out][1])>0:
                quanti_oggetti_non_risolti_hanno_formacolore += 1
        return quanti_oggetti_non_risolti_hanno_formacolore
        
    def solve_task(self,TEMPO_boost=1, solo_funzioni_generali=True,minimo_numero_immagini=2, fermati_con_else_colore=True, STAMPA=True):
        with_modulo = self.with_modulo
        
        #for ogg1 in self.t.trainIMM[0]['output'].list_obj:
        #    print(self.input_formacolore_e_AB[ogg1])
        if STAMPA:
            for condizione in self.IFsingoli_list+self.IFdoppi_list:
                condizione.stampa()
        
        start_solve_task = time.time()
        #crederci sempre arrendersi mai
        self.t.metti_tutti_oggetti_risolvibili()
        if self.magico_ELSE_COLORE_bool: #se sono costretto a mettere magico ELSE colore allora non ha senso guardare gli oggetti di quel colore
            print('Ho else colore forzato  ---> metto non risolvibili gli oggetti di colore',self.magico_ELSE_COLORE)
            for imm_out in self.immagini_train_output:
                for ogg_out in imm_out.list_obj:
                    if ogg_out.colore is self.magico_ELSE_COLORE:
                        ogg_out.is_risolvibile = False
        #decidi quanto tempo impiegare        
        TEMPO = self.BIG_TEMPO * TEMPO_boost
        #guardo le immagini iniziando da quelle con meno oggetti in input
        def num_coppie_AB(immagine_out):
            n=0
            for ogg_out in immagine_out.list_obj:
                if ogg_out.is_risolvibile and not ogg_out.is_risolto:
                    n += len(self.input_formacolore_e_AB[ogg_out][1])
            return n
        index_boni_immagini = [(index,num_coppie_AB(self.t.trainIMM[index]['output'])) for index in range(self.t.train_size)]
        index_boni_immagini.sort(key = lambda x : x[1])  #ordinate per numero di coppie AB-out
        if index_boni_immagini[self.t.train_size-1][1]==0: #se l'ultimo è 0 allora sono tutti 0 visto che sono in ordine crescente
            print('Bella qualcuno ha già fatto tutto il lavoro per me')
            index_boni_immagini = []
        else:
            print('Ordine con cui guarderò le immagini',[i[0] for i in index_boni_immagini],'con numero oggetti AB', index_boni_immagini)
            ratio = [2/3**i+0.05 for i in range(self.t.train_size)]
            ratio_per_num_ogg = [(index_e_num[0], ratio[order]*index_e_num[1]) for order,index_e_num in enumerate(index_boni_immagini)] #proporzionale a ratio e al numero di coppie che dovrò guardare
            ratio_per_num_ogg.sort(key = lambda x : x[0])  #ordino per index immagini
            ratio_per_num_ogg = [el[1] for el in ratio_per_num_ogg] #mi dimentico degli indici che tanto non servono più
            ratio_per_num_ogg = [el/sum(ratio_per_num_ogg) for el in ratio_per_num_ogg]
            TEMPO_imm = [ratio_per_num_ogg[i]*TEMPO for i in range(self.t.train_size)]  #occhio: è indicizzato dall'indice vero dell'immagine, non dall'ordine in cui le guardo
            print('TEMPO TOTALE =',TEMPO,' -----> ',TEMPO_imm,'a immagine')
            sto_guardando_oggetto_numero, sto_guardando_immagine_numero = 0, 0 #counter utili per stampare chiaramente cosa sta facendo
            index_boni_immagini = [i[0] for i in index_boni_immagini] #qui posso anche dimenticarmi del numero di coppie, tengo solo gli indici ordinati
        trainIMM_sorted = [(self.t.trainIMM[i]['input'], self.t.trainIMM[i]['output'],i) for i in index_boni_immagini]
        fermati = False
        for immagine_in, immagine_out, imm_index in trainIMM_sorted:
            if fermati:
                break
            quanti_oggetti_non_risolti_hanno_formacolore = self.num_oggetti_non_risolti_con_formacolore(immagine_out)
            if quanti_oggetti_non_risolti_hanno_formacolore==0:
                continue
            clock_imm = time.time()
            TEMPO_obj = TEMPO_imm[imm_index] / quanti_oggetti_non_risolti_hanno_formacolore
            print('  INIZIO Immagine',imm_index,'(',sto_guardando_immagine_numero,'^) che ha',quanti_oggetti_non_risolti_hanno_formacolore,'oggetti guardabili')
            print('    tempo totale =',TEMPO_imm[imm_index],' -----> ',TEMPO_obj,'a oggetto')
            
            while immagine_out.esistono_oggetti_risolvibili_non_risolti(self.input_formacolore_e_AB):
                tempo_residuo_immagine = TEMPO_imm[imm_index] - (time.time()-clock_imm)
                if tempo_residuo_immagine<0:
                    print('  TEMPO immagine sgravato')
                    break
                else:
                    quanti_oggetti_non_risolti_hanno_formacolore = self.num_oggetti_non_risolti_con_formacolore(immagine_out)
                    if quanti_oggetti_non_risolti_hanno_formacolore==0:
                        continue
                    else:
                        TEMPO_obj = tempo_residuo_immagine / quanti_oggetti_non_risolti_hanno_formacolore
                #if time.time()-clock_imm > 2*TEMPO_imm[imm_index]:
                #    print('  TEMPO immagine sgravato')
                #    break
                if self.train_score()==1:
                    print('GRAVIDANZA PRECOCE TRAINSCORE=1')
                    break                
                clock_obj = time.time()
                oggetto_out = immagine_out.get_first_obj_risolvibile_non_risolto(self.input_formacolore_e_AB)
                var_numero_oggettiAB = self.input_formacolore_e_AB[oggetto_out][0]
                var_ho_la_posizione = self.input_formacolore_e_AB[oggetto_out][2]
                if STAMPA:
                    print('    ##################################################################################')
                    print('    ##################################################################################')
                    print('    ##################################################################################')
                    print('    ##############  ALOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO  ##############')
                    print('    ##############  --------------------------------------------------  ##############')
                    print('    ##############   OGG',sto_guardando_oggetto_numero,' - immagine',imm_index)
                    print('    ##############       input AB =',self.input_formacolore_e_AB[oggetto_out][0],
                          '  input posizione in AB =',self.input_formacolore_e_AB[oggetto_out][2])
                    print('    ##############       geometry =',self.input_formacolore_e_AB[oggetto_out][3])
                    for linea in oggetto_out.forma:
                        print('    ##############    ',linea)
                    print('    ##############    c =',oggetto_out.colore)
                    print('    ##############    p =',oggetto_out.posizione)
                    print('    ##################################################################################')
                    print('    ##################################################################################')
                    print('    ##################################################################################')
                    print('    ###  TEMPO obj =',TEMPO_obj)
                    
                    
                #if sto_guardando_oggetto_numero==15:
                #    STAMPA=True
                funzioni_papabili = self.solve_obj(oggetto_out, immagine_in, immagine_out, TEMPO_obj,
                                                   solo_funzioni_generali,minimo_numero_immagini, with_modulo, STAMPA)
                self.funzioni += self.computa_funzioni_papabili(funzioni_papabili,STAMPA)
                
                if STAMPA:
                    print()
                    print('    Per oggetto',sto_guardando_oggetto_numero,'ci ho messo', time.time()-clock_obj,'secondi')
                    print('    Colori che mancano',self.t.get_lista_colori_non_risolti())
                    print('    Numero oggetti risolti =',len(self.ogg_che_ho_risolto),'  su un totale di',self.t.numero_totale_oggetti_out)
                    print()
                else:
                    print('    Per oggetto',sto_guardando_oggetto_numero,'ci ho messo', time.time()-clock_obj,'secondi')
                oggetto_out.is_risolvibile = False
                
                if STAMPA:
                    banane = []
                    for n in range(self.t.train_size):
                        banane.append(self.applica(n, is_train=True))
                    for n in range(self.t.test_size):
                        banane.append(self.applica(n))
                    mostra_tabs(banane)
                    #stampa_banane(s,add_reflected=True) ###############################################################################
                
                if fermati_con_else_colore:
                    if self.check_magico_ELSE_z():
                        fermati=True
                        break
                    '''if self.check_magico_ELSE_COLORE():
                        fermati=True
                        break'''
                
                sto_guardando_oggetto_numero += 1
            #fine while esistono oggetti risolvibili da risolvere
            sto_guardando_immagine_numero += 1
        #fine for scelta coppia di immagini da guardare
        if STAMPA:
            print('THE END')
            print()
        if not self.check_magico_ELSE_z():
        #if not self.check_magico_ELSE_COLORE():
            colori_non_risolti = self.t.get_lista_colori_non_risolti()
            print('buh mi sa che ho fallito, restano i colori',colori_non_risolti)
            print('  ho risolto',len(self.ogg_che_ho_risolto),'oggetti su un totale di',self.t.numero_totale_oggetti_out)
            for colore in colori_non_risolti:
                print('   mancano',len(self.t.find_forms_of(colore,'output')),'oggetti di colore', colore)
            #da considerare il caso magico else input
        print('Finisco solve_task con',len(self.funzioni),'funzioni in memoria, ci ho messo',time.time()-start_solve_task)
        #self.filter_function()
        print('Dopo il filtraggio non fatto restano',len(self.funzioni),'funzioni in memoria')
        print()
        if STAMPA:
            print()
            print()
            print()
            print()
            print()
    
    
    def solve_obj(self,oggetto_out,immagine_in,immagine_out,TEMPO_obj,solo_funzioni_generali,minimo_numero_immagini,with_modulo, STAMPA):
        solo_coperture_totali = True
        solo_almeno_due_oggetti = solo_funzioni_generali
        curr_max_n_ogg_trovati = 0 #massimo numero di oggetti risolti da una funzione con 0 errori, quelli sotto non li provo nemmeno
        
        clock_obj = time.time()
        
        numero_oggettiAB = self.input_formacolore_e_AB[oggetto_out][0]
        possibili_oggettiAB = self.input_formacolore_e_AB[oggetto_out][1]
        ho_la_posizione_negli_oggettiAB = self.input_formacolore_e_AB[oggetto_out][2]
        possibili_g = self.input_formacolore_e_AB[oggetto_out][3]
        
        basta_fpos_identita = True if len(immagine_in.get_obj_with(posizione=oggetto_out.posizione))>0 else False
        if self.shape_output_costante and self.t.trainIMM[0]['output'].shape == (1,1):
            basta_fpos_identita = False
        if self.shape_output_costante and not self.shape_output_comeinput:
            basta_fpos_identita = False
        #n_ogg,  (n_ogg_fpos, n_coeff_non_zero, coeff_max),  (n_if_singoli, n_if_coppia, n_if_totali)
        if numero_oggettiAB==1:
            if ho_la_posizione_negli_oggettiAB:
                livelli_incazzatura = [(1,(1,1,0),2),
                                      (2,(1,1,0),3),
                                      (3,(1,1,0),3),
                                      (4,(1,1,0),4)]
            elif basta_fpos_identita:
                livelli_incazzatura = [(2,(1,1,0),2),
                                      (3,(1,1,0),3),
                                      (4,(1,1,0),3)]
            else:
                livelli_incazzatura = [(1,(1,2,2),3),
                                      (2,(2,3,1),3),
                                      #(2,(2,3,2),2),# (2,(2,3,3),2),
                                      (3,(3,3,1),4),
                                      (3,(3,3,2),3),
                                      (4,(4,4,1),5)]
                #livelli_incazzatura = [(1,(1,1,2),2),
                #                      (2,(2,3,1),3)]
        elif numero_oggettiAB==2:
            if ho_la_posizione_negli_oggettiAB:
                livelli_incazzatura = [(2,(1,1,0),3),
                                      (3,(1,1,0),4),
                                      (4,(1,1,0),4)]
            elif basta_fpos_identita:
                livelli_incazzatura = [(3,(1,1,0),6),
                                      (4,(1,1,0),3)]
            else:
                livelli_incazzatura = [(2,(2,3,1),3),
                                      (2,(2,3,2),4), #(2,(2,3,3),4),
                                      (3,(3,3,1),5),
                                      (3,(2,3,2),4),
                                      (4,(3,3,1),4),
                                      (4,(2,3,2),4)]
        else:
            print('Questo non dovrebbe succedere')
            
        ''' ROBBA DELLA SUB DA 0.97 WATCH OUT MATERIALE CHE SCOTTA
        if numero_oggettiAB==1:
            if ho_la_posizione_negli_oggettiAB:
                livelli_incazzatura = [(1,(1,1,0),2),
                                      (2,(1,1,0),3),
                                      (3,(1,1,0),3),
                                      (4,(1,1,0),3)]
            elif basta_fpos_identita:
                livelli_incazzatura = [(2,(1,1,0),2),
                                      (3,(1,1,0),3),
                                      (4,(1,1,0),3)]
            else:
                livelli_incazzatura = [(1,(1,1,2),2),
                                      (2,(2,3,1),3),
                                      (2,(2,3,2),2),
                                      (3,(3,3,1),3),
                                      (3,(3,3,2),3),
                                      (4,(3,3,1),5)]
                #livelli_incazzatura = [(1,(1,1,2),2),
                #                      (2,(2,3,1),3)]
        elif numero_oggettiAB==2:
            if ho_la_posizione_negli_oggettiAB:
                livelli_incazzatura = [(2,(1,1,0),2),
                                      (3,(1,1,0),3),
                                      (4,(1,1,0),3)]
            elif basta_fpos_identita:
                livelli_incazzatura = [(3,(1,1,0),6),
                                      (4,(1,1,0),3)]
            else:
                livelli_incazzatura = [(2,(2,2,1),2),
                                      (2,(2,3,2),4), #(2,(2,3,3),4),
                                      (3,(3,3,1),5),
                                      (3,(2,3,2),4),
                                      (4,(3,3,1),4),
                                      (4,(2,3,2),4)]
                                      
                                      
                                      
                                      
                                  MERDA CASUALE    
            if ho_la_posizione_negli_oggettiAB:
                livelli_incazzatura = [(1,(1,1,0),3),
                                      (2,(1,1,0),5),
                                      (3,(1,1,0),6),
                                      (4,(1,1,0),7)]
            elif basta_fpos_identita:
                livelli_incazzatura = [(2,(1,1,0),4),
                                      (3,(1,1,0),5),
                                      (4,(1,1,0),6)]
            else:
                livelli_incazzatura = [(1,(1,2,2),2),
                                      (2,(2,3,1),3),
                                      (2,(2,3,2),2),
                                      (3,(3,3,1),3),
                                      (3,(3,3,2),3),
                                      (4,(3,3,1),5)]
                #livelli_incazzatura = [(1,(1,1,2),2),
                #                      (2,(2,3,1),3)]
        elif numero_oggettiAB==2:
            if ho_la_posizione_negli_oggettiAB:
                livelli_incazzatura = [(2,(1,1,0),5),
                                      (3,(1,1,0),8),
                                      (4,(1,1,0),6)]
            elif basta_fpos_identita:
                livelli_incazzatura = [(3,(1,1,0),6),
                                      (4,(1,1,0),3)]
            else:
                livelli_incazzatura = [(2,(2,3,1),4),
                                      (2,(2,3,2),4), #(2,(2,3,3),4),
                                      (3,(3,3,1),5),
                                      (3,(2,3,2),4),
                                      (4,(3,3,1),4),
                                      (4,(2,3,2),4)]'''
        
        funzioni_papabili = [[] for _ in range(self.t.train_size + 1)] #divise già per quante immagini coprono
            
        possibili_g_ordinati = list(possibili_g)
        possibili_g_ordinati.sort(reverse=True)
        #find_geo_invariants(tab)
        
        #################################################
        # 
        #
        #### fissa costanti aggiunte #### 
        for fpos_const in [(0,0)] + self.trova_costanti_posizione(): 
            if time.time()-clock_obj > TEMPO_obj:
                if STAMPA:
                    print('      STOP TEMPO 8 fpos const')
                break
            if len(funzioni_papabili[self.t.train_size]) > 0 and self.t.train_size>2:
                if STAMPA:
                    print('      STOP SBAMMMATO')
                break

            #### fissa  aggiunte #### 
            for g in possibili_g_ordinati:    
                if STAMPA:
                    print('    ######################################',geometry_name[g],'############################################')
                if time.time()-clock_obj > TEMPO_obj:
                    if STAMPA:
                        print('      STOP TEMPO 7 geometriess')
                    break
                if len(funzioni_papabili[self.t.train_size]) > 0 and self.t.train_size>2:
                    if STAMPA:
                        print('      STOP SBAMMMATO')
                    break
                    
                    
                for quanti_ne_prendo, quanto_li_sposto, quanto_ci_penso in livelli_incazzatura:
                    if time.time()-clock_obj > TEMPO_obj:
                        if STAMPA:
                            print('      STOP TEMPO 6 incazzatura')
                        break
                    if len(funzioni_papabili[self.t.train_size]) > 0 and self.t.train_size>2:
                        if STAMPA:
                            print('      STOP SBAMMMATO')
                        break
                    #fissa variabili
                    var_numero_oggetti_input = quanti_ne_prendo #valore esatto 
                    var_numero_oggettiEXTRA = var_numero_oggetti_input - numero_oggettiAB
                    #coefficienti fpos
                    var_max_oggetti_per_fpos = quanto_li_sposto[0]          #valore <=
                    var_max_coeff_nonzero_fpos = quanto_li_sposto[1]        #valore <=
                    var_max_coeff_fpos = quanto_li_sposto[2]                #valore <=
                    #print(possibili_fpos_const)
                    #condizioni IF
                    #var_max_ifsingoli_per_oggetto = quanto_ci_penso[0]   #valore <=
                    #var_max_ifdoppi_per_coppia = quanto_ci_penso[1]      #valore <=
                    var_max_numero_if_totali = quanto_ci_penso        #valore <=

                    
                    if STAMPA:
                        print('      ----------------------------------------------------------------------------------')
                        print('      - Funzioni con', var_numero_oggetti_input,'oggetti in input [ sposto=',quanto_li_sposto,' - penso=',quanto_ci_penso,']')
                        print('                         e aggiungo costanti',fpos_const)
                        print('      ----------------------------------------------------------------------------------')

                    numero_input = 2 + var_numero_oggettiEXTRA
                    predicati = [None for _ in range(numero_input)]
                    possibili_predicati = [[] for _ in range(numero_input)]
                    relazioni_oggA_oggB = []
                    relazioni_ogg1_ogg2 = []
                    relazioni_oggA = [None for _ in range(var_numero_oggettiEXTRA)]
                    relazioni_oggB = [None for _ in range(var_numero_oggettiEXTRA)]
                    possibili_relazioni = [[] for _ in range(2*var_numero_oggettiEXTRA+1)]


                
                        
                    def vicinanza(oggAB):
                        oggA,oggB = oggAB
                        if oggB is None:
                            vicinanza = oggetto_out.distanza(oggA)
                            if oggA.is_denoised:
                                vicinanza -=100
                        else:
                            vicinanza = oggetto_out.distanza(oggA)+oggA.distanza(oggB)
                        return vicinanza
                    possibili_oggettiAB.sort(key=vicinanza)
                    #
                    #
                    #### fissa oggetti A e B ####
                    for oggA,oggB in possibili_oggettiAB:
                        if time.time()-clock_obj > TEMPO_obj:
                            if STAMPA:
                                print('      STOP TEMPO 5 oggA e oggB')
                            break
                        if len(funzioni_papabili[self.t.train_size]) > 0 and self.t.train_size>2:
                            if STAMPA:
                                print('      STOP SBAMMMATO')
                            break
                        if not are_equal(oggA.forma,oggetto_out.forma,geometry[g]):
                            continue
                        ##### trova predicati (IF singoli) e relazioni (IF doppi) per Oggetto A e Oggetto B #####
                        if oggB is None:
                            predicati[0] = self.trova_predicati_validi(oggA)
                            predicati[1] = []
                            #aggiunge i controlli IF specifici di assegnamento agli oggetti input che assegnano cose
                            #if oggA.is_specific_ASS_form:
                            #    predicati[0].append(predicato_is_specific_ASS_form)
                            #if oggA.is_specific_ASS_color:
                            #    predicati[0].append(predicato_is_specific_ASS_color)
                            relazioni_oggA_oggB = []
                        else:
                            predicati[0] = self.trova_predicati_validi(oggA)
                            predicati[1] = self.trova_predicati_validi(oggB)
                            #aggiunge i controlli IF specifici di assegnamento agli oggetti input che assegnano cose
                            #if oggA.is_specific_ASS_form:
                            #    predicati[0].append(predicato_is_specific_ASS_form)
                            #if oggB.is_specific_ASS_color:
                            #    predicati[1].append(predicato_is_specific_ASS_color)
                            relazioni_oggA_oggB = self.trova_relazioni_valide(oggA,oggB)

                        if STAMPA:
                            print('      Ho',len(predicati[0]),'predicati per A e',len(predicati[1]),'predicati per B')

                        ##### trova oggetti EXTRA #####
                        #occhio che così NON POSSO PRENDERE 2 oggetti extra con posizione uguale (dai non succederà mai)
                        if basta_fpos_identita and not ho_la_posizione_negli_oggettiAB: #allora devo prendere un oggetto per posizione tra gli EXTRA
                            oggetti_Xposizione = immagine_in.get_obj_with(posizione=oggetto_out.posizione)
                            oggettiEXTRA = [ogg for ogg in immagine_in.list_obj if ogg is not oggA and ogg is not oggB and ogg not in oggetti_Xposizione]
                            possibili_oggettiEXTRA = elenca_solo_sottoinsiemi_di(oggettiEXTRA, cardinalita=var_numero_oggettiEXTRA-1)
                            possibili_oggettiEXTRA = [ [ogg_Xpos]+altri_ogg for ogg_Xpos,altri_ogg in product(oggetti_Xposizione,possibili_oggettiEXTRA) if ogg_Xpos not in altri_ogg]
                        else:
                            oggettiEXTRA = [ogg for ogg in immagine_in.list_obj if ogg is not oggA and ogg is not oggB]
                            possibili_oggettiEXTRA = elenca_solo_sottoinsiemi_di(oggettiEXTRA, cardinalita=var_numero_oggettiEXTRA)
                        
                        # se c'è oggB metti prima oggetti con stesso colore di oggA
                        def vicinanza_ad_AB(sottoinsieme):
                            dist_minima = 100000000
                            for oggE in sottoinsieme:
                                if oggB is None:
                                    dist_minima = min(dist_minima, oggA.distanza(oggE))
                                else:
                                    dist_minima = min(dist_minima, oggA.distanza(oggE), oggB.distanza(oggE))
                            if oggB is None:
                                if len(sottoinsieme)==1:
                                    oggE = sottoinsieme[0]
                                    if oggE.possiede(oggA):
                                        dist_minima -=100 #a CASOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
                                if len(sottoinsieme)==2:
                                    oggE1, oggE2 = sottoinsieme
                                    if oggE1.forma==oggE2.forma:
                                            dist_minima -= 1000
                                    if oggE1.colore==oggE2.colore:
                                            dist_minima -=100
                                    if oggE1.possiede(oggA) or oggE2.possiede(oggA):
                                        dist_minima -= 100
                                    if (oggA in oggE1.fratelli and oggA not in oggE2.fratelli) or (oggA not in oggE1.fratelli and oggA in oggE2.fratelli):
                                        dist_minima -= 100
                                        
                                    #if oggE1.contiene(oggE2) or oggE2.contiene(oggE1):
                                    #    dist_minima -= 100
                            else:
                                if len(sottoinsieme)>0:
                                    oggE = sottoinsieme[0]
                                    if oggA.colore is None:
                                        if oggE.colore==oggB.colore:
                                            dist_minima -= 1 #un po' di vantaggio 
                                    elif oggE.colore==oggA.colore:
                                        dist_minima -= 1000 # meno di ogni altro oggetto con colore diverso
                                        if STAMPA:
                                            print('eeei extr col',oggE.to_text())
                            for oggE in sottoinsieme:
                                if oggE.is_noise: #in teoria sono pochi (sperabilmente solo 1) gli oggetti con questa proprietà
                                    dist_minima -=300
                            return dist_minima
                        
                        if not self.t.solo_quadratini:
                            possibili_oggettiEXTRA.sort(key=vicinanza_ad_AB)
                            #if STAMPA:
                            #    print('possibili extra per AB',oggA.to_text(),[ogg.to_text() for ogg in [oggB] if oggB is not None])
                            #    for oggsE in possibili_oggettiEXTRA:
                            #        print(' -',vicinanza_ad_AB(oggsE),[ogg.to_text() for ogg in oggsE])
                        
                        
                        if self.t.solo_quadratini:
                            def are_vicini(oggsE):
                                if oggA.posizione is None:
                                    return all([oggE.dista(oggA,0) for oggE in oggsE])
                                else:
                                    return all([oggE.dista(oggetto_out,0) for oggE in oggsE])
                            def are_allined(oggsE):
                                #posizione di oggA non può mai essere None (?? vero?)
                                if len(oggsE)==0:
                                    return True
                                if oggA.posizione is None:
                                    return any([oggE.posizione[0]==oggA.posizione[0] or oggE.posizione[1]==oggA.posizione[1] for oggE in oggsE if oggE.posizione is not None])
                                else:
                                    return any([oggE.posizione[0]==oggetto_out.posizione[0] or oggE.posizione[1]==oggetto_out.posizione[1] for oggE in oggsE if oggE.posizione is not None])
                            def shape_mult(oggsE):
                                return any([oggE.is_shape_mult for oggE in oggsE]) if len(oggsE)>0 else True
                            possibili_oggettiEXTRA = [oggsE for oggsE in possibili_oggettiEXTRA if are_vicini(oggsE) or are_allined(oggsE) or shape_mult(oggsE)] #ocho a prendere l'insieme vuoto
                            #possibili_oggettiEXTRA = [oggsE for oggsE in possibili_oggettiEXTRA if are_allined(oggsE)] #ocho a prendere l'insieme vuoto
                            def quanto_sballano(oggsE):
                                if shape_mult(oggsE):
                                    return -3
                                for oggE in oggsE:
                                    if oggE.posizione is None:
                                        continue
                                    if oggE.posizione[0]+1==oggA.posizione[0] and oggE.posizione[1]+1==oggA.posizione[1]:
                                        return -2
                                    for const_pos in self.costanti_posizione_IF:
                                        if oggA.posizione is not None and oggE.posizione is not None:
                                            if oggA.posizione[0]==oggE.posizione[0]+const_pos[0] and oggA.posizione[1]==oggE.posizione[1]+const_pos[1]:
                                                return -1
                                    if oggB is not None and oggA.posizione is not None and oggE.posizione is not None:
                                        if oggE.colore==oggB.colore and (oggA.posizione[0]==oggE.posizione[0] or oggA.posizione[1]==oggE.posizione[1]):
                                            return 0
                                return 1
                            possibili_oggettiEXTRA.sort(key=quanto_sballano)
                            if STAMPA:
                                print('possibili extra per AB',oggA.to_text(),[ogg.to_text() for ogg in [oggB] if oggB is not None])
                                for oggsE in possibili_oggettiEXTRA:
                                    print(' -',quanto_sballano(oggsE),[ogg.to_text() for ogg in oggsE])
                        
                                
                        #
                        #
                        #### fissa oggetti EXTRA ####
                        for oggettiEXTRA in possibili_oggettiEXTRA:
                            if STAMPA and len(oggettiEXTRA)>0:
                                print('    fisso ',len(oggettiEXTRA),'ogg extra, il primo è',oggettiEXTRA[0].to_text())
                            #for oE in oggettiEXTRA:
                            #    print('exx',oE.to_text())
                            if time.time()-clock_obj > TEMPO_obj:
                                if STAMPA:
                                    print('      STOP TEMPO 4 oggEXTRA')
                                break
                            if len(funzioni_papabili[self.t.train_size]) > 0 and self.t.train_size>2:
                                if STAMPA:
                                    print('      STOP SBAMMMATO')
                                break
                            if len(oggettiEXTRA)==2:
                                if oggettiEXTRA[0] is oggettiEXTRA[1]:
                                    print('dio BUBUUUUUUUUUUUUUUUUUU')

                            oggetti_input = [oggA,oggB] + oggettiEXTRA

                            ##### trova predicati (IF singoli) per oggetti EXTRA #####
                            for index,ogg in enumerate(oggettiEXTRA):
                                predicati[index+2] = self.trova_predicati_validi(ogg)

                            ##### trova relazioni (IF doppi) per oggetti EXTRA con A e B #####
                            for index,ogg in enumerate(oggettiEXTRA):
                                relazioni_oggA[index] = self.trova_relazioni_valide(oggA,ogg)
                                relazioni_oggB[index] = self.trova_relazioni_valide(oggB,ogg)
                            relazioni = [relazioni_oggA_oggB] + relazioni_oggA + relazioni_oggB
                            
                            if len(oggettiEXTRA)>=2:
                                relazioni_ogg1_ogg2 = self.trova_relazioni_valide(oggettiEXTRA[0],oggettiEXTRA[1])
                                
                                
                            #salvo predicati e relazioni trovate in una forma "sensata" (non è vero fa cacare ma almeno meglio di come era prima)
                            all_condizioni = []
                            for rel in relazioni_oggA_oggB:
                                all_condizioni.append(((0,1),rel))
                            for rel in relazioni_ogg1_ogg2:
                                all_condizioni.append(((2,3),rel))
                            for i_ogg,rels in enumerate(relazioni_oggA):
                                for rel in rels:
                                    all_condizioni.append(((0,i_ogg+2),rel))
                            for i_ogg,rels in enumerate(relazioni_oggB):
                                for rel in rels:
                                    all_condizioni.append(((1,i_ogg+2),rel))
                            for i_ogg,preds in enumerate(predicati):
                                for pred in preds:
                                    all_condizioni.append(((i_ogg,None),pred))
                                
                                
                            ##### trova oggetti in input per supporto f_posizione #####
                            index_oggetti_usabili_per_posizione = []
                            possibili_indexes_supporto_fpos = []
                            if var_max_coeff_fpos==0:
                                for i,ogg in enumerate(oggetti_input):
                                    if ogg is not None and ogg.posizione is not None and ogg.posizione == oggetto_out.posizione:
                                        index_oggetti_usabili_per_posizione.append(i)
                            else:
                                for i,ogg in enumerate(oggetti_input):
                                    if ogg is not None and (ogg.posizione is not None or ogg.dimmezzi is not None):
                                    #if ogg is not None and ogg.dimmezzi is not None:
                                        index_oggetti_usabili_per_posizione.append(i)
                            #trova i sottoinsiemi
                            for cardinalita in range(1,var_max_oggetti_per_fpos+1):
                                possibili_indexes_supporto_fpos += elenca_solo_sottoinsiemi_di(index_oggetti_usabili_per_posizione, cardinalita=cardinalita)

                            for indexes_supporto_fpos in possibili_indexes_supporto_fpos:
                                if time.time()-clock_obj > TEMPO_obj:
                                    if STAMPA:
                                        print('      STOP TEMPO 3 index fpos')
                                    break
                                if len(funzioni_papabili[self.t.train_size]) > 0 and self.t.train_size>2:
                                    if STAMPA:
                                        print('      STOP SBAMMMATO')
                                    break
                                ##### trova f_posizioni #####
                                oggetti_supporto = [oggetti_input[index] for index in indexes_supporto_fpos]
                                x_target = oggetto_out.posizione[1] - fpos_const[1]
                                y_target = oggetto_out.posizione[0] - fpos_const[0]
                                if len(oggetti_supporto)==2:
                                    if STAMPA:
                                        print('         ',[oggetti_supporto[0].posizione,oggetti_supporto[0].dimmezzi],
                                               [oggetti_supporto[1].posizione,oggetti_supporto[1].dimmezzi],'---->(',y_target,x_target,')')
                                    possibili_sgn_pos = [False,True]
                                else:
                                    possibili_sgn_pos = [False]
                                for sgn_pos in possibili_sgn_pos:
                                    if time.time()-clock_obj > TEMPO_obj:
                                        if STAMPA:
                                            print('      STOP TEMPO 2 sgn pos')
                                        break
                                    if len(funzioni_papabili[self.t.train_size]) > 0 and self.t.train_size>2:
                                        if STAMPA:
                                            print('      STOP SBAMMMATO')
                                        break
                                    #print('cerco fposX',oggetti_supporto[0].posizione,oggetti_supporto[0].dimmezzi,'--->',x_target)
                                    shape_in_plus = ( immagine_in.shape[0]+1,immagine_in.shape[1]+1 )
                                    if with_modulo:
                                        fposizioni_X = self.trova_fposizioni_valide('x', oggetti_supporto, x_target, var_max_coeff_fpos, var_max_coeff_nonzero_fpos,
                                                                                    modulo=immagine_out.shape[1], sgn_pos=sgn_pos, geom=g, shape_geom=shape_in_plus)
                                        fposizioni_Y = self.trova_fposizioni_valide('y', oggetti_supporto, y_target, var_max_coeff_fpos, var_max_coeff_nonzero_fpos,
                                                                                    modulo=immagine_out.shape[0], sgn_pos=sgn_pos, geom=g, shape_geom=shape_in_plus)
                                    else:
                                        fposizioni_X = self.trova_fposizioni_valide('x', oggetti_supporto, x_target, var_max_coeff_fpos, var_max_coeff_nonzero_fpos,
                                                                                    sgn_pos=sgn_pos, geom=g, shape_geom=shape_in_plus)
                                        fposizioni_Y = self.trova_fposizioni_valide('y', oggetti_supporto, y_target, var_max_coeff_fpos, var_max_coeff_nonzero_fpos,
                                                                                    sgn_pos=sgn_pos, geom=g, shape_geom=shape_in_plus)
                                        
                                    
                                    fposizioni_X.sort(key=lambda fposX: 0 if fposX in fposizioni_Y else 1)
                                    fposizioni_Y.sort(key=lambda fposY: 0 if fposY in fposizioni_X else 1)
                                    
                                    if STAMPA:
                                        print('         DC:',indexes_supporto_fpos,'  x',fposizioni_X,'  y',fposizioni_Y, sgn_pos)
                                    def is_good2(sequenza, coeff_max): 
                                        #verifica che coeff_max si verifichi almeno una volta (in valore assoluto) in modo da evitare ripetizioni
                                        massimo_raggiunto = 0
                                        for pair in sequenza:
                                            for elem in pair:
                                                massimo_raggiunto = max(massimo_raggiunto,abs(elem))
                                        return coeff_max == massimo_raggiunto
                                    def is_good3(sequenzaX, sequenzaY):
                                        #verifica che non ci siano oggetti che hanno tutti e 2 i coefficienti nulli sia per X che per Y
                                        for pairX,pairY in zip(sequenzaX,sequenzaY):
                                            if pairX==(0,0) and pairY==(0,0):
                                                return False
                                        return True
                                    #
                                    #
                                    #### fissa coefficienti posizioni, predicati e relazioni ####
                                    for fpos_X, fpos_Y in cartesian([fposizioni_X,fposizioni_Y]):
                                        #print('           fpos',fpos_X,fpos_Y)
                                        if not is_good3(fpos_X,fpos_Y):
                                            continue
                                        if time.time()-clock_obj > TEMPO_obj:
                                            if STAMPA:
                                                print('      STOP TEMPO 1 fpos')
                                            break
                                        if len(funzioni_papabili[self.t.train_size]) > 0 and self.t.train_size>2:
                                            if STAMPA:
                                                print('      STOP SBAMMMATO')
                                            #break ############################################################# ocho al ttiempo


                                        
                                        gia_usato = [False for _ in range(len(all_condizioni))]

                                        #no_IF = [[] for _ in range(3*len(oggetti_input)-3)] 
                                        no_IF = [[] for _ in range(3*len(oggetti_input)-2)]    #aggiunta la relazione ogg1,ogg2     
                                        
                                        if self.t.solo_quadratini:
                                            if oggB is not None and oggB.forma is None and oggB.posizione is None:
                                                print('non parto senza IF perchè nze po ffa')
                                                no_IF[1].append(Predicato(attributo='colore', val=oggB.colore, applicazione=lambda x,val: x is val, appl_to_text='is'))
                                            elif oggB is not None and len(oggettiEXTRA)==1 and oggettiEXTRA[0].colore==oggB.colore:
                                                no_IF[2 + 2*len(oggetti_input)-3].append(Relazione(attributo1='colore', attributo2='colore', applicazione=lambda x,y: x is y, appl_to_text='is')) 
                                        
                                        funzione = FUN(len(oggetti_input), no_IF , indexes_supporto_fpos, fpos_X, fpos_Y, fpos_const=fpos_const,
                                                               geom=g, is_oggB_None=oggB==None, sgn_pos=sgn_pos)

                                        errori, caselle_coperte, immagini_coperte, oggetti_trovati = self.t.check_funzione_copertura(funzione,
                                                                                                            solo_coperture_totali=True, with_modulo=self.with_modulo)
                                        #print(errori,immagini_coperte,oggetti_trovati)
                                        if oggetto_out not in oggetti_trovati:
                                            print()
                                            print()
                                            print('MA GHE OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOH')
                                            print(fpos_X,fpos_Y)
                                        controllo_tempo = (time.time(), TEMPO_obj-time.time()+clock_obj)
                                        if immagini_coperte>=minimo_numero_immagini or (not solo_funzioni_generali and len(oggetti_trovati)>0):
                                            if errori==0:
                                                funzioni_papabili[immagini_coperte].append(funzione)
                                            elif controllo_tempo[1]>0:
                                                if len(oggetti_trovati)>=curr_max_n_ogg_trovati:
                                                    dati_per_fun = (indexes_supporto_fpos, fpos_X, fpos_Y, fpos_const, g, oggB==None, sgn_pos)
                                                    if STAMPA:
                                                        print('         lancio SOLVE_IF, errori',errori,' - imm coperte',immagini_coperte, ' - tempo',controllo_tempo[1])                 
                                                    max_n_ogg_trovati = self.solve_magic_IF(controllo_tempo,0,no_IF,errori,len(oggetti_input),dati_per_fun,funzioni_papabili,
                                                                                            var_max_numero_if_totali,all_condizioni,gia_usato,solo_funzioni_generali,minimo_numero_immagini,False)
                                                    curr_max_n_ogg_trovati = max(curr_max_n_ogg_trovati, max_n_ogg_trovati)
                                                else:
                                                    if STAMPA:
                                                        print('           non lancio SOLVE_IF, pochi ogg risolti',len(oggetti_trovati),'vs',curr_max_n_ogg_trovati)
                                            else:
                                                if STAMPA:
                                                    print('         non ho tempo di lanciare SOLVE_IF')
                                    #fine for scelta fpos
                                #fine for sgn_fpos
                            #fine for scelta oggetti supporto fpos
                            '''
                            if self.solo_boost_ij and self.prova_anche_boost_ij and oggA.posizione is not None:
                                if STAMPA:
                                    print('     e vai col boost')
                                sgn_pos=False #non dovrebbe servire mai True qui
                                index_oggetti_usabili_per_posizione = []
                                for i,ogg in enumerate(oggetti_input):
                                    if i!=0: #NON DEVO PRENDERE OGG A perchè lo userò per il BOOSTij
                                        if ogg is not None and ogg.posizione is not None:
                                            index_oggetti_usabili_per_posizione.append(i)
                                for index_supporto_boost_ij in index_oggetti_usabili_per_posizione:
                                    oggetto_supporto = oggetti_input[index_supporto_boost_ij]
                                    x_ottenuta = immagine_in.shape[1]*oggetto_supporto.posizione[1] + oggA.posizione[1]
                                    y_ottenuta = immagine_in.shape[0]*oggetto_supporto.posizione[0] + oggA.posizione[0]
                                    if x_ottenuta==oggetto_out.posizione[1] and y_ottenuta==oggetto_out.posizione[0]:
                                        if STAMPA:
                                            print('     bono lui')
                                        indexes_supporto_fpos = []
                                        fpos_X = [(1,0)]
                                        fpos_Y = [(1,0)]
                                        if time.time()-clock_obj > TEMPO_obj:
                                            if STAMPA:
                                                print('      STOP TEMPO 3b BOOSTij')
                                            break

                                        gia_usato = [False for _ in range(len(all_condizioni))]

                                        #no_IF = [[] for _ in range(3*len(oggetti_input)-3)] 
                                        no_IF = [[] for _ in range(3*len(oggetti_input)-2)]    #aggiunta la relazione ogg1,ogg2     

                                        if self.t.solo_quadratini:
                                            if oggB is not None and oggB.forma is None and oggB.posizione is None:
                                                no_IF[1].append(Predicato(attributo='colore', val=oggB.colore, applicazione=lambda x,val: x is val, appl_to_text='is'))
                                            elif oggB is not None and len(oggettiEXTRA)==1 and oggettiEXTRA[0].colore==oggB.colore:
                                                no_IF[2 + 2*len(oggetti_input)-3].append(Relazione(attributo1='colore', attributo2='colore', applicazione=lambda x,y: x is y, appl_to_text='is')) 

                                        funzione = FUN(len(oggetti_input), no_IF , indexes_supporto_fpos, fpos_X, fpos_Y, fpos_const=fpos_const,
                                                               geom=g, is_oggB_None=oggB==None, sgn_pos=sgn_pos, boost_ij=index_supporto_boost_ij)

                                        errori, caselle_coperte, immagini_coperte, oggetti_trovati = self.t.check_funzione_copertura(funzione,
                                                                                                            solo_coperture_totali=True, with_modulo=self.with_modulo)
                                        controllo_tempo = (time.time(), TEMPO_obj-time.time()+clock_obj)
                                        if immagini_coperte>=minimo_numero_immagini or (not solo_funzioni_generali and len(oggetti_trovati)>0):
                                            if errori==0:
                                                funzioni_papabili[immagini_coperte].append(funzione)
                                            elif controllo_tempo[1]>0:
                                                if len(oggetti_trovati)>=curr_max_n_ogg_trovati:
                                                    dati_per_fun = (indexes_supporto_fpos, fpos_X, fpos_Y, fpos_const, g, oggB==None, sgn_pos)
                                                    if STAMPA:
                                                        print('         lancio SOLVE_IF boost_ij, errori',errori,' - imm coperte',immagini_coperte, ' - tempo',controllo_tempo[1])                 
                                                    max_n_ogg_trovati = self.solve_magic_IF(controllo_tempo,0,no_IF,errori,len(oggetti_input),dati_per_fun,funzioni_papabili,
                                                                                            var_max_numero_if_totali,all_condizioni,gia_usato,solo_funzioni_generali,minimo_numero_immagini,False,
                                                                                            with_boost_ij=index_supporto_boost_ij)
                                                    curr_max_n_ogg_trovati = max(curr_max_n_ogg_trovati, max_n_ogg_trovati)
                                                else:
                                                    if STAMPA:
                                                        print('           non lancio SOLVE_IF, pochi ogg risolti',len(oggetti_trovati),'vs',curr_max_n_ogg_trovati)
                                            else:
                                                if STAMPA:
                                                    print('         non ho tempo di lanciare SOLVE_IF')   
                                '''
                        #fine for scelta oggettiEXTRA
                    #fine for scelta oggA e oggB
                #fine for scelta constanti fpos
                if STAMPA:
                    print()
                    print('    adesso mi incazzo di più')
                    print('    ..')
                if time.time()-clock_obj > TEMPO_obj:
                    if STAMPA:
                        print('     STOP TEMPO')
                    return funzioni_papabili
            #fine for livelli incazzatura
        #fine for geometry
        return funzioni_papabili
    
    
    def solve_magic_IF(self,tempo,profondita,IF,old_errori,fake_num_oggetti_input,dati_per_fun,funzioni_papabili,max_profondita,all_condizioni,gia_usato,solo_funzioni_generali,minimo_numero_immagini,STAMPA): #una condizione è una coppia ((indice/indici), (predicato/relazione))
        tentativi = []
        curr_max_n_ogg_trovati = 0
        stringa = '             -'
        stringa += '  '*profondita
        #for new_i_ogg,new_cond in all_condizioni:
        for index_cond, (new_i_ogg, new_cond) in enumerate(all_condizioni):  
            if gia_usato[index_cond]:
                continue
            if profondita==0 and new_cond.complexity>1:
                continue
            if profondita<=1 and new_cond.complexity>=3:
                continue
            if time.time()-tempo[0]>tempo[1]:
                if STAMPA:
                    print(stringa,'STOP TEMPO SOLVE MAGIC IF')
                return curr_max_n_ogg_trovati
            if new_i_ogg[1] is None: #è un predicato
                IF[new_i_ogg[0]].append(new_cond) 
            elif new_i_ogg[0] is 0: #è una relazione con oggA
                IF[new_i_ogg[1]+fake_num_oggetti_input-1].append(new_cond) 
            elif new_i_ogg[0] is 1: #è una relazione con oggB
                IF[new_i_ogg[1]+2*fake_num_oggetti_input-3].append(new_cond) 
            elif new_i_ogg[0] is 2: #è una relazione con oggEXTRA1
                IF[3*fake_num_oggetti_input-3].append(new_cond) 

            funzione = FUN(fake_num_oggetti_input, copy_list(IF) , dati_per_fun[0], dati_per_fun[1], dati_per_fun[2], fpos_const=dati_per_fun[3], geom=dati_per_fun[4] ,is_oggB_None=dati_per_fun[5], sgn_pos=dati_per_fun[6])
            #funzione.stampa()
            errori, caselle_coperte, immagini_coperte, oggetti_trovati = self.t.check_funzione_copertura(funzione,
                                                                                                    solo_coperture_totali=True, with_modulo=self.with_modulo)
            if new_i_ogg[1] is None: #è un predicato
                IF[new_i_ogg[0]].pop()
            elif new_i_ogg[0] is 0: #è una relazione con oggA
                IF[new_i_ogg[1]+fake_num_oggetti_input-1].pop()
            elif new_i_ogg[0] is 1: #è una relazione con oggB
                IF[new_i_ogg[1]+2*fake_num_oggetti_input-3].pop() 
            elif new_i_ogg[0] is 2: #è una relazione con oggEXTRA1
                IF[3*fake_num_oggetti_input-3].pop() 
                
            if immagini_coperte>=minimo_numero_immagini or (not solo_funzioni_generali and len(oggetti_trovati)>0): #quindi copro almeno un oggetto
                #if not solo_almeno_due_oggetti or (solo_almeno_due_oggetti and len(oggetti_trovati)>=2):
                if errori==0:
                    funzioni_papabili[immagini_coperte].append(funzione)
                    curr_max_n_ogg_trovati = max(curr_max_n_ogg_trovati,len(oggetti_trovati))
                    #print('+1')
                elif errori<old_errori:
                    tentativi.append(((new_i_ogg,new_cond), immagini_coperte, errori, len(oggetti_trovati), index_cond))
        if time.time()-tempo[0]>tempo[1]:
            if STAMPA:
                print(stringa,'STOP TEMPO SOLVE MAGIC IF')
            return curr_max_n_ogg_trovati
        if len(tentativi)==0:
            if STAMPA:
                print(stringa,'MAGIC IF facevano tutti cahà')
            return curr_max_n_ogg_trovati
        if profondita==max_profondita:
            if STAMPA:
                print(stringa,'MAGIC IF massima profondita')
            return curr_max_n_ogg_trovati
        tentativi.sort(key=lambda x : x[1]*1000000000-x[2], reverse=True)    # sorta per max immagini coperte e poi per min errori
        #tentativi.sort(key=lambda x : x[1]*1000000000 + x[3]*1000000 -x[2], reverse=True)    # sorta per max immagini coperte e poi per max oggetti coperti e poi per min errori
        #tentativi.sort(key=lambda x : x[3]*100000000 -x[2], reverse=True)    # sorta per max oggetti coperti e poi per min errori
        max_immagini_coperte = tentativi[0][1]
        max_n_oggetti_trovati = tentativi[0][3]
        min_errori = tentativi[0][2]
        if STAMPA:
            print(stringa,max_immagini_coperte,max_n_oggetti_trovati,min_errori)
        for tentativo in tentativi:  #################################################### REMINDER quando avrò più tempo: fare questa scrematura solo ogni due livelli
            if tentativo[1]==max_immagini_coperte and tentativo[2]==min_errori:
            #if tentativo[1]==max_immagini_coperte and tentativo[3]==max_n_oggetti_trovati and tentativo[2]==min_errori:
            #if tentativo[3]==max_n_oggetti_trovati and tentativo[2]==min_errori:
                new_i_ogg,new_cond = tentativo[0]
                index_cond = tentativo[4]
                if STAMPA:
                    print(stringa,new_cond.to_text(),new_i_ogg)
                    
                if new_i_ogg[1] is None: #è un predicato
                    IF[new_i_ogg[0]].append(new_cond) 
                elif new_i_ogg[0] is 0: #è una relazione con oggA
                    IF[new_i_ogg[1]+fake_num_oggetti_input-1].append(new_cond) 
                elif new_i_ogg[0] is 1: #è una relazione con oggB
                    IF[new_i_ogg[1]+2*fake_num_oggetti_input-3].append(new_cond) 
                elif new_i_ogg[0] is 2: #è una relazione con oggEXTRA1
                    IF[3*fake_num_oggetti_input-3].append(new_cond) 
                gia_usato[index_cond]=True
                    
                max_n_ogg_trovati = self.solve_magic_IF(tempo,profondita+1,IF,tentativo[2],fake_num_oggetti_input,dati_per_fun,funzioni_papabili,max_profondita,all_condizioni,gia_usato,solo_funzioni_generali,minimo_numero_immagini,STAMPA)
                curr_max_n_ogg_trovati = max(curr_max_n_ogg_trovati, max_n_ogg_trovati)
                
                gia_usato[index_cond]=False
                if new_i_ogg[1] is None: #è un predicato
                    IF[new_i_ogg[0]].pop()
                elif new_i_ogg[0] is 0: #è una relazione con oggA
                    IF[new_i_ogg[1]+fake_num_oggetti_input-1].pop()
                elif new_i_ogg[0] is 1: #è una relazione con oggB
                    IF[new_i_ogg[1]+2*fake_num_oggetti_input-3].pop() 
                elif new_i_ogg[0] is 2: #è una relazione con oggEXTRA1
                    IF[3*fake_num_oggetti_input-3].pop()
            else:
                return curr_max_n_ogg_trovati
        return curr_max_n_ogg_trovati
        
        
    def computa_funzioni_papabili(self,funzioni_papabili,STAMPA):
        new_ALLfunzioni = []
        funzioni = []
        funzioni_facili = []
        funzioni_facili_simmetriche = []
        ogg_che_ho_risolto_adesso = set()
        max_immagini_coperte = 0
        for immagini_coperte in range(self.t.train_size, 0, -1): #da tutte a 1
            if len(funzioni_papabili[immagini_coperte])>0:
                for fun in funzioni_papabili[immagini_coperte]:
                    funzioni.append(fun)
                    for ogg in fun.oggetti_trovati:
                        ogg_che_ho_risolto_adesso.add(ogg)
                max_immagini_coperte = immagini_coperte
                break
        if max_immagini_coperte==0:
            if STAMPA:
                print('      SBAMM un cazzo, non ho trovato nada')
            return []
        if STAMPA:
            print()
            print('       SBAAAMMMMMMMMMMMMMM   buone in ',max_immagini_coperte,'immagini')
            print('          - ho trovato',len(funzioni),'funzioni che in totale risolvono',len(ogg_che_ho_risolto_adesso),'oggetti')
        tot_n_oggetti_risolti_adesso = len(ogg_che_ho_risolto_adesso)
        def complexx(fun):
            nuovi_ogg_trovati = [ogg for ogg in fun.oggetti_trovati if not ogg.is_risolto]
            return 100*(tot_n_oggetti_risolti_adesso-len(nuovi_ogg_trovati))+fun.complessita
            #return 100*(tot_n_oggetti_risolti_adesso-len(fun.oggetti_trovati))+fun.complessita
        funzioni.sort(key = complexx)
        complessita_minima = complexx(funzioni[0])
        funzioni_facili = [fun for fun in funzioni if complexx(fun)==complessita_minima]
        if STAMPA:
            print('      funzioni facili ',len(funzioni),'->',len(funzioni_facili),'       (complessita =',complessita_minima,')')
        funzioni_facili_simmetriche = [fun for fun in funzioni_facili if fun.coefficienti_f_pos['x']==fun.coefficienti_f_pos['y']]
        if STAMPA:
            print('      funzioni simmetriche ',len(funzioni_facili),'->',len(funzioni_facili_simmetriche))

        new_ALLfunzioni =  funzioni_facili if len(funzioni_facili_simmetriche)==0 else funzioni_facili_simmetriche
        forme_coperte_almeno_una_volta, colori_coperti_almeno_una_volta, posizioni_coperte_almeno_una_volta = set(),set(),set()
        for fun in new_ALLfunzioni:
            for ogg in fun.oggetti_trovati:
                #print('risolta ->',ogg.colore,ogg.forma)
                forme_coperte_almeno_una_volta.add(ogg.forma)
                colori_coperti_almeno_una_volta.add(ogg.colore)
                posizioni_coperte_almeno_una_volta.add(ogg.posizione)
                ogg.is_risolto = True
                #print('    metto risolto',ogg.to_text())
                self.ogg_che_ho_risolto.add(ogg)
        if STAMPA:
            print('         -forme out: ',forme_coperte_almeno_una_volta)
            print('         -colori out: ',colori_coperti_almeno_una_volta)
            print('         -posizioni out: ',posizioni_coperte_almeno_una_volta)
            for fun in new_ALLfunzioni[:7]:
                fun.stampa()
        return new_ALLfunzioni
    
    
    
#-------------------------------------------------------------------------------------------#

    def prova_a_generalizzare(self,fun,TEMPO_gen=10,possibili_fpos_const=[(0,0)],STAMPA=False):
        #prendi gli if delle funzioni che swegano
        #fissa quegli IF
        #smattatela provando fpos a rota
        t = self.t
        clock_gen = time.time()
        def trova_fposizioni(oggetti_input, coeff_max=1, max_non_zero_elem=4):
            #funzione ausiliaria per trovare tutti i possibili coefficienti
            def all_sequences_with(length=6, massimo=1):
                if length==1:
                    return [[valore] for valore in range(-massimo,massimo+1)]
                all_sequences = []
                for valore in range(-massimo,massimo+1):
                    sequences = all_sequences_with(length=length-1, massimo=massimo)
                    for sequence in sequences:
                        sequence.append(valore)
                    all_sequences += sequences
                return all_sequences
            fposizioni_raw_pos = all_sequences_with(length=len(oggetti_input), massimo=coeff_max)
            fposizioni_raw_dim = all_sequences_with(length=len(oggetti_input), massimo=coeff_max)
            fposizioni_raw = []
            for seq_pos, seq_dim in product(fposizioni_raw_pos, fposizioni_raw_dim):
                joint_seq = [(seq_pos[i],seq_dim[i]) for i in range(len(oggetti_input))]
                fposizioni_raw.append(joint_seq)
            def is_good1(sequenza,max_non_zero_elem=max_non_zero_elem):
                #verifica che non ci siano troppi elementi diversi da 0
                non_zero_elem = 0
                for pair in sequenza:
                    for elem in pair:
                        if elem!=0:
                            non_zero_elem += 1
                if non_zero_elem <= max_non_zero_elem:
                    return True
                else:
                    return False
            def is_good2(sequenza,coeff_max): 
                massimo_raggiunto = 0
                for pair in sequenza:
                    for elem in pair:
                        massimo_raggiunto = max(massimo_raggiunto,abs(elem))
                return coeff_max==massimo_raggiunto
            '''def is_good3(sequenzaX, sequenzaY): ####### ha senso qui solo se guardo solo le simmetriche!!!!!!
                #verifica che non ci siano oggetti che hanno tutti e 2 i coefficienti nulli sia per X che per Y
                for pairX,pairY in zip(sequenzaX,sequenzaY):
                    if pairX==(0,0) and pairY==(0,0):
                        return False
                return True'''
            fposizioni = [sequenza for sequenza in fposizioni_raw if is_good1(sequenza) and is_good2(sequenza,coeff_max)]
            if len(oggetti_input) == 2: 
                fposizioni.append([(0.5, 0), (0.5, 0)])  #punto medio
            return fposizioni
        new_ALLfunzioni = []
        IF_boni = fun.get_IFs()
        if STAMPA:
            print('-------------')
            print('--------provo a generalizzare funzione')
            fun.stampa()
        indexes_supporto_fpos = fun.indexes_supporto_fpos
        oggetti_supporto = [None for index in indexes_supporto_fpos]
        def find_max(sequenza): 
            massimo_raggiunto = 0
            for pair in sequenza:
                for elem in pair:
                    massimo_raggiunto = max(massimo_raggiunto,abs(elem))
            return massimo_raggiunto
                                    
        coeff_max = int(max(find_max(fun.coefficienti_f_pos['x']), find_max(fun.coefficienti_f_pos['y']))) # coeff_max che amenta finchè trovo funzioni!
        if STAMPA:
            print('La funzione aveva coefficiente massimo = ',coeff_max)
        ho_speranza_di_trovare_funzioni_valide = True
        ho_ancora_speranza_di_trovare_funzioni_valide = True
        #ho_ancora_speranza_di_trovare_funzioni_valide = False
        while ho_speranza_di_trovare_funzioni_valide or ho_ancora_speranza_di_trovare_funzioni_valide and coeff_max<=6:
            if time.time()-clock_gen > TEMPO_gen:
                break
            if not ho_speranza_di_trovare_funzioni_valide:
                ho_ancora_speranza_di_trovare_funzioni_valide = False
            ho_speranza_di_trovare_funzioni_valide = False
            coeff_max += 1
            possibili_fpos_x = trova_fposizioni(oggetti_supporto, coeff_max=coeff_max, max_non_zero_elem=6)
            if STAMPA:
                print('PROVO coeff_max =',coeff_max)
                print('   fpos possibili =',len(possibili_fpos_x))
            #possibili_fpos_y = trova_fposizioni_valide('y', oggetti_supporto, None, coeff_max=coeff_max, max_non_zero_elem=6)
            #print('   fpos possibili =',len(possibili_fpos_x)*len(possibili_fpos_y))
            #for fpos_x,fpos_y in product(possibili_fpos_x,possibili_fpos_y):
            funzioni_papabili = [[] for _ in range(t.train_size + 1)] #divise già per quante immagini coprono
            #print(possibili_fpos_const)
            for fpos_const in possibili_fpos_const:
                if time.time()-clock_gen > TEMPO_gen:
                    break
                for fpos_x,fpos_y in zip(possibili_fpos_x,possibili_fpos_x):
                    if time.time()-clock_gen > TEMPO_gen:
                        break
                    new_fun = FUN(fun.numero_input, IF_boni, indexes_supporto_fpos, fpos_x, fpos_y, fpos_const=fpos_const, is_oggB_None=fun.is_oggB_None,
                                  geom=fun.geom, sgn_pos=fun.sgn_pos)
                    errori, caselle_coperte, immagini_coperte, oggetti_trovati = t.check_funzione_copertura(new_fun,
                                                                                                            solo_coperture_totali=True, with_modulo=self.with_modulo)
                    if errori==0 and immagini_coperte>1: #quindi copro almeno un oggetto
                        #if len(oggetti_trovati)>=2:
                        funzioni_papabili[immagini_coperte].append(new_fun)
            
            #new_new_ALLfunzioni = self.computa_funzioni_papabili(funzioni_papabili,STAMPA)
            new_new_ALLfunzioni = []
            for immagini_coperte in range(self.t.train_size, 1, -1): #da tutte a 2
                if len(funzioni_papabili[immagini_coperte])>0:
                    new_new_ALLfunzioni += funzioni_papabili[immagini_coperte]
            new_ALLfunzioni += new_new_ALLfunzioni
            if STAMPA:
                print('PROVA coeff_max =',coeff_max,' ----->',len(new_new_ALLfunzioni))
            if len(new_new_ALLfunzioni)>0:
                ho_speranza_di_trovare_funzioni_valide = True
                ho_ancora_speranza_di_trovare_funzioni_valide = True
            #if self.check_magico_ELSE_z():
            #    if STAMPA:
            #        print('FINITO')
            #    break
        if STAMPA:
            print('--------fine prova a generalizzare funzione')
            print('-------------')
        return new_ALLfunzioni
    
    
#----------------------------------------------------------------------------------------#
        
    
    #def prova_a_generalizzare_XY(self,fun,TEMPO_gen=10,STAMPA=False):
        
        
        
        
        
        
        
        
        
        
        
        
    ####################
    # shape dell'input #
    def trova_shape_input_type(self):
        for i in range(1,self.t.train_size):
            if self.t.trainIMM[i-1]['input'].shape != self.t.trainIMM[i]['input'].shape:
                self.shape_input_costante = False
        
    #####################
    # shape dell'output #
    def trova_shape_output_type(self):
        #uguale all'input
        for i in range(self.t.train_size):
            if self.t.trainIMM[i]['input'].shape != self.t.trainIMM[i]['output'].shape:
                self.shape_output_comeinput = False
        #costante per ogni output
        for i in range(1,self.t.train_size):
            if self.t.trainIMM[i-1]['output'].shape != self.t.trainIMM[i]['output'].shape:
                self.shape_output_costante = False
        #uguale alla dimensione di un'oggetto
        for i in range(self.t.train_size):
            immagine_input = self.t.trainIMM[i]['input']
            shape_output = self.t.trainIMM[i]['output'].shape
            if shape_output not in immagine_input.dimensione.keys():
                self.shape_output_comeoggetto = False
            else:
                for ogg in immagine_input.dimensione[shape_output]:
                    ogg.is_for_shape_out = True
            #ma quale oggetto? EH
        #uguale a dimensione input * (m,n)
        for m,n in product(range(1,5),range(1,5)):
            self.shape_output_comeinput_per = True
            for i in range(self.t.train_size):
                shape_in, shape_out = self.t.trainIMM[i]['input'].shape, self.t.trainIMM[i]['output'].shape
                if shape_in[0]*m != shape_out[0] or shape_in[1]*n != shape_out[1]:
                    self.shape_output_comeinput_per = False
                    break
            if self.shape_output_comeinput_per:
                self.shape_multiplier = (m,n)
                break
        if not self.shape_output_comeinput and not self.shape_output_costante and not self.shape_output_comeoggetto and not self.shape_output_comeinput_per:
            self.shape_output_IGNOTA = True
            
            self.shape_output_IGNOTA = False #provo comunque a cazzo tosto ??????
            
        #uguale a dimensione input per numero di oggetti / numero di caselle non nere / nere / numero di colori
                
    def stampa_shape_type(self):
        if self.shape_input_costante:
            print('Dimensione Input sempre costante ',self.t.trainIMM[0]['input'].shape)
        if self.shape_output_comeinput:
            print('Dimensione Output sempre uguale all''input')
        if self.shape_output_costante:
            print('Dimensione Output sempre costante ',self.t.trainIMM[0]['output'].shape)
        if self.shape_output_comeoggetto:
            print('Dimensione Output sempre uguale a quella di un oggetto')
        if self.shape_output_comeinput_per:
            print('Dimensione Output sempre uguale all''input per',self.shape_multiplier)
        if self.shape_output_IGNOTA:
            print('Non so che cazzo sia:')
            for i in range(self.t.train_size):
                print('  -',self.t.trainIMM[i]['input'].shape, self.t.trainIMM[i]['output'].shape)
                
    def trova_costanti_posizione_IF(self):  #per if
        costanti_posizioni = []
        if self.shape_output_costante and not self.shape_output_comeinput:
            #possibili_fpos_const += [shape_output] + [ (shape_output[0],0), (0,shape_output[1]) ]
            if self.shape_input_costante:
                shape_input = self.t.trainIMM[0]['input'].shape
                shape_output = self.t.trainIMM[0]['output'].shape
                if shape_output[0]-shape_input[0] != 0 or shape_output[1]-shape_input[1] != 0:
                    costanti_posizioni += [(shape_output[0]-shape_input[0] , shape_output[1]-shape_input[1])]
        return costanti_posizioni
    
    def trova_costanti_posizione(self):
        costanti_posizioni = []  #[(1,0),(0,1)]
        if self.shape_output_comeinput_per and self.shape_multiplier!=(1,1):
            if self.shape_input_costante:
                shape_input = self.t.trainIMM[0]['input'].shape
                shape_output = self.t.trainIMM[0]['output'].shape
                for mult_y,mult_x in product(range(self.shape_multiplier[0]),range(self.shape_multiplier[1])):
                    if mult_y!=0 or mult_x!=0:
                        costanti_posizioni += [(mult_y*shape_input[0] , mult_x*shape_input[1])]
        elif self.shape_output_costante and not self.shape_output_comeinput:
            #possibili_fpos_const += [shape_output] + [ (shape_output[0],0), (0,shape_output[1]) ] #questi li copre la modalità "with_modulo"
            if self.shape_input_costante:
                shape_input = self.t.trainIMM[0]['input'].shape
                shape_output = self.t.trainIMM[0]['output'].shape
                #print('sssshapess',shape_input,shape_output)
                if shape_output[0]-shape_input[0] != 0 and shape_output[1]-shape_input[1] != 0:
                    costanti_posizioni += [(shape_output[0]-shape_input[0] , shape_output[1]-shape_input[1])]
                if shape_output[0]-shape_input[0] != 0:
                    costanti_posizioni += [(shape_output[0]-shape_input[0], 0)] 
                if shape_output[1]-shape_input[1] != 0:
                    costanti_posizioni += [(0, shape_output[1]-shape_input[1])]
        return costanti_posizioni
                
    def trova_shape_output_for(self, immagine_in):
        if self.shape_output_comeinput:
            return immagine_in.shape
        if self.shape_output_costante:
            return self.t.trainIMM[0]['output'].shape
        if self.shape_output_comeinput_per:
            return (immagine_in.shape[0]*self.shape_multiplier[0], immagine_in.shape[1]*self.shape_multiplier[1])
        # gestisco insieme comeoggetto e ignota
        maxY = 1
        maxX = 1
        shapes = set()
        
        for fun in self.funzioni:
            for forma,_,posizione in fun.applica_a_immagine(immagine_in):
                shape_f = (len(forma),len(forma[0]))
                shape_rot_f = geometry[fun.geom](0,0,shape_f,to_shape=True) #shape forma eventualmente ruotata
                maxY = max(maxY,round(shape_rot_f[0]/2+posizione[0]))
                maxX = max(maxX,round(shape_rot_f[1]/2+posizione[1]))
                #maxY = max(maxY,shape_rot_f[0])
                #maxX = max(maxX,shape_rot_f[1])
                shapes.add(shape_rot_f)
        
        if len(shapes)==1:
            return list(shapes)[0]
        return (maxY,maxX) #altrimenti ritorna il lo shape minimo che può contenere tutte le forme generate , se non genero niente ritorna (1,1)
    
    
    ###############
    # magico ELSE #
    def genera_magico_ELSE_INPUT(self): #tabelle grosse come output, casella è True solo se esiste anche in input e è dello stesso colore
        self.magico_ELSE_INPUT = [[[False for _ in range(self.t.trainIMM[i]['output'].shape[1])] for _ in range(self.t.trainIMM[i]['output'].shape[0])] for i in range(self.t.train_size)]
        for i in range(self.t.train_size):
            immagine_in = self.t.trainIMM[i]['input']
            immagine_out = self.t.trainIMM[i]['output']
            for x,y in product( range(min(immagine_in.shape[1],immagine_out.shape[1])), range(min(immagine_in.shape[0],immagine_out.shape[0])) ):
                if immagine_in.tab[y][x]==immagine_out.tab[y][x]:
                    self.magico_ELSE_INPUT[i][y][x]=True
    
    def check_magico_ELSE_z(self):
        self.Best_Dict = -1
        self.Best_Q1 = -1
        self.Best_Q2 = -1
        self.Best_alias = None
        max_Q_to_try = 4
        pairs = []
        for Q1 in range(1,max_Q_to_try+1):
            for Q2 in range(1,max_Q_to_try+1):
                pairs.append((Q1,Q2))
        pairs.sort(key = lambda x:x[0]+x[1])

        for Q1, Q2 in pairs:
            if self.Best_Dict != -1:
                break
            possible = True
            Dict = {}
            for immagine_in, immagine_out in [(self.t.trainIMM[i]['input'],self.t.trainIMM[i]['output']) for i in range(self.t.train_size)]:
                shape_in = immagine_in.shape
                shape_out = immagine_out.shape
                for y_out,x_out in product(range(shape_out[0]), range(shape_out[1])):
                    if not immagine_out.map_tab_to_obj[y_out][x_out].is_risolto:
                        p1 = y_out%Q1
                        p2 = x_out%Q2
                        y_in = y_out % shape_in[0]
                        x_in = x_out % shape_in[1]
                        color1 = immagine_in.map_tab_to_obj[y_in][x_in].colore
                        color2 = immagine_out.map_tab_to_obj[y_out][x_out].colore
                        if color1 != color2:
                            rule = (p1, p2, color1)
                            if rule not in Dict:
                                Dict[rule] = color2
                            elif Dict[rule] != color2:
                                possible = False
            for immagine_in, immagine_out in [(self.t.trainIMM[i]['input'],self.t.trainIMM[i]['output']) for i in range(self.t.train_size)]:
                shape_in = immagine_in.shape
                shape_out = immagine_out.shape
                for y_out,x_out in product(range(shape_out[0]), range(shape_out[1])):
                    if not immagine_out.map_tab_to_obj[y_out][x_out].is_risolto:
                        p1 = y_out%Q1
                        p2 = x_out%Q2
                        y_in = y_out % shape_in[0]
                        x_in = x_out % shape_in[1]
                        color1 = immagine_in.map_tab_to_obj[y_in][x_in].colore

                        rule = (p1,p2,color1)

                        if rule in Dict:
                            color2 = 0 + Dict[rule]
                        else:
                            color2 = 0 + color1
                        if color2 != immagine_out.map_tab_to_obj[y_out][x_out].colore:
                            possible = False 
            if possible:
                self.Best_Dict = Dict
                self.Best_Q1 = Q1
                self.Best_Q2 = Q2
        if self.Best_Dict == -1:
            return False #meaning that we didn't find a rule that works
        else:
            unique_values = set(self.Best_Dict.values())
            if len(unique_values)==1:
                self.Best_alias = list(unique_values)[0]
            print('(check) Metto magico ELSE z -',self.Best_Q1,self.Best_Q2,self.Best_Dict, self.Best_alias)
            return True
    
    def use_magico_ELSE_z(self,immagine_in,tab):
        if self.Best_Dict == -1:
            return
        shape_in = immagine_in.shape
        shape_out = (len(tab),len(tab[0]))
        for y_out,x_out in product(range(shape_out[0]), range(shape_out[1])):
            p1 = y_out % self.Best_Q1
            p2 = x_out % self.Best_Q2
            y_in = y_out % shape_in[0]
            x_in = x_out % shape_in[1]
            color1 = immagine_in.map_tab_to_obj[y_in][x_in].colore
            rule = (p1, p2, color1)
            if rule in self.Best_Dict:
                tab[y_out][x_out] = 0 + self.Best_Dict[rule]
            elif self.Best_alias is not None:
                tab[y_out][x_out] = 0 + self.Best_alias
            else:
                tab[y_out][x_out] = 0 + color1

    
    #########################################
    # trova oggetti AB (per forma e colore) #
    # aggiusta dizionari self.input_formacolore_e_AB e self.occ_input_formacolore
    def trova_oggetti_AB(self, priorita=1):
        self.genera_magico_ELSE_INPUT()
        
        oggetti_che_proprio_boh = set()
        forme_merdose_non_al_bordo = set()
        n_coppie_AB_out = 0
        #for immagine_in,immagine_out in zip(self.immagini_train_input, self.immagini_train_output):
        for n_immagine in range(self.t.train_size):
            immagine_in = self.immagini_train_input[n_immagine]
            immagine_out = self.immagini_train_output[n_immagine]
            self.occ_input_formacolore[immagine_out] = [0,0,0]
            for oggetto_out in immagine_out.list_obj:
                if self.t.solo_quadratini and oggetto_out.posizione[0]<immagine_in.shape[0] and oggetto_out.posizione[1]<immagine_in.shape[1] and oggetto_out.colore == immagine_in.tab[int(oggetto_out.posizione[0])][int(oggetto_out.posizione[1])]:
                    oggetto_out.is_risolto = True
                if oggetto_out.is_risolto:
                    self.input_formacolore_e_AB[oggetto_out] = (0,[],False,set(),None)
                    self.occ_input_formacolore[immagine_out][0]+=1
                    continue
                if oggetto_out in self.input_formacolore_e_AB.keys():
                    continue
                ogg_Xforma = immagine_in.get_obj_with(forma=oggetto_out.forma)
                ogg_Xcolore = immagine_in.get_obj_with(colore=oggetto_out.colore)
                if self.shape_output_costante and self.t.trainIMM[0]['output'].shape == (1,1):
                    ogg_Xposizione = set()
                else:
                    ogg_Xposizione = immagine_in.get_obj_with(posizione=oggetto_out.posizione)
                geo_Xforma = set()

                #guarda se ci sono forme mai viste :(
                if len(ogg_Xforma)==0:
                    occorrenze_forma = 0 #contate senza ripetizioni nella singola immagine
                    for immagine in self.immagini_train_output: #guardo output
                        oggetti_con_quella_forma = immagine.get_obj_with(forma=oggetto_out.forma)
                        if len(oggetti_con_quella_forma)>0:
                            occorrenze_forma += 1
                    if occorrenze_forma >= 2 and oggetto_out.colore!=0: #se la forma NON è NERA e compare in almeno 2 immagini input allora la aggiungo come "FORMA SPECIFICA"
                        colori_di_questa_forma_specifica = list(self.t.find_colors_of(oggetto_out.forma,'output'))
                        colore = colori_di_questa_forma_specifica[0] if len(colori_di_questa_forma_specifica)==1 else None
                        posizioni_di_questa_forma_specifica = list(self.t.find_positions_of(oggetto_out.forma,'output'))
                        posizione = posizioni_di_questa_forma_specifica[0] if len(posizioni_di_questa_forma_specifica)==1 else None
                        #print('---< forma sconosciuta >--- ma almeno ricorrente e con colore',colore,'e posizione',posizione)
                        self.t.specific['ASS']['forms'].add(oggetto_out.forma)  #aggiungo a forme specifiche (così poi ci genero anche l'if singolo)
                        for immagine in self.immagini_train_input + self.immagini_test_input: #guardo input
                            oggetti_con_quella_forma = immagine.get_obj_with(forma=oggetto_out.forma)
                            if len(oggetti_con_quella_forma)==0:
                                nuovo_oggetto = Oggetto(forma=oggetto_out.forma,colore=colore,posizione=posizione,is_specific_ASS_form=True)
                                immagine.aggiungi(nuovo_oggetto)
                                if immagine is immagine_in:
                                    ogg_Xforma.add(nuovo_oggetto)
                                    if colore is not None:
                                        ogg_Xcolore.add(nuovo_oggetto) #colore deve essere uguale a oggetto_out.colore
                                    if posizione is not None:
                                        ogg_Xposizione.add(nuovo_oggetto) #posizione deve essere uguale a oggetto_out.posizione
                                #print('aggiunto',oggetto_out.forma,'in input     con colore',colore,'e posizione',posizione)
                            else:
                                for oggetto in oggetti_con_quella_forma:
                                    oggetto.is_specific_ASS_form=True
                        geo_Xforma.add(0)
                    else:
                        for oggetto_in in immagine_in.list_obj:
                            if oggetto_in.area==oggetto_out.area:
                                for g in range(1,8):
                                    if are_equal(oggetto_in.forma, oggetto_out.forma, geometry[g]):
                                        #print(geometry_name[g],g)
                                        #mostra_tabs([oggetto_in.forma, oggetto_out.forma])
                                        ogg_Xforma.add(oggetto_in)
                                        geo_Xforma.add(g)
                else:
                    geo_Xforma.add(0)
                            
                #guarda se ci sono colori mai visti
                if len(ogg_Xcolore)==0:
                    #print('---< colore sconosciuta >--- ma almeno ricorrente',oggetto_out.colore)
                    self.t.specific['ASS']['colors'].add(oggetto_out.colore)  #aggiungo a colori specifici
                    for immagine in self.immagini_train_input + self.immagini_test_input: #guardo input
                        oggetti_con_quel_colore = immagine.get_obj_with(colore=oggetto_out.colore)
                        if len(oggetti_con_quel_colore)==0:
                            nuovo_oggetto = Oggetto(colore=oggetto_out.colore,is_specific_ASS_color=True)
                            immagine.aggiungi(nuovo_oggetto)
                            if immagine is immagine_in:
                                ogg_Xcolore.add(nuovo_oggetto) #quindi ogg_Xcolore non sarà mai vuota dopo
                            #print('aggiunto',oggetto_out.colore,'in input')
                        else:
                            for oggetto in oggetti_con_quel_colore:
                                oggetto.is_specific_ASS_color=True

                #swegata posizione forse non ha sensoooooooooooooooooooooooooooooooooo
                ho_la_posizione = False
                
                if True or not self.t.solo_quadratini:
                    if len(ogg_Xposizione.intersection(ogg_Xforma))>0:
                        ogg_Xforma = ogg_Xposizione.intersection(ogg_Xforma)
                        ho_la_posizione = True
                    if len(ogg_Xposizione.intersection(ogg_Xcolore))>0:
                        ogg_Xcolore = ogg_Xposizione.intersection(ogg_Xcolore)
                        ho_la_posizione = True

                if len(ogg_Xforma)>0 and len(ogg_Xcolore)>0:
                    #se conosco sia forma che colore di quell'oggetto_out
                    ogg_Xformacolore = list(ogg_Xforma.intersection(ogg_Xcolore))
                    if len(ogg_Xformacolore)>0:
                        input_formacolore = zip(ogg_Xformacolore, [None for _ in range(len(ogg_Xformacolore))])
                        input_formacolore = list(input_formacolore)
                        n_coppie_AB_out+=len(input_formacolore)
                        self.input_formacolore_e_AB[oggetto_out] = (1,input_formacolore,ho_la_posizione,geo_Xforma,priorita)
                        self.occ_input_formacolore[immagine_out][1]+=1
                    else:
                        ogg_Xcolore = list(ogg_Xcolore)
                        if self.t.solo_quadratini:
                            def allineato(ogg):
                                if ogg.posizione is None:
                                    return 1
                                if ogg.posizione[0]==oggetto_out.posizione[0] or ogg.posizione[1]==oggetto_out.posizione[1]:
                                    return 0
                                return 1
                            ogg_Xcolore.sort(key=allineato)
                        input_formacolore = product(list(ogg_Xforma),ogg_Xcolore)
                        input_formacolore = list(input_formacolore)
                        n_coppie_AB_out+=len(input_formacolore)
                        self.input_formacolore_e_AB[oggetto_out] = (2,input_formacolore,ho_la_posizione,geo_Xforma,priorita)
                        self.occ_input_formacolore[immagine_out][2]+=1
                else:
                    #se non so come ricostruire quell'oggetto_out (aka forma merdosa)
                    self.input_formacolore_e_AB[oggetto_out] = (0,[],False,geo_Xforma,priorita)
                    self.occ_input_formacolore[immagine_out][0]+=1
                    if not oggetto_out.is_come_input:
                        oggetti_che_proprio_boh.add((oggetto_out,n_immagine))
                    if not oggetto_out.tocca_un_bordo:
                        forme_merdose_non_al_bordo.add(oggetto_out)
        #print('COPPIE AB-OUT in totale =',n_coppie_AB_out)
        self.n_coppie_AB_out_iniziali = n_coppie_AB_out
        
        # oggetti in output composti solo dalle cose che non sono in input?
        # cioè: per gli oggetti not .is_come_input e con forma ignota creo oggetto/i in output che è il 'ritaglio ai punti' comuni/non comuni all'input

        
        geo_usate = set()
        geo_comuni = set(range(1,8))
        for _,_,_,geo_Xforma,_ in self.input_formacolore_e_AB.values():
            if len(geo_Xforma)>0 and not 0 in geo_Xforma:
                geo_usate = geo_usate.union(geo_Xforma)
                geo_comuni = geo_comuni.intersection(geo_Xforma)
        #print('Geometrie - comuni=',geo_comuni,' usate=',geo_usate)
        if len(geo_comuni)>0:
            #print('   uso solo le comuni')
            for ogg_out in self.input_formacolore_e_AB.keys():
                if len(self.input_formacolore_e_AB[ogg_out][3])>0 and not 0 in self.input_formacolore_e_AB[ogg_out][3]:
                    self.input_formacolore_e_AB[ogg_out] = ( self.input_formacolore_e_AB[ogg_out][0], self.input_formacolore_e_AB[ogg_out][1], self.input_formacolore_e_AB[ogg_out][2],
                                                            self.input_formacolore_e_AB[ogg_out][3].intersection(geo_comuni),  self.input_formacolore_e_AB[ogg_out][4] )
                    if len(self.input_formacolore_e_AB[ogg_out][3])==0:
                        print('mannaggia a jesussss')
        #print('after commm')
        for _,_,_,geo_Xforma,_ in self.input_formacolore_e_AB.values():
            if len(geo_Xforma)>0 and not 0 in geo_Xforma:
                print(geo_Xforma)               
        
        #if len(oggetti_che_proprio_boh)==0:
        #    print('Bella le forme (non come input) ce le ho tutteee')
        #else:
        #    print('Ci sono delle forme in output che proprio non saprei risolvere, mi disp')
    
        return oggetti_che_proprio_boh
    
    
    
    
    
    
    
    #######################
    # selezione predicati #
    def trova_predicati_validi(self,oggetto):
        predicati_validi = []
        for predicato in self.IFsingoli_list:
            #controlla che il predicato sia verificato per oggetto
            if predicato.is_True(oggetto):
                predicati_validi.append(predicato)
        return predicati_validi
    
    #######################
    # selezione relazioni #
    def trova_relazioni_valide(self,ogg1,ogg2):
        relazioni_valide = []
        for relazione in self.IFdoppi_list:
            if relazione.is_True(ogg1,ogg2):
                relazioni_valide.append(relazione)
                #relazione.stampa()
        return relazioni_valide
    
    ########################
    # selezione fposizioni #
    def trova_fposizioni_valide(self, x_or_y, oggetti_input, posizione_x_or_y, coeff_max, max_non_zero_elem, modulo=None, sgn_pos=False, geom=0, shape_geom=(1,1)):
        if coeff_max == 0: #caso speciale <--- prendi solo identita
            #print(oggetti_input[0].posizione)
            if len(oggetti_input)!=1 or oggetti_input[0].posizione is None:
                return []
            rotated_pos = geometry[geom](oggetti_input[0].posizione[0],oggetti_input[0].posizione[1],shape_geom)
            rotated_pos_x_or_y = rotated_pos[0] if x_or_y=='y' else rotated_pos[1]
            if rotated_pos_x_or_y!=posizione_x_or_y:
                return []
            else:
                return [[(1,0)]]
        #funzione ausiliaria per trovare tutti i possibili coefficienti
        def all_sequences_with(length=6, massimo=1, somma=1):
            if length==1:
                if somma is None:
                    return [[valore] for valore in range(-massimo,massimo+1)]
                if somma<=massimo and somma>=-massimo:
                    return [[somma]]
                else:
                    return []
            all_sequences = []
            for valore in range(-massimo,massimo+1):
                new_somma = None if somma is None else somma-valore
                sequences = all_sequences_with(length=length-1, massimo=massimo, somma=new_somma)
                for sequence in sequences:
                    sequence.append(valore)
                all_sequences += sequences
            return all_sequences
        #fine definizione funzione ausiliaria
        if modulo is None:
            if self.shape_output_comeinput:
                fposizioni_raw_pos = all_sequences_with(length=len(oggetti_input), massimo=coeff_max, somma=1)
                fposizioni_raw_dim = all_sequences_with(length=len(oggetti_input), massimo=coeff_max, somma=None)
            elif self.shape_output_costante: #se dim_output=costante togli il vincolo di somma 1 delle posizioni
                fposizioni_raw_pos = all_sequences_with(length=len(oggetti_input), massimo=coeff_max, somma=None)
                fposizioni_raw_dim = all_sequences_with(length=len(oggetti_input), massimo=coeff_max, somma=None)
            elif self.shape_output_comeoggetto: #solo posizione relativa (?)
                fposizioni_raw_pos = all_sequences_with(length=len(oggetti_input), massimo=coeff_max, somma=0)
                fposizioni_raw_dim = all_sequences_with(length=len(oggetti_input), massimo=coeff_max, somma=1)
            else:
                fposizioni_raw_pos = all_sequences_with(length=len(oggetti_input), massimo=coeff_max, somma=None)
                fposizioni_raw_dim = all_sequences_with(length=len(oggetti_input), massimo=coeff_max, somma=None)
        else:
            fposizioni_raw_pos = all_sequences_with(length=len(oggetti_input), massimo=coeff_max, somma=None)
            fposizioni_raw_dim = all_sequences_with(length=len(oggetti_input), massimo=coeff_max, somma=None)
            
        def is_good1(sequenza,max_non_zero_elem=max_non_zero_elem):
            #verifica che non ci siano troppi elementi diversi da 0
            non_zero_elem = 0
            for pair in sequenza:
                for elem in pair:
                    if elem!=0:
                        non_zero_elem += 1
            if non_zero_elem <= max_non_zero_elem:
                return True
            else:
                return False
        def is_good3(sequenza): #NON QUI
            #verifica che non ci siano oggetti che hanno tutti e 2 i coefficienti nulli
            for pair in sequenza:
                if pair==(0,0):
                    return False
            return True
        fposizioni = []
        for seq_pos, seq_dim in product(fposizioni_raw_pos, fposizioni_raw_dim):
            if len(seq_pos)!=len(seq_dim):
                print('aaaaaaaa mannaggia a dio')
            else:
                joint_seq = [None for _ in range(len(oggetti_input))]
                for i in range(len(oggetti_input)):
                    joint_seq[i] = (seq_pos[i],seq_dim[i])
                if is_good1(joint_seq):
                    if posizione_x_or_y is not None:
                        if valuta_fpos(x_or_y, oggetti_input, joint_seq, 0, modulo=modulo, sgn_pos=sgn_pos, geom=geom, shape_geom=shape_geom)==posizione_x_or_y:
                            fposizioni.append(joint_seq)
                        #else:
                        #    print('scarto', joint_seq, '--->',valuta_fpos(x_or_y, oggetti_input, joint_seq, 0, modulo=modulo, sgn_pos=sgn_pos, geom=geom, shape_geom=shape_geom),'vs',posizione_x_or_y)
                    else:
                        fposizioni.append(joint_seq)
        if len(oggetti_input) == 2: #punto medio
            sequenza_punto_medio = [(0.5, 0), (0.5, 0)]
            if posizione_x_or_y is not None:
                if valuta_fpos(x_or_y, oggetti_input, sequenza_punto_medio, 0, sgn_pos=sgn_pos, geom=geom, shape_geom=shape_geom)==posizione_x_or_y:
                    fposizioni.append(sequenza_punto_medio)
            else:
                fposizioni.append(sequenza_punto_medio)
        return fposizioni
    

<h2>"Classe" Preprocessing</h2>
Fa cose

In [ ]:
class Denoiser:
    def __init__(self,tab, iterazioni=2):
        self.tab = tab
        new_tab, _, g, noise_color_eliminato= self.denoisss(tab)
        #print(noise_color_eliminato,iterazioni)
        if not noise_color_eliminato and iterazioni>=2:
            #print('insisto a cazzo tosto evitando',geometry_name[g])
            new_tab, _, g2, noise_color_eliminato = self.denoisss(new_tab, evita_g=[g])
        if not noise_color_eliminato and iterazioni>=3:
            #print('insisto a cazzo tosto evitando',geometry_name[g],'e',geometry_name[g2])
            new_tab, _, _, _ = self.denoisss(new_tab, evita_g=[g,g2])
        '''
        new_tab=tab
        candidati, esiste_un_colore_eliminabile = self.find_candidati_for_tab(tab)
                
        candidato = candidati[0]
        noise_color = candidato[0]
        xyg = candidato[1][0]
        p = xyg[0][0]
        g = xyg[0][1]
        saltate = xyg[1]
        print(noise_color,'--->',saltate,p,geometry_name[g])
        new_tab = self.try_to_denoise(tab,p,g, noise_color)
        mostra_tab(new_tab)
            
        if not esiste_un_colore_eliminabile:
            print('insisto a cazzo tosto evitando',geometry_name[g])
            candidati, esiste_un_colore_eliminabile = self.find_candidati_for_tab(new_tab,evita_g=g)
            candidato = candidati[0]
            noise_color = candidato[0]
            xyg = candidato[1][0]
            p = xyg[0][0]
            g = xyg[0][1]
            saltate = xyg[1]
            print(noise_color,'--->',saltate,p,geometry_name[g])
            new_tab = self.try_to_denoise(new_tab,p,g, noise_color)
            mostra_tab(new_tab)
            
            ######################################## serve triplo per eval 336???
        '''
        self.denoised_tab = new_tab
    
    def denoisss(self,tab,evita_g=None):
        new_tab=tab
        candidati, esiste_un_colore_eliminabile = self.find_candidati_for_tab(tab, evita_g=evita_g)
                
        candidato = candidati[0]
        noise_color = candidato[0]
        xyg = candidato[1][0]
        p = xyg[0][0]
        g = xyg[0][1]
        saltate = xyg[1]
        #print(noise_color,'--->',saltate,p,geometry_name[g])
        new_tab, noise_color_eliminato = self.try_to_denoise(tab,p,g, noise_color)
        #mostra_tab(new_tab)
        return new_tab, p, g, noise_color_eliminato
        
    def get_colors_in_tab(self,tab):
        colori= set()
        for row in tab:
            for c in row:
                colori.add(c)
        return colori
        
    def find_perinv(self,tab,noise_color,evita_g=None):
        shape = (len(tab),len(tab[0]))
        pairs = []
        for Py in range(min(shape[0],10)):
            for Px in range(max(-shape[1]+1,-15),min(shape[1],15)):
                pairs.append((Py,Px))
        pairs.sort(key = lambda x:abs(x[0])+abs(x[1]))

        candidati = []
        esiste_una_che_elimina_nc = False
        for Py,Px in pairs:
            for g in range(8):
                if evita_g is not None and g in evita_g:
                    continue
                if g is 0 and abs(Px)+abs(Py)<=1:
                    continue
                if g is not 0 and abs(Px)+abs(Py)>10:
                    continue
                geom = geometry[g]
                giuste,totali, saltate = 0,0,0
                noise_color_eliminabile = True
                for y,x in product(range(shape[0]),range(shape[1])):
                    new_y,new_x = geom(y,x,shape)
                    if new_y+Py in range(shape[0]) and new_x+Px in range(shape[1]):
                        if (tab[y][x]==noise_color and tab[new_y+Py][new_x+Px]!=noise_color) or (tab[y][x]!=noise_color and tab[new_y+Py][new_x+Px]==noise_color):
                            saltate += 1
                            continue
                        totali += 1
                        if tab[y][x]==tab[new_y+Py][new_x+Px]:
                            giuste += 1
                        if tab[y][x]==noise_color and tab[new_y+Py][new_x+Px]==noise_color:
                            noise_color_eliminabile = False
                    else:
                        if tab[y][x]==noise_color:
                            noise_color_eliminabile = False
                if totali>0 and giuste==totali and saltate>0:
                    if noise_color_eliminabile:
                        esiste_una_che_elimina_nc = True
                    #if noise_color_eliminabile:
                    #    print(Py,Px,g)
                    candidati.append((((Py,Px),g),saltate,noise_color_eliminabile,giuste))
       
        if len(candidati)>0:
            candidati.sort(key = lambda x: x[3] if not x[2] else x[3]+1000000000, reverse=True) #prima i perinv con più copertura(giustetotali) e sempre prima quelli che riescono a eliminare il noise color
            best_copertura = candidati[0][3]
            if not esiste_una_che_elimina_nc:
                candidati = [c[:2] for c in candidati if c[3]==best_copertura]
            else:
                candidati = [c[:2] for c in candidati if c[3]==best_copertura and c[2]]
            return best_copertura, esiste_una_che_elimina_nc, candidati
        else:
            return [0, False, [(((0,0),0), 1000000)]]
    
    def find_candidati_for_tab(self,tab,evita_g=None):
        candidati = []
        esiste_un_colore_eliminabile=False
        for noise_color in self.get_colors_in_tab(tab):
            copertura, elimina_nc, perinv = self.find_perinv(tab,noise_color,evita_g=evita_g)
            #if elimina_nc:
            #    print(copertura,perinv)
            esiste_un_colore_eliminabile = esiste_un_colore_eliminabile or elimina_nc
            #for xyg,saltate in perinv:
            #    print(noise_color,'-->',copertura,elimina_nc,saltate,xyg,geometry_name[xyg[1]])
            candidati.append((noise_color,perinv,copertura,elimina_nc))
        if esiste_un_colore_eliminabile:
            candidati = [c[:3] for c in candidati if c[3]]
            #print('colore eliminabile')
        candidati.sort(key=lambda c:c[2], reverse=True) #prima il/i colori che hanno maggiore copertura
        best_copertura = candidati[0][2]
        #print('massima copertura',best_copertura)
        #candidati = [c[:2] for c in candidati if c[2]==best_copertura]
        candidati = [c[:2] for c in candidati]
        return candidati,esiste_un_colore_eliminabile

    def try_to_denoise(self,tab,p,g, noise_color):
        shape = (len(tab),len(tab[0]))
        new_tab = [[tab[y][x] for x in range(shape[1])] for y in range(shape[0])]
        noise_color_eliminato = True
        for y,x in product(range(shape[0]),range(shape[1])):
            new_y,new_x = geometry[g](y,x,shape)
            new_y += p[0]
            new_x += p[1]
            if -1<new_y<shape[0] and -1<new_x<shape[1]:
                if tab[new_y][new_x]==noise_color and tab[y][x]!=noise_color:
                    new_tab[new_y][new_x] = tab[y][x]
                elif tab[y][x]==noise_color and tab[new_y][new_x]!=noise_color:
                    new_tab[y][x] = tab[new_y][new_x]
                else:
                    new_tab[y][x] = tab[y][x]
                    new_tab[new_y][new_x] = tab[new_y][new_x]
                    if new_tab[y][x]==noise_color or new_tab[new_y][new_x]==noise_color:
                        noise_color_eliminato=False
            if new_tab[y][x]==noise_color:
                noise_color_eliminato=False
        return new_tab, noise_color_eliminato
    

# Esempi facili
su un campione *assolutamente casuale*

In [ ]:
mode = 'train'
if mode=='eval':
    task_path = evaluation_path
    ALL_tasks = evaluation_tasks
elif mode=='train':
    task_path = training_path
    ALL_tasks = training_tasks
    
task_boni = [0,1,15,78,81,86,127,224,227,250,252,278,281,289,371] #43?

for task_da_fare in task_boni:
    print('\n\n\n\n\n\n PROVO TASK ',task_da_fare,'\n')
    task_file = str(task_path / ALL_tasks[task_da_fare])
    with open(task_file, 'r') as f:
        task = json.load(f)
    plot_task(task)
    t = Task(task,comp_connesse_diagonali=True)
    #t_ref = Task(task,comp_connesse_diagonali=True,reflected=True)
    
    s = Solver(t, 200, with_modulo=False,split_come_input=True,add_denoise=None)
    s.solve_task(TEMPO_boost=1, solo_funzioni_generali=True, minimo_numero_immagini=2,STAMPA=False)
    
    
    for fun in s.funzioni[:5]:
        fun.stampa()
    print('TRAIN SCORE: ',s.train_score())
    numero_test = len(task['test'])
    for n_test in range(numero_test):
        guessed_tab = s.applica(n_test)
        mostra_tabs([task['test'][n_test]['output'], [[9]]] + [guessed_tab])

# Esempi tosti
su un campione *assolutamente casuale*

In [ ]:
mode = 'eval'
if mode=='eval':
    task_path = evaluation_path
    ALL_tasks = evaluation_tasks
elif mode=='train':
    task_path = training_path
    ALL_tasks = training_tasks
    
task_boni = [197,48,42,71,83,104,156,255,284,288,338,343]

for task_da_fare in task_boni:
    print('\n\n\n\n\n\n PROVO TASK ',task_da_fare,'\n')
    task_file = str(task_path / ALL_tasks[task_da_fare])
    with open(task_file, 'r') as f:
        task = json.load(f)
    plot_task(task)
    numero_test = len(task['test'])
    guessed_tabs = [[] for _ in range(numero_test)]
    
    t1 = Task(task,comp_connesse_diagonali=True)
    t2 = Task(task,comp_connesse_diagonali=False)
    
    s1 = Solver(t1, 200, with_modulo=False,split_come_input=True,add_denoise=None)
    s2 = Solver(t1, 200, with_modulo=True,split_come_input=True,add_denoise=None)
    s3 = Solver(t1, 200, with_modulo=False,split_come_input=True,add_denoise=2)
    s4 = Solver(t2, 200, with_modulo=False,split_come_input=True,add_denoise=None)
    
    for s in [s1,s2,s3,s4]:
        s.solve_task(TEMPO_boost=1, solo_funzioni_generali=True, minimo_numero_immagini=2,STAMPA=False)
        for fun in s.funzioni[:5]:
            fun.stampa()
        print('TRAIN SCORE: ',s.train_score())
        for n_test in range(numero_test):
            guessed_tab = s.applica(n_test)
            guessed_tabs[n_test].append(guessed_tab)
            
    for n_test in range(numero_test):
        mostra_tabs([task['test'][n_test]['output'], [[9]]] + guessed_tabs[n_test])